In [1]:

# import re
# from pathlib import Path

import pandas as pd
from am_notebooks import *

from source.utils.associate import TOP_AM_DIR
from source.utils.associate import adjust_am_names as adjust_assoc_columns
# from source.utils.dataframes import catify_hit_table as catify
from source.utils.general import HIT_TABLES_DIR, timestamp_today
from source.utils.sample import sample_pickle as sp

REFILTER_NEG = False
VERBOSE = True
K = 8
N_EX_PER_BIGRAM = 99
BIGRAM_F_FLOOR=50
ADV_F_FLOOR=5000

TAG='ALL'
METRIC_PRIORITY = ['LRC', 'P1', 'G2', 'P2'] if TAG=='NEQ' else ['dP1', 'LRC', 'G2', 'P1']
TOP_AM_TAG_DIR = TOP_AM_DIR / TAG
TAG_TOP_STR = f'{TAG}-Top{K}'
OUT_DIR = TOP_AM_TAG_DIR / TAG_TOP_STR
BK = max(K+2, 10)
DATE = timestamp_today()
# FOCUS = adjust_assoc_columns(['f', 
#                               'adv_total', 'adj_total',
#                               'am_p1_given2', 
#                               'am_p1_given2_simple', 
#                               'conservative_log_ratio',
#                               'am_log_likelihood',
#                               'E11', 'unexpected_f',
#                               #   't_score', 'mutual_information', 'am_odds_ratio_disc',
#                               'N', 'f1', 'f2', 
#                               'l1', 'l2', 
#                               'adv', 'adj'
#                               ])
FOCUS=adjust_am_names(FOCUS_DICT[TAG]['polar'])
FOCUS_MEANS = [f'mean_{c}' for c in FOCUS]
SET_FOCUS = [f'{c}_SET' for c in FOCUS]
MIR_FOCUS = [f'{c}_MIR' for c in FOCUS]
pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_colwidth", 70)


NEG_HITS_PARQ = HIT_TABLES_DIR /'RBdirect'/'ALL-RBdirect_final.parq'

In [2]:
PRESAMPLED_PARQ = find_most_recent_top_am(DATE, data_dir=OUT_DIR, undated_stem=f'{TAG_TOP_STR}adv_sample-hits_', suffix='.parq', verbose=VERBOSE)


_inputs_
* 2024-07-31
* /share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8
* ALL-Top8adv_sample-hits_
* .parq
  > seeking... 2024-07-30

* Selected Path Match: `/share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8/ALL-Top8adv_sample-hits_2024-07-30.parq`


_inputs_
* 2024-07-31
* /share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8
* ALL-Top8adv_sample-hits_
* .parq
  > seeking... 2024-07-30

* Selected Path Match: `/share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8/ALL-Top8adv_sample-hits_2024-07-30.parq`


In [3]:

adv_am = seek_top_adv_am(date_str=DATE, adv_floor=ADV_F_FLOOR, tag_top_dir=OUT_DIR)


> Loaded top adv AM table from  
> `/share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8/ALL-Top8_NEG-ADV_combined-5000.2024-07-30.csv`


> Loaded top adv AM table from  
> `/share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8/ALL-Top8_NEG-ADV_combined-5000.2024-07-30.csv`


In [4]:
adv_am = adjust_assoc_columns(adv_am).convert_dtypes()
# nb_show_table(adv_am.filter(SET_FOCUS).filter(regex=r'^[^Nl]'))

```python
nb_show_table(adv_am.filter(SET_FOCUS).filter(regex=r'^[^Nl]'))
```

|                  |   `f_SET` |   `dP1_SET` |   `LRC_SET` |   `P1_SET` |   `G2_SET` |   `f1_SET` |   `f2_SET` |   `exp_f_SET` |   `unexp_f_SET` |   `unexp_r_SET` |   `odds_r_disc_SET` |   `t_SET` |   `MI_SET` |
|:-----------------|----------:|------------:|------------:|-----------:|-----------:|-----------:|-----------:|--------------:|----------------:|----------------:|--------------------:|----------:|-----------:|
| **necessarily**  |    42,595 |        0.83 |        7.10 |       0.87 | 230,256.76 |  3,173,681 |     48,947 |      2,132.66 |       40,462.34 |            0.95 |                2.17 |    196.05 |       1.30 |
| **that**         |   164,768 |        0.75 |        6.34 |       0.79 | 831,134.96 |  3,173,681 |    208,262 |      9,074.15 |      155,693.85 |            0.94 |                1.94 |    383.56 |       1.26 |
| **exactly**      |    43,813 |        0.70 |        5.94 |       0.75 | 210,126.00 |  3,173,681 |     58,643 |      2,555.12 |       41,257.88 |            0.94 |                1.82 |    197.11 |       1.23 |
| **any**          |    15,384 |        0.40 |        4.07 |       0.45 |  50,880.76 |  3,173,681 |     34,382 |      1,498.05 |       13,885.95 |            0.90 |                1.25 |    111.95 |       1.01 |
| **remotely**     |     5,661 |        0.30 |        3.40 |       0.34 |  15,284.42 |  3,173,681 |     16,426 |        715.70 |        4,945.31 |            0.87 |                1.06 |     65.73 |       0.90 |
| **ever**         |     5,932 |        0.01 |        0.16 |       0.05 |     183.91 |  3,173,681 |    114,075 |      4,970.34 |          961.66 |            0.16 |                0.08 |     12.49 |       0.08 |
| **yet**          |    51,867 |        0.50 |        4.65 |       0.54 | 197,610.29 |  3,173,681 |     95,763 |      4,172.47 |       47,694.53 |            0.92 |                1.42 |    209.42 |       1.09 |
| **immediately**  |    56,099 |        0.54 |        4.86 |       0.58 | 224,058.80 |  3,173,681 |     96,973 |      4,225.19 |       51,873.81 |            0.92 |                1.49 |    219.01 |       1.12 |
| **particularly** |    55,527 |        0.07 |        1.38 |       0.11 |  37,272.26 |  3,173,681 |    513,668 |     22,380.94 |       33,146.06 |            0.60 |                0.43 |    140.66 |       0.39 |
| **inherently**   |     6,743 |        0.10 |        1.75 |       0.14 |   7,021.95 |  3,173,681 |     47,803 |      2,082.82 |        4,660.18 |            0.69 |                0.56 |     56.75 |       0.51 |
| **terribly**     |    17,949 |        0.26 |        3.19 |       0.30 |  43,741.22 |  3,173,681 |     58,964 |      2,569.11 |       15,379.89 |            0.86 |                0.98 |    114.80 |       0.84 |



In [5]:
# nb_show_table(adv_am.filter(MIR_FOCUS).filter(regex=r'^[^Nl]'))

```python
nb_show_table(adv_am.filter(MIR_FOCUS).filter(regex=r'^[^Nl]'))
```

|                  |   `f_MIR` |   `dP1_MIR` |   `LRC_MIR` |   `P1_MIR` |   `G2_MIR` |   `f1_MIR` |   `f2_MIR` |   `exp_f_MIR` |   `unexp_f_MIR` |   `unexp_r_MIR` |   `odds_r_disc_MIR` |   `t_MIR` |   `MI_MIR` |
|:-----------------|----------:|------------:|------------:|-----------:|-----------:|-----------:|-----------:|--------------:|----------------:|----------------:|--------------------:|----------:|-----------:|
| **necessarily**  |       963 |        0.70 |        4.39 |       0.87 |   2,597.66 |    291,735 |      1,107 |        189.76 |          773.24 |            0.80 |                1.51 |     24.92 |       0.71 |
| **that**         |     4,308 |        0.61 |        3.90 |       0.78 |   9,957.28 |    291,735 |      5,494 |        941.75 |        3,366.25 |            0.78 |                1.25 |     51.29 |       0.66 |
| **exactly**      |       813 |        0.61 |        3.57 |       0.78 |   1,860.71 |    291,735 |      1,041 |        178.44 |          634.56 |            0.78 |                1.24 |     22.25 |       0.66 |
| **any**          |     1,066 |        0.72 |        4.65 |       0.89 |   2,985.73 |    291,735 |      1,197 |        205.18 |          860.82 |            0.81 |                1.59 |     26.37 |       0.72 |
| **remotely**     |     1,840 |        0.62 |        3.79 |       0.79 |   4,256.31 |    291,735 |      2,341 |        401.28 |        1,438.72 |            0.78 |                1.25 |     33.54 |       0.66 |
| **ever**         |     4,709 |        0.76 |        5.63 |       0.93 |  14,253.47 |    291,735 |      5,060 |        867.36 |        3,841.64 |            0.82 |                1.82 |     55.98 |       0.73 |
| **yet**          |       320 |        0.22 |        1.11 |       0.39 |     223.08 |    291,735 |        815 |        139.70 |          180.30 |            0.56 |                0.50 |     10.08 |       0.36 |
| **immediately**  |       403 |        0.17 |        0.84 |       0.34 |     191.87 |    291,735 |      1,195 |        204.84 |          198.16 |            0.49 |                0.39 |      9.87 |       0.29 |
| **particularly** |     9,243 |        0.54 |        3.43 |       0.71 |  18,583.63 |    291,735 |     13,003 |      2,228.90 |        7,014.10 |            0.76 |                1.09 |     72.96 |       0.62 |
| **inherently**   |     2,864 |        0.39 |        2.40 |       0.56 |   3,925.26 |    291,735 |      5,133 |        879.87 |        1,984.13 |            0.69 |                0.79 |     37.08 |       0.51 |
| **terribly**     |     1,567 |        0.17 |        1.09 |       0.34 |     764.42 |    291,735 |      4,610 |        790.22 |          776.78 |            0.50 |                0.40 |     19.62 |       0.30 |



In [6]:
# nb_show_table(adv_am.filter(items=FOCUS_MEANS).filter(regex=r'^[^Nl]'))

```python
nb_show_table(adv_am.filter(items=FOCUS_MEANS).filter(regex=r'^[^Nl]'))
```

|                  |   `mean_f` |   `mean_dP1` |   `mean_LRC` |   `mean_P1` |   `mean_G2` |   `mean_f1` |   `mean_f2` |   `mean_N` |   `mean_t` |   `mean_MI` |
|:-----------------|-----------:|-------------:|-------------:|------------:|------------:|------------:|------------:|-----------:|-----------:|------------:|
| **necessarily**  | 593,171.33 |         0.76 |         5.74 |        0.87 |  116,427.21 |   1,732,708 |   25,027.00 | 37,270,750 |     110.48 |        1.00 |
| **that**         | 641,374.67 |         0.68 |         5.12 |        0.79 |  420,546.12 |   1,732,708 |  106,878.00 | 37,270,750 |     217.42 |        0.96 |
| **exactly**      | 594,954.33 |         0.66 |         4.76 |        0.76 |  105,993.36 |   1,732,708 |   29,842.00 | 37,270,750 |     109.68 |        0.95 |
| **any**          | 586,240.83 |         0.56 |         4.36 |        0.67 |   26,933.25 |   1,732,708 |   17,789.50 | 37,270,750 |      69.16 |        0.86 |
| **remotely**     | 581,947.33 |         0.46 |         3.59 |        0.57 |    9,770.36 |   1,732,708 |    9,383.50 | 37,270,750 |      49.63 |        0.78 |
| **ever**         | 599,198.67 |         0.39 |         2.90 |        0.49 |    7,218.69 |   1,732,708 |   59,567.50 | 37,270,750 |      34.23 |        0.41 |
| **yet**          | 602,363.50 |         0.36 |         2.88 |        0.47 |   98,916.68 |   1,732,708 |   48,289.00 | 37,270,750 |     109.75 |        0.73 |
| **immediately**  | 603,347.67 |         0.35 |         2.85 |        0.46 |  112,125.34 |   1,732,708 |   49,084.00 | 37,270,750 |     114.44 |        0.71 |
| **particularly** | 676,142.83 |         0.30 |         2.40 |        0.41 |   27,927.95 |   1,732,708 |  263,335.50 | 37,270,750 |     106.81 |        0.51 |
| **inherently**   | 587,993.17 |         0.24 |         2.07 |        0.35 |    5,473.60 |   1,732,708 |   26,468.00 | 37,270,750 |      46.91 |        0.51 |
| **terribly**     | 591,417.67 |         0.21 |         2.14 |        0.32 |   22,252.82 |   1,732,708 |   31,787.00 | 37,270,750 |      67.21 |        0.57 |



In [7]:
# compare_datasets(adv_am, METRIC_PRIORITY[0])

```py
compare_datasets(adv_am, METRIC_PRIORITY[0])
```

Top 5 by descending `dP1_SET`

|                 |   `dP1_SET` |   `dP1_MIR` |   `mean_dP1` |
|:----------------|------------:|------------:|-------------:|
| **necessarily** |       0.827 |       0.699 |        0.763 |
| **that**        |       0.750 |       0.615 |        0.682 |
| **exactly**     |       0.704 |       0.610 |        0.657 |
| **immediately** |       0.536 |       0.166 |        0.351 |
| **yet**         |       0.499 |       0.221 |        0.360 |

Top 5 by descending `dP1_MIR`

|                 |   `dP1_SET` |   `dP1_MIR` |   `mean_dP1` |
|:----------------|------------:|------------:|-------------:|
| **ever**        |       0.008 |       0.761 |        0.385 |
| **any**         |       0.404 |       0.720 |        0.562 |
| **necessarily** |       0.827 |       0.699 |        0.763 |
| **remotely**    |       0.301 |       0.615 |        0.458 |
| **that**        |       0.750 |       0.615 |        0.682 |

Top 5 by descending `mean_dP1`

|                 |   `dP1_SET` |   `dP1_MIR` |   `mean_dP1` |
|:----------------|------------:|------------:|-------------:|
| **necessarily** |       0.827 |       0.699 |        0.763 |
| **that**        |       0.750 |       0.615 |        0.682 |
| **exactly**     |       0.704 |       0.610 |        0.657 |
| **any**         |       0.404 |       0.720 |        0.562 |
| **remotely**    |       0.301 |       0.615 |        0.458 |



In [8]:
# compare_datasets(adv_am, METRIC_PRIORITY[1])

```py
compare_datasets(adv_am, METRIC_PRIORITY[1])
```
Top 5 by descending `LRC_SET`

|                 |   `LRC_SET` |   `LRC_MIR` |   `mean_LRC` |
|:----------------|------------:|------------:|-------------:|
| **necessarily** |         7.1 |         4.4 |          5.7 |
| **that**        |         6.3 |         3.9 |          5.1 |
| **exactly**     |         5.9 |         3.6 |          4.8 |
| **immediately** |         4.9 |         0.8 |          2.9 |
| **yet**         |         4.6 |         1.1 |          2.9 |

Top 5 by descending `LRC_MIR`

|                 |   `LRC_SET` |   `LRC_MIR` |   `mean_LRC` |
|:----------------|------------:|------------:|-------------:|
| **ever**        |         0.2 |         5.6 |          2.9 |
| **any**         |         4.1 |         4.7 |          4.4 |
| **necessarily** |         7.1 |         4.4 |          5.7 |
| **that**        |         6.3 |         3.9 |          5.1 |
| **remotely**    |         3.4 |         3.8 |          3.6 |

Top 5 by descending `mean_LRC`

|                 |   `LRC_SET` |   `LRC_MIR` |   `mean_LRC` |
|:----------------|------------:|------------:|-------------:|
| **necessarily** |         7.1 |         4.4 |          5.7 |
| **that**        |         6.3 |         3.9 |          5.1 |
| **exactly**     |         5.9 |         3.6 |          4.8 |
| **any**         |         4.1 |         4.7 |          4.4 |
| **remotely**    |         3.4 |         3.8 |          3.6 |



In [9]:
# compare_datasets(adv_am, METRIC_PRIORITY[2])

```python
compare_datasets(adv_am, METRIC_PRIORITY[2])
```

Top 5 by descending `G2_SET`

|                 |  `G2_SET` | `G2_MIR` | `mean_G2` |
|:----------------|----------:|---------:|----------:|
| **that**        | 831,135.0 |  9,957.3 | 420,546.1 |
| **necessarily** | 230,256.8 |  2,597.7 | 116,427.2 |
| **immediately** | 224,058.8 |    191.9 | 112,125.3 |
| **exactly**     | 210,126.0 |  1,860.7 | 105,993.4 |
| **yet**         | 197,610.3 |    223.1 |  98,916.7 |

Top 5 by descending `G2_MIR`

|                  |  `G2_SET` | `G2_MIR` | `mean_G2` |
|:-----------------|----------:|---------:|----------:|
| **particularly** |  37,272.3 | 18,583.6 |  27,927.9 |
| **ever**         |     183.9 | 14,253.5 |   7,218.7 |
| **that**         | 831,135.0 |  9,957.3 | 420,546.1 |
| **remotely**     |  15,284.4 |  4,256.3 |   9,770.4 |
| **inherently**   |   7,022.0 |  3,925.3 |   5,473.6 |

Top 5 by descending `mean_G2`

|                 |  `G2_SET` | `G2_MIR` | `mean_G2` |
|:----------------|----------:|---------:|----------:|
| **that**        | 831,135.0 |  9,957.3 | 420,546.1 |
| **necessarily** | 230,256.8 |  2,597.7 | 116,427.2 |
| **immediately** | 224,058.8 |    191.9 | 112,125.3 |
| **exactly**     | 210,126.0 |  1,860.7 | 105,993.4 |
| **yet**         | 197,610.3 |    223.1 |  98,916.7 |



In [10]:
# compare_datasets(adv_am, 'f_')

```python
compare_datasets(adv_am, 'f_')
```

Top 5 by descending `f_SET`

|                  |   `f_SET` |   `exp_f_SET` |   `unexp_f_SET` |   `f_MIR` |   `exp_f_MIR` |   `unexp_f_MIR` |   `%_f_MIR` |
|:-----------------|----------:|--------------:|----------------:|----------:|--------------:|----------------:|------------:|
| **that**         |   164,768 |       9,074.1 |       155,693.9 |     4,308 |         941.8 |         3,366.2 |         2.6 |
| **immediately**  |    56,099 |       4,225.2 |        51,873.8 |       403 |         204.8 |           198.2 |         0.7 |
| **particularly** |    55,527 |      22,380.9 |        33,146.1 |     9,243 |       2,228.9 |         7,014.1 |        16.7 |
| **yet**          |    51,867 |       4,172.5 |        47,694.5 |       320 |         139.7 |           180.3 |         0.6 |
| **exactly**      |    43,813 |       2,555.1 |        41,257.9 |       813 |         178.4 |           634.6 |         1.9 |

Top 5 by descending `exp_f_SET`

|                  |   `f_SET` |   `exp_f_SET` |   `unexp_f_SET` |   `f_MIR` |   `exp_f_MIR` |   `unexp_f_MIR` |   `%_f_MIR` |
|:-----------------|----------:|--------------:|----------------:|----------:|--------------:|----------------:|------------:|
| **particularly** |    55,527 |      22,380.9 |        33,146.1 |     9,243 |       2,228.9 |         7,014.1 |        16.7 |
| **that**         |   164,768 |       9,074.1 |       155,693.9 |     4,308 |         941.8 |         3,366.2 |         2.6 |
| **ever**         |     5,932 |       4,970.3 |           961.7 |     4,709 |         867.4 |         3,841.6 |        79.4 |
| **immediately**  |    56,099 |       4,225.2 |        51,873.8 |       403 |         204.8 |           198.2 |         0.7 |
| **yet**          |    51,867 |       4,172.5 |        47,694.5 |       320 |         139.7 |           180.3 |         0.6 |

Top 5 by descending `unexp_f_SET`

|                 |   `f_SET` |   `exp_f_SET` |   `unexp_f_SET` |   `f_MIR` |   `exp_f_MIR` |   `unexp_f_MIR` |   `%_f_MIR` |
|:----------------|----------:|--------------:|----------------:|----------:|--------------:|----------------:|------------:|
| **that**        |   164,768 |       9,074.1 |       155,693.9 |     4,308 |         941.8 |         3,366.2 |         2.6 |
| **immediately** |    56,099 |       4,225.2 |        51,873.8 |       403 |         204.8 |           198.2 |         0.7 |
| **yet**         |    51,867 |       4,172.5 |        47,694.5 |       320 |         139.7 |           180.3 |         0.6 |
| **exactly**     |    43,813 |       2,555.1 |        41,257.9 |       813 |         178.4 |           634.6 |         1.9 |
| **necessarily** |    42,595 |       2,132.7 |        40,462.3 |       963 |         189.8 |           773.2 |         2.3 |

Top 5 by descending `f_MIR`

|                  |   `f_SET` |   `exp_f_SET` |   `unexp_f_SET` |   `f_MIR` |   `exp_f_MIR` |   `unexp_f_MIR` |   `%_f_MIR` |
|:-----------------|----------:|--------------:|----------------:|----------:|--------------:|----------------:|------------:|
| **particularly** |    55,527 |      22,380.9 |        33,146.1 |     9,243 |       2,228.9 |         7,014.1 |        16.7 |
| **ever**         |     5,932 |       4,970.3 |           961.7 |     4,709 |         867.4 |         3,841.6 |        79.4 |
| **that**         |   164,768 |       9,074.1 |       155,693.9 |     4,308 |         941.8 |         3,366.2 |         2.6 |
| **inherently**   |     6,743 |       2,082.8 |         4,660.2 |     2,864 |         879.9 |         1,984.1 |        42.5 |
| **remotely**     |     5,661 |         715.7 |         4,945.3 |     1,840 |         401.3 |         1,438.7 |        32.5 |

Top 5 by descending `exp_f_MIR`

|                  |   `f_SET` |   `exp_f_SET` |   `unexp_f_SET` |   `f_MIR` |   `exp_f_MIR` |   `unexp_f_MIR` |   `%_f_MIR` |
|:-----------------|----------:|--------------:|----------------:|----------:|--------------:|----------------:|------------:|
| **particularly** |    55,527 |      22,380.9 |        33,146.1 |     9,243 |       2,228.9 |         7,014.1 |        16.7 |
| **that**         |   164,768 |       9,074.1 |       155,693.9 |     4,308 |         941.8 |         3,366.2 |         2.6 |
| **inherently**   |     6,743 |       2,082.8 |         4,660.2 |     2,864 |         879.9 |         1,984.1 |        42.5 |
| **ever**         |     5,932 |       4,970.3 |           961.7 |     4,709 |         867.4 |         3,841.6 |        79.4 |
| **terribly**     |    17,949 |       2,569.1 |        15,379.9 |     1,567 |         790.2 |           776.8 |         8.7 |

Top 5 by descending `unexp_f_MIR`

|                  |   `f_SET` |   `exp_f_SET` |   `unexp_f_SET` |   `f_MIR` |   `exp_f_MIR` |   `unexp_f_MIR` |   `%_f_MIR` |
|:-----------------|----------:|--------------:|----------------:|----------:|--------------:|----------------:|------------:|
| **particularly** |    55,527 |      22,380.9 |        33,146.1 |     9,243 |       2,228.9 |         7,014.1 |        16.7 |
| **ever**         |     5,932 |       4,970.3 |           961.7 |     4,709 |         867.4 |         3,841.6 |        79.4 |
| **that**         |   164,768 |       9,074.1 |       155,693.9 |     4,308 |         941.8 |         3,366.2 |         2.6 |
| **inherently**   |     6,743 |       2,082.8 |         4,660.2 |     2,864 |         879.9 |         1,984.1 |        42.5 |
| **remotely**     |     5,661 |         715.7 |         4,945.3 |     1,840 |         401.3 |         1,438.7 |        32.5 |

Top 5 by descending `%_f_MIR`

|                  |   `f_SET` |   `exp_f_SET` |   `unexp_f_SET` |   `f_MIR` |   `exp_f_MIR` |   `unexp_f_MIR` |   `%_f_MIR` |
|:-----------------|----------:|--------------:|----------------:|----------:|--------------:|----------------:|------------:|
| **ever**         |     5,932 |       4,970.3 |           961.7 |     4,709 |         867.4 |         3,841.6 |        79.4 |
| **inherently**   |     6,743 |       2,082.8 |         4,660.2 |     2,864 |         879.9 |         1,984.1 |        42.5 |
| **remotely**     |     5,661 |         715.7 |         4,945.3 |     1,840 |         401.3 |         1,438.7 |        32.5 |
| **particularly** |    55,527 |      22,380.9 |        33,146.1 |     9,243 |       2,228.9 |         7,014.1 |        16.7 |
| **terribly**     |    17,949 |       2,569.1 |        15,379.9 |     1,567 |         790.2 |           776.8 |         8.7 |



In [11]:
# compare_datasets(adv_am, 'f2')

```python
compare_datasets(adv_am, 'f2')
```
Top 5 by descending `f2_SET`

|                  |   `f2_SET` |   `f2_MIR` |   `mean_f2` |   `%_f2_MIR` |
|:-----------------|-----------:|-----------:|------------:|-------------:|
| **particularly** |    513,668 |     13,003 |   263,335.5 |          2.5 |
| **that**         |    208,262 |      5,494 |   106,878.0 |          2.6 |
| **ever**         |    114,075 |      5,060 |    59,567.5 |          4.4 |
| **immediately**  |     96,973 |      1,195 |    49,084.0 |          1.2 |
| **yet**          |     95,763 |        815 |    48,289.0 |          0.9 |

Top 5 by descending `f2_MIR`

|                  |   `f2_SET` |   `f2_MIR` |   `mean_f2` |   `%_f2_MIR` |
|:-----------------|-----------:|-----------:|------------:|-------------:|
| **particularly** |    513,668 |     13,003 |   263,335.5 |          2.5 |
| **that**         |    208,262 |      5,494 |   106,878.0 |          2.6 |
| **inherently**   |     47,803 |      5,133 |    26,468.0 |         10.7 |
| **ever**         |    114,075 |      5,060 |    59,567.5 |          4.4 |
| **terribly**     |     58,964 |      4,610 |    31,787.0 |          7.8 |

Top 5 by descending `mean_f2`

|                  |   `f2_SET` |   `f2_MIR` |   `mean_f2` |   `%_f2_MIR` |
|:-----------------|-----------:|-----------:|------------:|-------------:|
| **particularly** |    513,668 |     13,003 |   263,335.5 |          2.5 |
| **that**         |    208,262 |      5,494 |   106,878.0 |          2.6 |
| **ever**         |    114,075 |      5,060 |    59,567.5 |          4.4 |
| **immediately**  |     96,973 |      1,195 |    49,084.0 |          1.2 |
| **yet**          |     95,763 |        815 |    48,289.0 |          0.9 |

Top 5 by descending `%_f2_MIR`

|                |   `f2_SET` |   `f2_MIR` |   `mean_f2` |   `%_f2_MIR` |
|:---------------|-----------:|-----------:|------------:|-------------:|
| **remotely**   |     16,426 |      2,341 |     9,383.5 |         14.2 |
| **inherently** |     47,803 |      5,133 |    26,468.0 |         10.7 |
| **terribly**   |     58,964 |      4,610 |    31,787.0 |          7.8 |
| **ever**       |    114,075 |      5,060 |    59,567.5 |          4.4 |
| **any**        |     34,382 |      1,197 |    17,789.5 |          3.5 |



In [12]:

TOP_ADV, adv_am = pin_top_adv(adv_am, select_col=f'mean_{METRIC_PRIORITY[0]}')

Top Adverb Selection, ranked by descending `'mean_dP1'`

|    | adv          |   mean_dP1 |
|---:|:-------------|-----------:|
|  1 | necessarily  |      0.763 |
|  2 | that         |      0.682 |
|  3 | exactly      |      0.657 |
|  4 | any          |      0.562 |
|  5 | remotely     |      0.458 |
|  6 | ever         |      0.385 |
|  7 | yet          |      0.360 |
|  8 | immediately  |      0.351 |
|  9 | particularly |      0.304 |
| 10 | inherently   |      0.243 |
| 11 | terribly     |      0.215 |



```python
TOP_ADV, adv_am = pin_top_adv(adv_am, select_col=f'mean_{METRIC_PRIORITY[0]}')
```
Top Adverb Selection, ranked by descending `'mean_dP1'`

|    | adv          |   mean_dP1 |
|---:|:-------------|-----------:|
|  1 | necessarily  |      0.763 |
|  2 | that         |      0.682 |
|  3 | exactly      |      0.657 |
|  4 | any          |      0.562 |
|  5 | remotely     |      0.458 |
|  6 | ever         |      0.385 |
|  7 | yet          |      0.360 |
|  8 | immediately  |      0.351 |
|  9 | particularly |      0.304 |
| 10 | inherently   |      0.243 |
| 11 | terribly     |      0.215 |



In [13]:
# __ = pin_top_adv(adv_am, select_col=[f'mean_{m}' for m in METRIC_PRIORITY])

```python
__ = pin_top_adv(adv_am, select_col=[f'mean_{m}' for m in METRIC_PRIORITY])
```
Top Adverb Selection, ranked by descending `['mean_dP1', 'mean_LRC', 'mean_G2', 'mean_P1']`

|    | adv          |   mean_dP1 |   mean_LRC |     mean_G2 |   mean_P1 |
|---:|:-------------|-----------:|-----------:|------------:|----------:|
|  1 | necessarily  |      0.763 |      5.743 | 116,427.210 |     0.870 |
|  2 | that         |      0.682 |      5.117 | 420,546.121 |     0.788 |
|  3 | exactly      |      0.657 |      4.759 | 105,993.356 |     0.764 |
|  4 | any          |      0.562 |      4.360 |  26,933.246 |     0.669 |
|  5 | remotely     |      0.458 |      3.594 |   9,770.362 |     0.565 |
|  6 | ever         |      0.385 |      2.895 |   7,218.690 |     0.491 |
|  7 | yet          |      0.360 |      2.877 |  98,916.683 |     0.467 |
|  8 | immediately  |      0.351 |      2.852 | 112,125.335 |     0.458 |
|  9 | particularly |      0.304 |      2.403 |  27,927.947 |     0.409 |
| 10 | inherently   |      0.243 |      2.075 |   5,473.603 |     0.349 |
| 11 | terribly     |      0.215 |      2.140 |  22,252.819 |     0.322 |



In [14]:
undated_stem = f'{TAG_TOP_STR}_NEG-ADV-{ADV_F_FLOOR}_Top{BK}-bigrams-{BIGRAM_F_FLOOR}'
top_polar_bigram_am_csv = find_most_recent_top_am(DATE, OUT_DIR, undated_stem)
print(f'\n* Loading from `{top_polar_bigram_am_csv}`')


* Loading from `/share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8/ALL-Top8_NEG-ADV-5000_Top10-bigrams-50.2024-07-30.csv`



* Loading from `/share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8/ALL-Top8_NEG-ADV-5000_Top10-bigrams-50.2024-07-30.csv`


In [15]:
polar_bigram_am = adjust_assoc_columns(pd.read_csv(top_polar_bigram_am_csv)
             .set_index('key')
             #> not strictly necessary (loaded table should already satisfy this) but just in case...
             .filter(regex=r'|'.join([f'~{a}_' for a in TOP_ADV]), axis=0)
             ).filter(items=FOCUS).convert_dtypes()
polar_bigram_am.sort_values(METRIC_PRIORITY, ascending=False)

,f,dP1,LRC,P1,G2,l1,l2,f1,f2,N,exp_f,unexp_f,unexp_r,odds_r_disc,t,MI,adv,adv_total,adj,adj_total
key,,,,,,,,,,,,,,,,,,,,
NEGany~yet_eligible,448,0.96,8.96,1.00,"2,807.56",NEGATED,yet_eligible,3173681,448,72839571,19.52,428.48,0.96,4.29,20.24,1.36,yet,95763,eligible,23252
NEGany~yet_official,352,0.96,8.61,1.00,"2,205.93",NEGATED,yet_official,3173681,352,72839571,15.34,336.66,0.96,4.19,17.94,1.36,yet,95763,official,6853
NEGany~exactly_conducive,208,0.96,7.82,1.00,"1,303.49",NEGATED,exactly_conducive,3173681,208,72839571,9.06,198.94,0.96,3.96,13.79,1.36,exactly,58643,conducive,9110
NEGany~yet_convinced,169,0.96,7.50,1.00,"1,059.09",NEGATED,yet_convinced,3173681,169,72839571,7.36,161.64,0.96,3.87,12.43,1.36,yet,95763,convinced,12132
NEGany~exactly_shocking,151,0.96,7.33,1.00,946.28,NEGATED,exactly_shocking,3173681,151,72839571,6.58,144.42,0.96,3.82,11.75,1.36,exactly,58643,shocking,35115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NEGany~remotely_interested,330,0.27,2.74,0.31,817.05,NEGATED,remotely_interested,3173681,1062,72839571,46.27,283.73,0.86,1.00,15.62,0.85,remotely,16426,interested,264528
NEGany~remotely_similar,152,0.23,2.20,0.27,334.61,NEGATED,remotely_similar,3173681,559,72839571,24.36,127.64,0.84,0.91,10.35,0.80,remotely,16426,similar,203453
NEGany~inherently_good,283,0.20,2.21,0.24,554.72,NEGATED,inherently_good,3173681,1177,72839571,51.28,231.72,0.82,0.84,13.77,0.74,inherently,47803,good,1681795


In [16]:
overall_k = int(BK/2 * K)
overall_md_name = f'{TAG}-Top{K}_NEG-ADV-{ADV_F_FLOOR}_top{overall_k}bigrams-overall.min{BIGRAM_F_FLOOR}.{timestamp_today()}.md'
nb_show_table(
    polar_bigram_am.round(2).nlargest(overall_k, columns=METRIC_PRIORITY),
    outpath=OUT_DIR / overall_md_name,
    suppress_printing=not VERBOSE)

print(f'Saved as  \n`{OUT_DIR / overall_md_name}`')


|                                       |    `f` |   `dP1` |   `LRC` |   `P1` |      `G2` | `l1`    | `l2`                       |      `f1` |   `f2` |        `N` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `odds_r_disc` |   `t` |   `MI` | `adv`       |   `adv_total` | `adj`          |   `adj_total` |
|:--------------------------------------|-------:|--------:|--------:|-------:|----------:|:--------|:---------------------------|----------:|-------:|-----------:|----------:|------------:|------------:|----------------:|------:|-------:|:------------|--------------:|:---------------|--------------:|
| **NEGany~yet_eligible**               |    448 |    0.96 |    8.96 |   1.00 |  2,807.56 | NEGATED | yet_eligible               | 3,173,681 |    448 | 72,839,571 |     19.52 |      428.48 |        0.96 |            4.29 | 20.24 |   1.36 | yet         |        95,763 | eligible       |        23,252 |
| **NEGany~yet_official**               |    352 |    0.96 |    8.61 |   1.00 |  2,205.93 

```python
overall_k = int(BK/2 * K)
overall_md_name = f'{TAG}-Top{K}_NEG-ADV-{ADV_F_FLOOR}_top{overall_k}bigrams-overall.min{BIGRAM_F_FLOOR}.{timestamp_today()}.md'
nb_show_table(
    polar_bigram_am.round(2).nlargest(overall_k, columns=METRIC_PRIORITY),
    outpath=OUT_DIR / overall_md_name,
    suppress_printing=not VERBOSE)
```

|                                       |    `f` |   `dP1` |   `LRC` |   `P1` |      `G2` | `l1`    | `l2`                       |      `f1` |   `f2` |        `N` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `odds_r_disc` |   `t` |   `MI` | `adv`       |   `adv_total` | `adj`          |   `adj_total` |
|:--------------------------------------|-------:|--------:|--------:|-------:|----------:|:--------|:---------------------------|----------:|-------:|-----------:|----------:|------------:|------------:|----------------:|------:|-------:|:------------|--------------:|:---------------|--------------:|
| **NEGany~yet_eligible**               |    448 |    0.96 |    8.96 |   1.00 |  2,807.56 | NEGATED | yet_eligible               | 3,173,681 |    448 | 72,839,571 |     19.52 |      428.48 |        0.96 |            4.29 | 20.24 |   1.36 | yet         |        95,763 | eligible       |        23,252 |
| **NEGany~yet_official**               |    352 |    0.96 |    8.61 |   1.00 |  2,205.93 | NEGATED | yet_official               | 3,173,681 |    352 | 72,839,571 |     15.34 |      336.66 |        0.96 |            4.19 | 17.94 |   1.36 | yet         |        95,763 | official       |         6,853 |
| **NEGany~exactly_conducive**          |    208 |    0.96 |    7.82 |   1.00 |  1,303.49 | NEGATED | exactly_conducive          | 3,173,681 |    208 | 72,839,571 |      9.06 |      198.94 |        0.96 |            3.96 | 13.79 |   1.36 | exactly     |        58,643 | conducive      |         9,110 |
| **NEGany~yet_convinced**              |    169 |    0.96 |    7.50 |   1.00 |  1,059.09 | NEGATED | yet_convinced              | 3,173,681 |    169 | 72,839,571 |      7.36 |      161.64 |        0.96 |            3.87 | 12.43 |   1.36 | yet         |        95,763 | convinced      |        12,132 |
| **NEGany~exactly_shocking**           |    151 |    0.96 |    7.33 |   1.00 |    946.28 | NEGATED | exactly_shocking           | 3,173,681 |    151 | 72,839,571 |      6.58 |      144.42 |        0.96 |            3.82 | 11.75 |   1.36 | exactly     |        58,643 | shocking       |        35,115 |
| **NEGany~exactly_pleasant**           |    142 |    0.96 |    7.24 |   1.00 |    889.88 | NEGATED | exactly_pleasant           | 3,173,681 |    142 | 72,839,571 |      6.19 |      135.81 |        0.96 |            3.80 | 11.40 |   1.36 | exactly     |        58,643 | pleasant       |        52,223 |
| **NEGany~exactly_famous**             |    130 |    0.96 |    7.10 |   1.00 |    814.68 | NEGATED | exactly_famous             | 3,173,681 |    130 | 72,839,571 |      5.66 |      124.34 |        0.96 |            3.76 | 10.90 |   1.36 | exactly     |        58,643 | famous         |       223,813 |
| **NEGany~exactly_difficult**          |    126 |    0.96 |    7.05 |   1.00 |    789.61 | NEGATED | exactly_difficult          | 3,173,681 |    126 | 72,839,571 |      5.49 |      120.51 |        0.96 |            3.74 | 10.74 |   1.36 | exactly     |        58,643 | difficult      |       732,106 |
| **NEGany~necessarily_useful**         |    104 |    0.96 |    6.75 |   1.00 |    651.74 | NEGATED | necessarily_useful         | 3,173,681 |    104 | 72,839,571 |      4.53 |       99.47 |        0.96 |            3.66 |  9.75 |   1.36 | necessarily |        48,947 | useful         |       227,709 |
| **NEGany~yet_online**                 |     98 |    0.96 |    6.66 |   1.00 |    614.14 | NEGATED | yet_online                 | 3,173,681 |     98 | 72,839,571 |      4.27 |       93.73 |        0.96 |            3.64 |  9.47 |   1.36 | yet         |        95,763 | online         |        15,650 |
| **NEGany~necessarily_fun**            |     96 |    0.96 |    6.62 |   1.00 |    601.61 | NEGATED | necessarily_fun            | 3,173,681 |     96 | 72,839,571 |      4.18 |       91.82 |        0.96 |            3.63 |  9.37 |   1.36 | necessarily |        48,947 | fun            |       190,026 |
| **NEGany~necessarily_essential**      |     93 |    0.96 |    6.57 |   1.00 |    582.81 | NEGATED | necessarily_essential      | 3,173,681 |     93 | 72,839,571 |      4.05 |       88.95 |        0.96 |            3.61 |  9.22 |   1.36 | necessarily |        48,947 | essential      |        69,845 |
| **NEGany~necessarily_reliable**       |     81 |    0.96 |    6.35 |   1.00 |    507.61 | NEGATED | necessarily_reliable       | 3,173,681 |     81 | 72,839,571 |      3.53 |       77.47 |        0.96 |            3.55 |  8.61 |   1.36 | necessarily |        48,947 | reliable       |        90,598 |
| **NEGany~necessarily_proud**          |     71 |    0.96 |    6.14 |   1.00 |    444.94 | NEGATED | necessarily_proud          | 3,173,681 |     71 | 72,839,571 |      3.09 |       67.91 |        0.96 |            3.50 |  8.06 |   1.36 | necessarily |        48,947 | proud          |       207,536 |
| **NEGany~yet_mainstream**             |     70 |    0.96 |    6.11 |   1.00 |    438.67 | NEGATED | yet_mainstream             | 3,173,681 |     70 | 72,839,571 |      3.05 |       66.95 |        0.96 |            3.49 |  8.00 |   1.36 | yet         |        95,763 | mainstream     |        17,792 |
| **NEGany~that_thrilled**              |     59 |    0.96 |    5.83 |   1.00 |    369.74 | NEGATED | that_thrilled              | 3,173,681 |     59 | 72,839,571 |      2.57 |       56.43 |        0.96 |            3.42 |  7.35 |   1.36 | that        |       208,262 | thrilled       |        24,182 |
| **NEGany~that_far-fetched**           |     59 |    0.96 |    5.83 |   1.00 |    369.74 | NEGATED | that_far-fetched           | 3,173,681 |     59 | 72,839,571 |      2.57 |       56.43 |        0.96 |            3.42 |  7.35 |   1.36 | that        |       208,262 | far-fetched    |         5,185 |
| **NEGany~yet_clear**                  | 10,406 |    0.95 |   10.77 |   0.99 | 64,409.83 | NEGATED | yet_clear                  | 3,173,681 | 10,476 | 72,839,571 |    456.45 |    9,949.55 |        0.96 |            3.51 | 97.54 |   1.36 | yet         |        95,763 | clear          |       349,214 |
| **NEGany~necessarily_indicative**     |  1,389 |    0.95 |    9.43 |   0.99 |  8,577.52 | NEGATED | necessarily_indicative     | 3,173,681 |  1,400 | 72,839,571 |     61.00 |    1,328.00 |        0.96 |            3.42 | 35.63 |   1.36 | necessarily |        48,947 | indicative     |         8,148 |
| **NEGany~that_uncommon**              |    802 |    0.95 |    9.43 |   1.00 |  4,998.31 | NEGATED | that_uncommon              | 3,173,681 |    804 | 72,839,571 |     35.03 |      766.97 |        0.96 |            3.85 | 27.08 |   1.36 | that        |       208,262 | uncommon       |        11,312 |
| **NEGany~exactly_easy**               |  1,066 |    0.95 |    9.32 |   0.99 |  6,596.90 | NEGATED | exactly_easy               | 3,173,681 |  1,073 | 72,839,571 |     46.75 |    1,019.25 |        0.96 |            3.49 | 31.22 |   1.36 | exactly     |        58,643 | easy           |       579,827 |
| **NEGany~that_surprising**            |  1,133 |    0.95 |    9.20 |   0.99 |  6,986.80 | NEGATED | that_surprising            | 3,173,681 |  1,143 | 72,839,571 |     49.80 |    1,083.20 |        0.96 |            3.37 | 32.18 |   1.36 | that        |       208,262 | surprising     |        70,540 |
| **NEGany~necessarily_easy**           |    909 |    0.95 |    9.01 |   0.99 |  5,605.63 | NEGATED | necessarily_easy           | 3,173,681 |    917 | 72,839,571 |     39.95 |      869.05 |        0.96 |            3.37 | 28.82 |   1.36 | necessarily |        48,947 | easy           |       579,827 |
| **NEGany~yet_final**                  |    640 |    0.95 |    8.97 |   1.00 |  3,972.91 | NEGATED | yet_final                  | 3,173,681 |    643 | 72,839,571 |     28.02 |      611.98 |        0.96 |            3.60 | 24.19 |   1.36 | yet         |        95,763 | final          |         5,860 |
| **NEGany~exactly_cheap**              |    691 |    0.95 |    8.96 |   0.99 |  4,281.58 | NEGATED | exactly_cheap              | 3,173,681 |    695 | 72,839,571 |     30.28 |      660.72 |        0.96 |            3.53 | 25.13 |   1.36 | exactly     |        58,643 | cheap          |        60,531 |
| **NEGany~that_unusual**               |    977 |    0.95 |    8.92 |   0.99 |  6,003.04 | NEGATED | that_unusual               | 3,173,681 |    988 | 72,839,571 |     43.05 |      933.95 |        0.96 |            3.27 | 29.88 |   1.36 | that        |       208,262 | unusual        |        71,234 |
| **NEGany~exactly_surprising**         |    440 |    0.95 |    8.71 |   1.00 |  2,743.33 | NEGATED | exactly_surprising         | 3,173,681 |    441 | 72,839,571 |     19.21 |      420.79 |        0.96 |            3.81 | 20.06 |   1.36 | exactly     |        58,643 | surprising     |        70,540 |
| **NEGany~necessarily_representative** |    487 |    0.95 |    8.34 |   0.99 |  2,996.58 | NEGATED | necessarily_representative | 3,173,681 |    492 | 72,839,571 |     21.44 |      465.56 |        0.96 |            3.29 | 21.10 |   1.36 | necessarily |        48,947 | representative |        18,355 |
| **NEGany~necessarily_surprising**     |    340 |    0.95 |    8.33 |   1.00 |  2,117.15 | NEGATED | necessarily_surprising     | 3,173,681 |    341 | 72,839,571 |     14.86 |      325.14 |        0.96 |            3.70 | 17.63 |   1.36 | necessarily |        48,947 | surprising     |        70,540 |
| **NEGany~that_dissimilar**            |    304 |    0.95 |    7.86 |   0.99 |  1,871.64 | NEGATED | that_dissimilar            | 3,173,681 |    307 | 72,839,571 |     13.38 |      290.62 |        0.96 |            3.28 | 16.67 |   1.36 | that        |       208,262 | dissimilar     |         4,605 |
| **NEGany~that_noticeable**            |    264 |    0.95 |    7.65 |   0.99 |  1,621.81 | NEGATED | that_noticeable            | 3,173,681 |    267 | 72,839,571 |     11.63 |      252.37 |        0.96 |            3.22 | 15.53 |   1.36 | that        |       208,262 | noticeable     |        31,467 |
| **NEGany~yet_over**                   |    162 |    0.95 |    7.21 |   0.99 |  1,003.13 | NEGATED | yet_over                   | 3,173,681 |    163 | 72,839,571 |      7.10 |      154.90 |        0.96 |            3.38 | 12.17 |   1.36 | yet         |        95,763 | over           |         3,983 |
| **NEGany~yet_ready**                  |  7,501 |    0.94 |    9.93 |   0.99 | 45,984.97 | NEGATED | yet_ready                  | 3,173,681 |  7,599 | 72,839,571 |    331.09 |    7,169.91 |        0.96 |            3.22 | 82.79 |   1.36 | yet         |        95,763 | ready          |       141,590 |
| **NEGany~yet_complete**               |  2,174 |    0.94 |    9.20 |   0.98 | 13,277.06 | NEGATED | yet_complete               | 3,173,681 |  2,208 | 72,839,571 |     96.20 |    2,077.80 |        0.96 |            3.14 | 44.56 |   1.35 | yet         |        95,763 | complete       |        86,361 |
| **NEGany~exactly_new**                |  1,371 |    0.94 |    9.10 |   0.99 |  8,410.30 | NEGATED | exactly_new                | 3,173,681 |  1,388 | 72,839,571 |     60.48 |    1,310.52 |        0.96 |            3.24 | 35.39 |   1.36 | exactly     |        58,643 | new            |       253,862 |
| **NEGany~exactly_clear**              |  1,746 |    0.94 |    8.78 |   0.98 | 10,578.31 | NEGATED | exactly_clear              | 3,173,681 |  1,784 | 72,839,571 |     77.73 |    1,668.27 |        0.96 |            3.00 | 39.92 |   1.35 | exactly     |        58,643 | clear          |       349,214 |
| **NEGany~that_complicated**           |  1,207 |    0.94 |    8.68 |   0.98 |  7,337.82 | NEGATED | that_complicated           | 3,173,681 |  1,230 | 72,839,571 |     53.59 |    1,153.41 |        0.96 |            3.05 | 33.20 |   1.35 | that        |       208,262 | complicated    |       159,822 |
| **NEGany~terribly_surprising**        |    949 |    0.94 |    8.66 |   0.98 |  5,794.08 | NEGATED | terribly_surprising        | 3,173,681 |    964 | 72,839,571 |     42.00 |      907.00 |        0.96 |            3.13 | 29.44 |   1.35 | terribly    |        58,964 | surprising     |        70,540 |
| **NEGany~necessarily_new**            |    482 |    0.94 |    7.94 |   0.98 |  2,923.81 | NEGATED | necessarily_new            | 3,173,681 |    492 | 72,839,571 |     21.44 |      460.56 |        0.96 |            3.00 | 20.98 |   1.35 | necessarily |        48,947 | new            |       253,862 |
| **NEGany~terribly_uncommon**          |    103 |    0.94 |    6.33 |   0.98 |    625.85 | NEGATED | terribly_uncommon          | 3,173,681 |    105 | 72,839,571 |      4.57 |       98.43 |        0.96 |            2.96 |  9.70 |   1.35 | terribly    |        58,964 | uncommon       |        11,312 |

Saved as  
`/share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8/ALL-Top8_NEG-ADV-5000_top40bigrams-overall.min50.2024-07-31.md`


### Load Negated Hit Table

In [17]:
if NEG_HITS_PARQ.suffix.startswith('.parq'):
    neg_hits = pd.read_parquet(NEG_HITS_PARQ,
                               filters=[('adv_form_lower', 'in', TOP_ADV)])
elif 'pkl' in NEG_HITS_PARQ.suffixes:
    neg_hits = pd.read_pickle(NEG_HITS_PARQ)

    neg_hits = neg_hits.loc[(neg_hits.adv_lemma.isin(TOP_ADV))
                            | (neg_hits.adv_form_lower.isin(TOP_ADV)), :]
if VERBOSE:
    print(neg_hits.neg_lemma.value_counts().to_frame('subtotal in loaded negated hits sample').to_markdown(intfmt=','))

neg_hits = neg_hits.filter(
    regex=r'^[nab].*lower|text|token|g_head|(adv|neg)_lemma'
).drop_duplicates(['text_window', 'all_forms_lower'])
neg_hits = clarify_neg_categories(neg_hits)

| neg_lemma   |   subtotal in loaded negated hits sample |
|:------------|-----------------------------------------:|
| not         |                                  123,151 |
| nothing     |                                    3,233 |
| never       |                                    1,811 |
| none        |                                    1,107 |
| nor         |                                      556 |
| neither     |                                      448 |
| hardly      |                                      329 |
| nobody      |                                      216 |
| rarely      |                                      158 |
| without     |                                      102 |
| ain't       |                                       82 |
| few         |                                       56 |
| barely      |                                       23 |
| no          |                                       20 |
| scarcely    |                                       17

Updated `neg_lemma` counts with "ain(')t" merged with "not"

| neg_lemma   |   subtotal in loaded negated hits sample |
|:------------|-----------------------------------------:|
| not         |                                  123,151 |
| nothing     |                                    3,233 |
| never       |                                    1,811 |
| none        |                                    1,107 |
| nor         |                                      556 |
| neither     |                                      448 |
| hardly      |                                      329 |
| nobody      |                                      216 |
| rarely      |                                      158 |
| without     |                                      102 |
| ain't       |                                       82 |
| few         |                                       56 |
| barely      |                                       23 |
| no          |                                       20 |
| scarcely    |                                       17 |
| aint        |                                       12 |
| seldom      |                                       10 |


In [18]:
# nb_show_table(neg_hits.assign(sent_text = '*'+ neg_hits.sent_text +'*', 
#                               hit_text = '*'+ neg_hits.hit_text +'*').sample(2).filter(regex=r'^[^t]').T)
nb_show_table(neg_hits.sample(2), transpose=True)


|                       | `pcc_eng_09_082.5708_x1319642_114:5-6-7`                                                   | `pcc_eng_17_051.2970_x0812660_35:49-50-51`                                                                                                                                                                                                                           |
|:----------------------|:-------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`sent_text`**       | *His latest album wasn't that good, but he's not as bad as people make him out to be.*     | *The massive internet outage in October, for example, was traced back to'smart' home gadgets; drones, while cool for


|                       | `pcc_eng_13_046.9733_x0743284_083:10-12-13`                                         | `apw_eng_20090211_0896_4:3-4-5`                                      |
|:----------------------|:------------------------------------------------------------------------------------|:---------------------------------------------------------------------|
| **`sent_text`**       | *I went seven innings and my right arm had not been that tired in my entire life.*  | *It was not immediately clear what distinction Suleman was making.*  |
| **`hit_text`**        | *not been that tired in my entire life*                                             | *not immediately clear what distinction Suleman was*                 |
| **`text_window`**     | *and my right arm had not been that tired in my entire life .*                      | *it was not immediately clear what distinction suleman was making .* |
| **`token_str`**       | *I went seven innings and my right arm had not been that tired in my entire life .* | *it was not immediately clear what distinction Suleman was making .* |
| **`neg_head`**        | ADJ                                                                                 | ADJ                                                                  |
| **`neg_lemma`**       | not                                                                                 | not                                                                  |
| **`adv_lemma`**       | that                                                                                | immediately                                                          |
| **`adv_form_lower`**  | that                                                                                | immediately                                                          |
| **`adj_form_lower`**  | tired                                                                               | clear                                                                |
| **`bigram_lower`**    | that_tired                                                                          | immediately_clear                                                    |
| **`neg_form_lower`**  | not                                                                                 | not                                                                  |
| **`all_forms_lower`** | not_that_tired                                                                      | not_immediately_clear                                                |



In [19]:
if PRESAMPLED_PARQ.exists():
    all_hits = pd.read_parquet(PRESAMPLED_PARQ, engine='pyarrow')
    


In [20]:
from source.utils.dataframes import get_preceding_text #, NEG_REGEX


pos_hits = all_hits.copy().filter(like='pcc_eng_28', axis=0
                                  ).loc[all_hits.polarity=='pos', :]
pos_hits['up_to_adj'] = get_preceding_text(pos_hits.token_str)
show_sample(pos_hits.sample(6).filter(like='adj'))


+-----------------------+------------------+------------------------------------------------------------------------------------------------------------------------------------------+
| hit_id                | adj_form_lower   | up_to_adj                                                                                                                                |
+=======================+==================+==========================================================================================================================================+
| pcc_eng_28_084.1231_x | true             | this was particularly                                                                                                                    |
| 1344711_03:3-4        |                  |                                                                                                                                          |
+-----------------------+------------------+------------------------------------

In [21]:
import re
NEG_REGEX = re.compile(
    (r"\b(?P<no>no)\b|\b(?P<noX>n[o'](?:[tr]|ne?|body|thing|where))\b"
        r"|\b(?<!\w a|the|[oe]se) (?P<few>few)\b|\b(?P<NEGly>(?:[br]are|scarce|seldom)l?y?|hardly)\b"
        r"|\b(?P<aint>ain'?t)\b|\b(?P<neither>neither)\b|\b(?P<without>without)\b|\b(?P<never>never)\b"),
    flags=re.I)

X = 'No one seemed exactly sure what to expect , and looked around in silent acknowledgement that it was a slightly absurd way to spend a Saturday afternoon'
NEG_REGEX.findall(X.lower())

[('no', '', '', '', '', '', '', '')]

In [22]:
neg_found = pos_hits.up_to_adj.str.extract(NEG_REGEX).fillna('')
neg_found


,no,noX,few,NEGly,aint,neither,without,never
hit_id,,,,,,,,
pcc_eng_28_015.4396_x0233796_099:3-4,,,,,,,,
pcc_eng_28_016.0330_x0243369_347:23-24,no,,,,,,,
pcc_eng_28_010.0076_x0145563_19:20-21,,,,,,,,
pcc_eng_28_010.1860_x0148524_56:34-35,no,,,,,,,
pcc_eng_28_010.3043_x0150491_90:5-6,,,,,,,,
...,...,...,...,...,...,...,...,...
pcc_eng_28_083.7694_x1339025_15:10-11,,,,,,,,
pcc_eng_28_084.0500_x1343556_24:4-5,,,,,,,,
pcc_eng_28_084.1739_x1345538_7:13-14,,,,,,,,


In [23]:

neg_found = neg_found.loc[neg_found.apply(any, axis=1), neg_found.apply(any)]
neg_found

,no
hit_id,
pcc_eng_28_016.0330_x0243369_347:23-24,no
pcc_eng_28_010.1860_x0148524_56:34-35,no
pcc_eng_28_001.1964_x0003234_11:22-23,no
pcc_eng_28_002.2570_x0020362_10:09-10,no
pcc_eng_28_049.2065_x0779984_52:16-17,no
...,...
pcc_eng_28_032.5231_x0509686_17:4-5,no
pcc_eng_28_012.5343_x0186736_12:31-32,no
pcc_eng_28_032.9117_x0515978_03:7-8,no


In [24]:
neg_found = neg_found.assign(adv=pos_hits.loc[neg_found.index, 'adv_form_lower'],
                               all_forms_lower= pos_hits.loc[neg_found.index,'all_forms_lower'], 
                             preceding_str=embolden(pos_hits.loc[neg_found.index,'up_to_adj']))
nb_show_table(neg_found, 
              outpath=f'/share/compling/projects/sanpi/results/MISSED_NEG-in-COM.{DATE}.md')


|                                             | `no`   | `adv`        | `all_forms_lower`            | `preceding_str`                                                                                                                                                                                                                                              |
|:--------------------------------------------|:-------|:-------------|:-----------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **pcc_eng_28_016.0330_x0243369_347:23-24**  | no     | any          | (+)_any_deeper               | whoever he was -- perhaps one of club bwua'tu 's more sinister operatives -- he clearly had __`no`__ wish to involve himself any                                                    

In [25]:
print(neg_found.iloc[:, :-2].value_counts().to_frame().reset_index().to_markdown(index=None))

| no   | adv          |   count |
|:-----|:-------------|--------:|
| no   | particularly |      32 |
| no   | ever         |      25 |
| no   | any          |      20 |
| no   | exactly      |      20 |
| no   | immediately  |      10 |
| no   | that         |       9 |
| no   | inherently   |       7 |
| no   | yet          |       7 |
| no   | remotely     |       7 |
| no   | terribly     |       3 |
| no   | necessarily  |       2 |


In [26]:
pd.set_option('display.max_rows', 180)
neg_found.groupby('adv').value_counts(['no']).to_frame()

/tmp/ipykernel_6723/2730695706.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  neg_found.groupby('adv').value_counts(['no']).to_frame()


,,count
adv,no,
any,no,20
ever,no,25
exactly,no,20
immediately,no,10
inherently,no,7
necessarily,no,2
particularly,no,32
remotely,no,7
terribly,no,3


In [27]:
pos_hits[pos_hits.token_str.str.findall(NEG_REGEX).astype('bool')]

,token_str,text_window,bigram_lower,adj_form_lower,all_forms_lower,neg_lemma,polarity,adv_form_lower,up_to_adj
hit_id,,,,,,,,,
pcc_eng_28_015.4396_x0233796_099:3-4,"He isnt any better at creating his own offense than Paul , and is ...",he isnt any better at creating his own offense than,any_better,better,(+)_any_better,,pos,any,he isnt any
pcc_eng_28_016.0330_x0243369_347:23-24,Whoever he was -- perhaps one of Club Bwua'tu 's more sinister ope...,clearly had no wish to involve himself any deeper than he already ...,any_deeper,deeper,(+)_any_deeper,,pos,any,whoever he was -- perhaps one of club bwua'tu 's more sinister ope...
pcc_eng_28_010.1860_x0148524_56:34-35,"Learn everything about the craft , but in the end you just have to...",in no uncertain terms whether you 're any good or not !,any_good,good,(+)_any_good,,pos,any,"learn everything about the craft , but in the end you just have to..."
pcc_eng_28_010.3043_x0150491_90:5-6,"Wonder if he feels any better about that ( well , given his reacti...","wonder if he feels any better about that ( well , given",any_better,better,(+)_any_better,,pos,any,wonder if he feels any
pcc_eng_28_001.1964_x0003234_11:22-23,"But even if we were following Lee Reagan 's advice , there 's no g...",good reason to believe we 'd be any healthier .,any_healthier,healthier,(+)_any_healthier,,pos,any,"but even if we were following lee reagan 's advice , there 's no g..."
...,...,...,...,...,...,...,...,...,...
pcc_eng_28_007.3949_x0103294_44:3-4,"Riled up yet devoid of all energy - how it 's possible , I 'm not ...",riled up yet devoid of all energy - how it,yet_devoid,devoid,(+)_yet_devoid,,pos,yet,riled up yet
pcc_eng_28_032.9117_x0515978_03:7-8,There 's been no easy and yet fast way to test the functionality .,there 's been no easy and yet fast way to test the functionality .,yet_fast,fast,(+)_yet_fast,,pos,yet,there 's been no easy and yet
pcc_eng_28_055.7684_x0886388_11:6-7,"Your grip should be light yet supportive , and there should be no ...","your grip should be light yet supportive , and there should be no",yet_supportive,supportive,(+)_yet_supportive,,pos,yet,your grip should be light yet


In [28]:
show_sample(all_hits.filter(['polarity', 'all_forms_lower', 'text_window', 'token_str']
                            ).sample(5).sort_values('all_forms_lower'),
            format='simple_grid', 
            width_override={'text_window':40, 'token_str':50})


┌───────────────────────┬────────────┬──────────────────────────┬──────────────────────────────────────────┬────────────────────────────────────────────────────┐
│ hit_id                │ polarity   │ all_forms_lower          │ text_window                              │ token_str                                          │
├───────────────────────┼────────────┼──────────────────────────┼──────────────────────────────────────────┼────────────────────────────────────────────────────┤
│ pcc_eng_01_042.6284_x │ pos        │ (+)_ever_further         │ of " global capitalism " is an ever      │ The reality of " global capitalism " is an ever    │
│ 0672650_102:10-11     │            │                          │ further concentration of wealth and      │ further concentration of wealth and power until    │
│                       │            │                          │ power until                              │ the bottom rebels , and then everyone has to       │
│                       │   

In [29]:

# sourcery skip: use-fstring-for-concatenation
if 'all_forms_lower' not in neg_hits.columns: 
    neg_hits['all_forms_lower'] = (
        neg_hits.neg_form_lower.astype('string') 
        + '_' 
        + neg_hits.bigram_lower.astype('string')
        ).astype('category')

In [30]:
neg_hits.loc[neg_hits.adv_lemma.astype('string') 
             != neg_hits.adv_form_lower.astype('string')
             ].filter(regex=r'adv|window')

,text_window,adv_lemma,adv_form_lower
hit_id,,,


In [31]:
if VERBOSE:
    print(neg_hits.adv_form_lower.value_counts()
          .to_frame('Tokens in loaded *negated* sample')
          .to_markdown(floatfmt=',.0f', intfmt=','))
    for pol, pol_hits in all_hits.groupby('polarity'): 
        print(f'\n ### `{pol}` adv subtotals\n')
        print(pol_hits.adv_form_lower.value_counts()
              .to_frame('Tokens in loaded sample')
              .to_markdown(floatfmt=',.0f', intfmt=','))

| adv_form_lower   |   Tokens in loaded *negated* sample |
|:-----------------|------------------------------------:|
| that             |                              45,368 |
| particularly     |                              15,535 |
| immediately      |                              14,728 |
| yet              |                              14,575 |
| exactly          |                              12,315 |
| necessarily      |                              11,679 |
| terribly         |                               4,887 |
| any              |                               4,246 |
| inherently       |                               1,901 |
| ever             |                               1,698 |
| remotely         |                               1,633 |

 ### `neg` adv subtotals

| adv_form_lower   |   Tokens in loaded sample |
|:-----------------|--------------------------:|
| that             |                    45,367 |
| particularly     |                    15,535 |
| immediat

/tmp/ipykernel_6723/1028066531.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for pol, pol_hits in all_hits.groupby('polarity'):


```py
if VERBOSE:
    print(neg_hits.adv_form_lower.value_counts()
          .to_frame('Tokens in loaded *negated* sample')
          .to_markdown(floatfmt=',.0f', intfmt=','))
    for pol, pol_hits in all_hits.groupby('polarity'): 
        print(f'\n ### `{pol}` adv subtotals\n')
        print(pol_hits.adv_form_lower.value_counts()
              .to_frame('Tokens in loaded sample')
              .to_markdown(floatfmt=',.0f', intfmt=','))
```
| adv_form_lower   |   Tokens in loaded *negated* sample |
|:-----------------|------------------------------------:|
| that             |                              45,368 |
| particularly     |                              15,535 |
| immediately      |                              14,728 |
| yet              |                              14,575 |
| exactly          |                              12,315 |
| necessarily      |                              11,679 |
| terribly         |                               4,887 |
| any              |                               4,246 |
| inherently       |                               1,901 |
| ever             |                               1,698 |
| remotely         |                               1,633 |

 ### `neg` adv subtotals

| adv_form_lower   |   Tokens in loaded sample |
|:-----------------|--------------------------:|
| that             |                    45,367 |
| particularly     |                    15,535 |
| immediately      |                    14,728 |
| yet              |                    14,575 |
| exactly          |                    12,315 |
| necessarily      |                    11,678 |
| terribly         |                     4,887 |
| any              |                     4,246 |
| inherently       |                     1,901 |
| ever             |                     1,698 |
| remotely         |                     1,633 |

 ### `pos` adv subtotals

| adv_form_lower   |   Tokens in loaded sample |
|:-----------------|--------------------------:|
| particularly     |                    85,806 |
| ever             |                    19,815 |
| yet              |                     8,371 |
| that             |                     8,187 |
| terribly         |                     7,875 |
| immediately      |                     7,758 |
| inherently       |                     7,727 |
| any              |                     3,702 |
| exactly          |                     2,732 |
| remotely         |                     2,023 |
| necessarily      |                     1,213 |

In [32]:
if VERBOSE:
    fewer = sp(data=all_hits, regex=True, print_sample=False,
           columns=['WITH::all|text'], 
           filters=['all_forms_lower==_?fewer_'])
    nb_show_table(fewer.assign(text_window=embolden(fewer.text_window, r' (fewer) ')), adjust_columns=False)


- *filtering rows...*
  - regex parsing = True
  - ✓ Applied filter: `all_forms_lower==_?fewer_`

### All (1) row(s) matching filter(s) from `input frame`


|                                          | `text_window`                                                             | `all_forms_lower`      |
|:-----------------------------------------|:--------------------------------------------------------------------------|:-----------------------|
| **pcc_eng_04_001.4775_x0007747_3:4-6-7** | in 2d far__`fewer`__are exactly solvable , the simplest being a rectangle | fewer_exactly_solvable |



```python
if VERBOSE:
    fewer = sp(data=neg_hits, regex=True, print_sample=False,
           columns=['WITH::bigram|neg|str'], 
           filters=['neg_form_lower==fewer'])
    nb_show_table(fewer.assign(token_str=embolden(fewer.token_str, r' (fewer) ')), adjust_columns=False)
```

- *filtering rows...*
  - regex parsing = True
  - ✓ Applied filter: `neg_form_lower==fewer`

### All (1) row(s) matching filter(s) from `input frame`


|                                          | `token_str`                                                                                                      | `neg_head`   | `neg_lemma`   | `bigram_lower`   | `neg_form_lower`   |
|:-----------------------------------------|:-----------------------------------------------------------------------------------------------------------------|:-------------|:--------------|:-----------------|:-------------------|
| **pcc_eng_04_001.4775_x0007747_3:4-6-7** | In 2D far __`fewer`__ are exactly solvable , the simplest being a rectangle with Dirichlet boundary conditions . | ADJ          | few           | exactly_solvable | fewer              |


In [33]:
if VERBOSE:
    rare_forms = neg_hits.neg_form_lower.value_counts().nsmallest(6).index
    # nb_show_table(neg_hits.loc[neg_hits.neg_form_lower.isin(rare_forms), :].sort_values('neg_form_lower').filter(regex=r'bigram|neg|text'))
    show_sample(neg_hits.loc[neg_hits.neg_form_lower.isin(rare_forms), :].sort_values('neg_form_lower').filter(regex=r'all_forms|win|sent'), width_override={'sent_text':65}, format='simple_grid')

┌───────────────────────┬───────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────┬─────────────────────────────────┐
│ hit_id                │ sent_text                                                         │ text_window                                      │ all_forms_lower                 │
├───────────────────────┼───────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────┼─────────────────────────────────┤
│ pcc_eng_04_001.4775_x │ In 2D far fewer are exactly solvable, the simplest being a        │ in 2d far fewer are exactly solvable , the       │ fewer_exactly_solvable          │
│ 0007747_3:4-6-7       │ rectangle with Dirichlet boundary conditions.                     │ simplest being a rectangle                       │                                 │
├───────────────────────┼───────────────────────────────────────────────────────────────────┼────────────

In [39]:
if VERBOSE:
    _sample = neg_hits.loc[
        ((neg_hits.neg_form_lower != "n't")
         & (neg_hits.neg_lemma.astype('string')
            != neg_hits.neg_form_lower.astype('string'))),
        ['neg_lemma', 'neg_form_lower', 'text_window']
    ].sample(10).sort_values('neg_form_lower')

    nb_show_table(
        _sample.assign(
            text_window=embolden(
                _sample.text_window,
                bold_regex=r'(\b'+r'\b|\b'.join(_sample.neg_form_lower)+r')\b')))


|                                              | `neg_lemma`   | `neg_form_lower`   | `text_window`                                                                          |
|:---------------------------------------------|:--------------|:-------------------|:---------------------------------------------------------------------------------------|
| **pcc_eng_24_024.8274_x0385234_38:09-10-11** | not           | ain't              | coming from the feel that this __`ain't`__ exactly real , or it 's real ,              |
| **pcc_eng_07_059.5261_x0946050_05:19-20-21** | not           | ain't              | happening here , what it is __`ain't`__ exactly clear " in a scene about vietnam       |
| **pcc_eng_04_075.9094_x1209755_083:4-5-6**   | not           | ain't              | honestly , it __`ain't`__ that much of a secret when you 're                           |
| **pcc_eng_01_047.3006_x0748159_56:7-8-9**    | not           | ain't              | well , some of us girls __`ain't`__ th

```python
if VERBOSE:
    _sample = neg_hits.loc[
        ((neg_hits.neg_form_lower != "n't")
         & (neg_hits.neg_lemma.astype('string')
            != neg_hits.neg_form_lower.astype('string'))),
        ['neg_lemma', 'neg_form_lower', 'text_window']
    ].sample(10).sort_values('neg_form_lower')

    nb_show_table(
        _sample.assign(
            text_window=embolden(
                _sample.text_window,
                bold_regex=r'(\b'+r'\b|\b'.join(_sample.neg_form_lower)+r')\b')))
```                   

|                                              | `neg_lemma` | `neg_form_lower` | `text_window`                                                                          |
|:---------------------------------------------|:------------|:-----------------|:---------------------------------------------------------------------------------------|
| **pcc_eng_24_024.8274_x0385234_38:09-10-11** | not         | ain't            | coming from the feel that this __`ain't`__ exactly real , or it 's real ,              |
| **pcc_eng_07_059.5261_x0946050_05:19-20-21** | not         | ain't            | happening here , what it is __`ain't`__ exactly clear " in a scene about vietnam       |
| **pcc_eng_04_075.9094_x1209755_083:4-5-6**   | not         | ain't            | honestly , it __`ain't`__ that much of a secret when you 're                           |
| **pcc_eng_01_047.3006_x0748159_56:7-8-9**    | not         | ain't            | well , some of us girls __`ain't`__ that good with electric stuff .                    |
| **pcc_eng_23_033.6484_x0527066_058:4-5-6**   | not         | ain't            | Which means it __`ain't`__ that funny at all .                                         |
| **pcc_eng_10_027.6125_x0430065_28:36-37-38** | not         | ain't            | in a fancy restaurants because they __`ain't`__ that famous .                          |
| **pcc_eng_07_059.2522_x0941539_20:08-09-10** | not         | nit              | auto dealersi argument for exclusivity is __`nit`__ terribly persuasive .              |
| **pcc_eng_29_011.2389_x0165615_028:5-7-8**   | not         | nit              | the non-conference schedule is __`nit`__ all that bad after starting out the season at |
| **pcc_eng_03_007.1447_x0099289_31:27-28-29** | not         | nit              | track record for big announcements is __`nit`__ that great .                           |
| **pcc_eng_18_002.1227_x0018259_110:3-4-5**   | not         | nt               | This is __`nt`__ that much of a hardship .                                             |



In [41]:
if VERBOSE:
    _sample = neg_hits.loc[
        ((neg_hits.neg_lemma != "not")
         & (neg_hits.neg_lemma.astype('string') != neg_hits.neg_form_lower.astype('string'))),
        ['neg_lemma', 'neg_form_lower', 'text_window']].sort_values('neg_form_lower')
    
    nb_show_table(
        _sample.assign(
            text_window=embolden(
                _sample.text_window,
                bold_regex=r'(\b'+r'\b|\b'.join(_sample.neg_form_lower)+r')\b')))


|                                               | `neg_lemma`   | `neg_form_lower`   | `text_window`                                                                           |
|:----------------------------------------------|:--------------|:-------------------|:----------------------------------------------------------------------------------------|
| **pcc_eng_04_001.4775_x0007747_3:4-6-7**      | few           | fewer              | in 2d far __`fewer`__ are exactly solvable , the simplest being a rectangle             |
| **pcc_eng_20_080.0820_x1277728_116:20-23-24** | nobody        | nobodies           | away from the story __`nobodies`__ dick is that big and if it was having a              |
| **pcc_eng_09_031.5011_x0493764_43:04-12-13**  | nothing       | nothings           | And the ' __`nothings`__ ' that these people talk about are inherently postmodern too . |



```python
if VERBOSE:
    _sample = neg_hits.loc[
        ((neg_hits.neg_lemma != "not")
         & (neg_hits.neg_lemma.astype('string') != neg_hits.neg_form_lower.astype('string'))),
        ['neg_lemma', 'neg_form_lower', 'text_window']].sort_values('neg_form_lower')
    
    nb_show_table(
        _sample.assign(
            text_window=embolden(
                _sample.text_window,
                bold_regex=r'(\b'+r'\b|\b'.join(_sample.neg_form_lower)+r')\b')))
```

|                                               | `neg_lemma`   | `neg_form_lower`   | `text_window`                                                                           |
|:----------------------------------------------|:--------------|:-------------------|:----------------------------------------------------------------------------------------|
| **pcc_eng_04_001.4775_x0007747_3:4-6-7**      | few           | fewer              | in 2d far __`fewer`__ are exactly solvable , the simplest being a rectangle             |
| **pcc_eng_20_080.0820_x1277728_116:20-23-24** | nobody        | nobodies           | away from the story __`nobodies`__ dick is that big and if it was having a              |
| **pcc_eng_09_031.5011_x0493764_43:04-12-13**  | nothing       | nothings           | And the ' __`nothings`__ ' that these people talk about are inherently postmodern too . |



In [42]:
if VERBOSE:
    print('Weird example, but illustrates structural relationship which is probably not caught by the patterns for accurate parses: Possessive quantified pronoun in subject', 
      neg_hits.loc[neg_hits.neg_form_lower=='nobodies', ['neg_lemma', 'neg_form_lower', 'bigram_lower', 'text_window', 'token_str']].T.to_markdown(), 
      sep='\n\n')

Weird example, but illustrates structural relationship which is probably not caught by the patterns for accurate parses: Possessive quantified pronoun in subject

|                | pcc_eng_20_080.0820_x1277728_116:20-23-24                                                                                                                                                                                                                                                                                                                                                       |
|:---------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| neg_lemma      | nobody  

Weird example, but illustrates structural relationship which is probably not caught by the patterns for accurate parses: Possessive quantified pronoun in subject

|                | pcc_eng_20_080.0820_x1277728_116:20-23-24                                                                                                                                                                                                                                                                                                                                                       |
|:---------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| neg_lemma      | nobody                                                                                                                                                                                                                                                                                                                                                                                          |
| neg_form_lower | nobodies                                                                                                                                                                                                                                                                                                                                                                                        |
| bigram_lower   | that_big                                                                                                                                                                                                                                                                                                                                                                                        |
| text_window    | away from the story nobodies dick is that big and if it was                                                                                                                                                                                                                                                                                                                                     |
| token_str      | really the size of the dick is totally a lie and is just a turn away from the story nobodies dick is that big and if it was having a bonor for 2 hours would most certianly kill you because all of the blood would got to your dick and you would die so this is just a bull story and i hate how people make there dicks that big it is just not right and makes it absolutely unbelievable . |


## 📌 Was working _here_

- [ ] modifiy `populate_adv_dir` and `collect_adv_bigram_ex` to extract examples for _pos_ tokens, as well as _neg_

In [45]:
adverb = 'terribly'
#BUG
populate_adv_dir(
        adverb,
        bigram_am=polar_bigram_am, 
        adv_ex_stem= f'{TAG}-{adverb}_{BK}mostNEG-bigrams-{BIGRAM_F_FLOOR}_AMscores',
        hits_df=neg_hits,
        data_dir=OUT_DIR, 
        rank_by=METRIC_PRIORITY,
        n_bigrams=BK, 
        n_ex=N_EX_PER_BIGRAM,
        verbose=VERBOSE)

KeyError: 'P1'

In [ ]:

print(f'# {BK} Most Negative Bigrams for each of the {K} Most Negative Adverbs\n')

# TODO: update this to pull examples from both `polarity` values in `all_hits`
for rank, adverb in enumerate(adv_am.index, start=1):
    print(f'\n## {rank}. *{adverb}*')
    populate_adv_dir(
        adverb,
        bigram_am=polar_bigram_am, 
        hits_df=neg_hits,
        data_dir=OUT_DIR, rank_by=['dP1', 'LRC'],
        n_bigrams=BK, n_ex=N_EX_PER_BIGRAM,
        verbose=VERBOSE)

## TODO 🚧 ☑️
- [ ] update to pull examples of **both** polarities, and save separately


# 10 Most Negative Bigrams for each of the 8 Most Negative Adverbs


## 1. *necessarily*

|                         |        `N` |      `f1` |   `adv_total` |
|:------------------------|-----------:|----------:|--------------:|
| **NEGATED_necessarily** | 72,839,589 | 3,173,660 |        48,947 |
| **NEGMIR_necessarily**  |  1,701,929 |   291,732 |         1,107 |


|                                       |   `f` |   `adj_total` |   `dP1` |   `LRC` |     `G2` |   `exp_f` |   `unexp_f` |   `f2` |
|:--------------------------------------|------:|--------------:|--------:|--------:|---------:|----------:|------------:|-------:|
| **NEGany~necessarily_useful**         |   104 |       227,709 |    0.96 |    6.75 |   651.75 |      4.53 |       99.47 |    104 |
| **NEGany~necessarily_fun**            |    96 |       190,026 |    0.96 |    6.62 |   601.61 |      4.18 |       91.82 |     96 |
| **NEGany~necessarily_essential**      |    93 |        69,845 |    0.96 |    6.57 |   582.81 |      4.05 |       88.95 |     93 |
| **NEGany~necessarily_reliable**       |    81 |        90,598 |    0.96 |    6.35 |   507.61 |      3.53 |       77.47 |     81 |
| **NEGany~necessarily_proud**          |    71 |       207,536 |    0.96 |    6.14 |   444.94 |      3.09 |       67.91 |     71 |
| **NEGany~necessarily_indicative**     | 1,389 |         8,148 |    0.95 |    9.43 | 8,577.54 |     61.00 |    1,328.00 |  1,400 |
| **NEGany~necessarily_easy**           |   909 |       579,827 |    0.95 |    9.01 | 5,605.64 |     39.95 |      869.05 |    917 |
| **NEGany~necessarily_representative** |   487 |        18,355 |    0.95 |    8.34 | 2,996.58 |     21.44 |      465.56 |    492 |
| **NEGany~necessarily_surprising**     |   340 |        70,540 |    0.95 |    8.33 | 2,117.16 |     14.86 |      325.14 |    341 |
| **NEGany~necessarily_new**            |   482 |       253,862 |    0.94 |    7.94 | 2,923.82 |     21.44 |      460.56 |    492 |
| **NEGmir~necessarily_right**          |    23 |         5,576 |    0.83 |    2.15 |    81.13 |      3.94 |       19.06 |     23 |
| **NEGmir~necessarily_illegal**        |    15 |           937 |    0.83 |    1.20 |    52.91 |      2.57 |       12.43 |     15 |
| **NEGmir~necessarily_wrong**          |   211 |        20,880 |    0.81 |    5.04 |   698.74 |     37.03 |      173.97 |    216 |
| **NEGmir~necessarily_new**            |    23 |        12,836 |    0.79 |    1.84 |    73.19 |      4.11 |       18.89 |     24 |
| **NEGmir~necessarily_bad**            |    50 |        10,261 |    0.77 |    2.95 |   154.45 |      9.08 |       40.92 |     53 |
| **NEGmir~necessarily_true**           |    53 |         6,191 |    0.73 |    2.69 |   150.42 |     10.11 |       42.89 |     59 |
| **NEGmir~necessarily_better**         |    27 |        14,013 |    0.70 |    1.63 |    72.90 |      5.31 |       21.69 |     31 |


1. _necessarily useful_

|                                                | `token_str`                                                                                                                                                                                                                                                                     |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_16_024.1662_x0374968_60:11-12-13`** | The talent and skill required to win an election is not __`necessarily useful`__ for the actual practice of good governing .                                                                                                                                                    |
| **`pcc_eng_16_040.9012_x0645895_24:32-33-34`** | I find that is more wise to spend your store reward money on something * useful * rather than just rolling your reward money into products that are FREE , but not __`necessarily useful`__ to your family !                                                                    |
| **`pcc_eng_04_054.7457_x0868076_11:4-5-6`**    | So it is n't __`necessarily useful`__ for monitoring disease activity .                                                                                                                                                                                                         |
| **`pcc_eng_02_003.9319_x0047305_11:36-37-38`** | He speaks to Mariko Kosaka , a Developer Advocate on the Google Developers Relations Team , about the art she creates with code and how developers can embrace the notion of making something that is n't __`necessarily useful`__ but still has value .                        |
| **`pcc_eng_11_086.0944_x1377399_23:2-3-4`**    | Though not __`necessarily useful`__ in day to day treatment of the common condition , the ingestion of burdock may help to control predicted spikes in blood pressure following meals which can be useful in persons who struggle with unhealthy fluctuations around mealtime . |


2. _necessarily fun_

|                                                 | `token_str`                                                                                                                                                           |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_22_007.6486_x0107392_049:12-13-14`** | " When you are a 19 - year-old , it 's not __`necessarily fun`__ to be pounding kale smoothies and not going out to Chipotle with your te ......                      |
| **`pcc_eng_07_012.4303_x0185352_05:23-24-25`**  | Snapchat 's new photo filter that allows users to change into a man or woman with the tap of a finger is n't __`necessarily fun`__ and games for transgender people . |
| **`pcc_eng_22_058.2949_x0926215_216:18-19-20`** | " We were aspiring to achieve a level that was maybe beyond us , and that 's not __`necessarily fun`__ , " he says .                                                  |
| **`pcc_eng_19_043.2627_x0682345_13:15-16-17`**  | It 's a great concept leads to some interesting situations , though it 's not __`necessarily fun`__ .                                                                 |
| **`pcc_eng_12_063.5070_x1010663_50:6-7-8`**     | Writing a better story was n't __`necessarily fun`__ , conflict resolution took much longer because there was actually conflict to resolve .                          |


3. _necessarily essential_

|                                                  | `token_str`                                                                                                                                                                                                                                                                                                             |
|:-------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_16_087.2409_x1396021_45:14-15-16`**   | Can it possibly be that the values we have established in America are not __`necessarily essential`__ to the well - being of the human race , or do we sometimes have a definite edge over the Germans ?                                                                                                                |
| **`pcc_eng_27_066.6822_x1061853_28:3-4-5`**      | Speed is n't __`necessarily essential`__ , though .                                                                                                                                                                                                                                                                     |
| **`apw_eng_20090115_1256_106:12-13-14`**         | for this job , intelligence experience is certainly helpful but is n't __`necessarily essential`__ .                                                                                                                                                                                                                    |
| **`pcc_eng_20_080.4367_x1283424_0334:35-36-37`** | 00:12:51,170 PROFESSOR : That is true , but although in English in 286 00:12:51,170 --> 00:12:56,110 particular the phonetic meaning , the way how a certain 287 00:12:56,110 --> 00:13:00,397 word is pronounced is n't __`necessarily essential`__ for 288 00:13:00,397 --> 00:13:01,540 solving a crossword puzzle . |
| **`pcc_eng_07_028.1119_x0438479_17:08-09-10`**   | Of course , understanding the science is n't __`necessarily essential`__ to the key sci-fi and techno-thriller readers .                                                                                                                                                                                                |


4. _necessarily reliable_

|                                                 | `token_str`                                                                                                                                                                                                                                                                                                               |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_01_061.1479_x0972872_28:29-30-31`**  | While they do find an acceleration in MFP outside the IT sector during the late 1990s , the extent of this increase depends upon price indexes that are not __`necessarily reliable`__ right now .                                                                                                                        |
| **`pcc_eng_12_066.3925_x1056872_207:21-22-23`** | We need to get vaccinations , amass enormous quantities of prescription and over- the- counter drugs ( Vietnam 's are not __`necessarily reliable`__ or accessible ) , prepare Olivia for a big trip , find her old crib and baby stuff , and on and on .                                                                 |
| **`pcc_eng_19_045.6425_x0720669_30:21-22-23`**  | If you are reliant on the Middle East , or Iran or Russia to get your gas -- that 's not __`necessarily reliable`__ .                                                                                                                                                                                                     |
| **`pcc_eng_10_011.5500_x0170445_12:18-19-20`**  | While previous studies have identified a difference in mercury levels between tuna species , those studies were n't __`necessarily reliable`__ because they did n't have a fool - proof way of determining which tuna came from which species , said study researcher Joanna Burger , a professor at Rutgers University . |
| **`pcc_eng_17_079.9551_x1276018_02:37-38-39`**  | ( Reuters Health ) - Nearly 88 percent of online reviews of U.S. plastic surgeons who provide breast augmentation were found to be positive in a recent study , but that snapshot of customer satisfaction is not __`necessarily reliable`__ , researchers say .                                                          |


5. _necessarily proud_

|                                               | `token_str`                                                                                                                                                     |
|:----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20070520_0021_49:4-5-6`**          | `` I 'm not __`necessarily proud`__ of being German , but I am proud of being from Cologne .                                                                    |
| **`pcc_eng_01_095.6067_x1529843_5:24-25-26`** | I got a young man named George W. Bush in the National Guard when I was Lt. Gov. of Texas and I 'm not __`necessarily proud`__ of that .                        |
| **`pcc_eng_17_070.2041_x1118143_08:3-4-5`**   | I 'm not __`necessarily proud`__ of it , but there you go .                                                                                                     |
| **`pcc_eng_02_001.9430_x0015306_52:6-7-8`**   | Even about things I 'm not __`necessarily proud`__ of .                                                                                                         |
| **`pcc_eng_26_092.9107_x1486511_065:5-6-7`**  | Afterward , he was n't __`necessarily proud`__ of the achievement , insisting what comes next in a series is always more important than what 's just happened . |


6. _necessarily surprising_

|                                                | `token_str`                                                                                                                                                                                                                                                                               |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_09_038.8901_x0613229_64:7-8-9`**    | Of course , these results are n't __`necessarily surprising`__ .                                                                                                                                                                                                                          |
| **`pcc_eng_06_093.9383_x1503133_33:3-4-5`**    | It 's not __`necessarily surprising`__ that the article puts the responsibility on what moms " see , " indicating that this is all about individual responsibility .                                                                                                                      |
| **`pcc_eng_29_083.4727_x1332100_06:18-19-20`** | So , while the record number of home sales through the first five months of 2016 is not __`necessarily surprising`__ , it does sometimes mask the larger story in the GTA : the shortage of listings , which has resulted in strong upward pressure on home prices , " said Mr. Mc Lean . |
| **`pcc_eng_16_080.2583_x1282733_45:20-21-22`** | Kahrl 's piece is a far better analysis of what went wrong than Simmons ' , though that is n't __`necessarily surprising`__ .                                                                                                                                                             |
| **`pcc_eng_02_080.6539_x1287909_02:08-09-10`** | The Desktop ( RKG ) This is not __`necessarily surprising`__ , since the screen real estate of a tablet approximates the desktop's .                                                                                                                                                      |


7. _necessarily indicative_

|                                                 | `token_str`                                                                                                                                                                           |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_13_034.8920_x0547952_033:14-15-16`** | Remember , all of these numbers are subject to small sample size and not __`necessarily indicative`__ of players ' true talents .                                                     |
| **`pcc_eng_16_081.5307_x1303380_16:10-11-12`**  | The titles available in the Shinra Technologies beta are not __`necessarily indicative`__ of the company 's service lineup , said Kristin De Rosa , director of marketing at Shinra . |
| **`pcc_eng_23_019.0750_x0291648_16:11-13-14`**  | The past performance of any trading method or methodology will not be __`necessarily indicative`__ of future outcomes .                                                               |
| **`pcc_eng_19_016.7237_x0253666_15:4-5-6`**     | But this was not __`necessarily indicative`__ of Washington 's broader geographical use of ' joystick bombers ' in the previous year .                                                |
| **`pcc_eng_28_077.4646_x1236830_36:5-6-7`**     | Remember that price is not __`necessarily indicative`__ of quality and not all the best companies are the most expensive .                                                            |


8. _necessarily easy_

|                                                 | `token_str`                                                                                                                                                                           |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_27_057.5629_x0914301_22:28-29-30`**  | And getting folks in their seats for a battle royale over the question of whether Locke or Plutarch 's more correct about the nature of law is n't __`necessarily easy`__ .           |
| **`pcc_eng_19_010.0168_x0145829_05:24-25-26`**  | According to Kathleen O'Reilly , the museum 's vice president , " The site is a top destination for Houston visitors , but not __`necessarily easy`__ to navigate for pedestrians . " |
| **`pcc_eng_15_096.9216_x1550382_29:6-7-8`**     | But making the shift is n't __`necessarily easy`__ or simple .                                                                                                                        |
| **`pcc_eng_07_024.8096_x0385118_24:09-10-11`**  | In fact , many of mindfulness practices were n't __`necessarily easy`__ even if they were simple to do .                                                                              |
| **`pcc_eng_20_008.7627_x0125310_094:27-28-29`** | The one on the Audi , VW 's luxury division , looks all right , and is somewhat simpler to handle than i Drive , but not __`necessarily easy`__ .                                     |


9. _necessarily representative_

|                                                 | `token_str`                                                                                                                                                                                                                                                        |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_29_037.5350_x0589619_208:15-17-18`** | He recommends using only rents , even if few rents are available and may not be __`necessarily representative`__ of the housing stock and market .                                                                                                                 |
| **`pcc_eng_22_086.2539_x1378067_29:6-7-8`**     | Because A ) pictures are n't __`necessarily representative`__ of how a girl looks in person and B ) there 's a chemistry element that you ca n't quantify beforehand .                                                                                             |
| **`pcc_eng_21_076.0376_x1212613_8:14-15-16`**   | The opinions expressed on this blog are those of the individual authors and not __`necessarily representative`__ of the GMCTE , CGSR , or the University of Saskatchewan .                                                                                         |
| **`pcc_eng_11_098.5959_x1579848_18:11-12-13`**  | Top dating sites for lesbians Mingle dating site Experiences are not __`necessarily representative`__ of , credits for an hour is all that online dating really something works both you , but he has no say in how .                                              |
| **`pcc_eng_05_083.7342_x1338981_30:27-28-29`**  | The views and opinions expressed by the SOI Industry Consortium through officers in the SOI Industry Consortium or in this presentation or other communication vehicles are not __`necessarily representative`__ of the views and opinions of individual members . |


10. _necessarily new_

|                                                | `token_str`                                                                                                                                                                                                                   |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_04_042.8037_x0675485_20:36-37-38`** | That HTC has chosen to not put Android front and center , instead marketing its own innovations , may be worrisome to Android purists and certainly to Google 's marketing department , but it 's not __`necessarily new`__ . |
| **`pcc_eng_07_059.2916_x0942174_18:08-09-10`** | For technology observers , patent battles are not __`necessarily new`__ .                                                                                                                                                     |
| **`pcc_eng_11_098.0215_x1570595_10:08-09-10`** | Liptak notes that this particular protocol is n't __`necessarily new`__ or malicious .                                                                                                                                        |
| **`pcc_eng_16_021.8265_x0337269_23:08-09-10`** | There is another option and it 's not __`necessarily new`__ .                                                                                                                                                                 |
| **`pcc_eng_14_085.3775_x1363859_12:22-23-24`** | Women 's nomenclature has been a point of contention for centuries -- and the practice of keeping one 's own is n't __`necessarily new`__ .                                                                                   |


11. _necessarily right_

|                                                 | `token_str`                                                                                                  |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_07_019.6821_x0302170_38:09-10-11`**  | What 's right for a college paper is not __`necessarily right`__ for a big city daily .                      |
| **`pcc_eng_00_067.0580_x1067739_14:14-15-16`**  | Schwartz said the parting was amicable , but the shakeup ' ' was n't __`necessarily right`__ for me . ''     |
| **`pcc_eng_18_043.6320_x0689874_11:14-15-16`**  | As in much of life , what is right for one person is not __`necessarily right`__ for another .               |
| **`pcc_eng_15_096.7466_x1547526_036:12-13-14`** | The plan that made sense for you a year ago is n't __`necessarily right`__ for you now .                     |
| **`pcc_eng_22_003.2273_x0036168_18:6-7-8`**     | But I realized it was n't __`necessarily right`__ for me - there was just something missing about design . " |


12. _necessarily illegal_

|                                                | `token_str`                                                                                                                                                                 |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_22_080.8030_x1289779_03:25-26-27`** | We 're not clear if he sold them or not - we 'd hope he could give them away certainly - but it 's not __`necessarily illegal`__ as the Greensboro News & Record suggests . |
| **`nyt_eng_20000608_0216_35:24-25-26`**        | handing out gifts of food , T-shirts , building supplies and even medicines is standard fare during political campaigns in Mexico and is not __`necessarily illegal`__ .    |
| **`nyt_eng_19961022_0361_4:22-23-24`**         | he noted , however , that a solicitation differed under law from an actual physical transmission of child pornography and was not __`necessarily illegal`__ .               |
| **`pcc_eng_09_003.7452_x0044623_12:3-4-5`**    | Backdating is not __`necessarily illegal`__ , but it must be disclosed to investors and accounted for properly .                                                            |
| **`pcc_eng_03_009.4563_x0136773_27:10-11-12`** | The abuse of power means favours - which are not __`necessarily illegal`__ .                                                                                                |


13. _necessarily wrong_

|                                              | `token_str`                                                                                                                                                                                        |
|:---------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_26_039.3422_x0619945_35:4-5-6`**  | The consensus is n't __`necessarily wrong`__ , and the dogma may contain a great deal of truth , but critical analysis can get lost along the way and important aspects can be taken for granted . |
| **`pcc_eng_00_063.4881_x1010223_30:3-4-5`**  | There 's nothing __`necessarily wrong`__ with that .                                                                                                                                               |
| **`pcc_eng_23_032.1180_x0502425_59:4-5-6`**  | Although there 's nothing __`necessarily wrong`__ with that approach , it 's not the most efficient or consistent way to produce a great image .                                                   |
| **`nyt_eng_20060816_0013_21:09-10-11`**      | although the comments irritated Woods , they were not __`necessarily wrong`__ ; Woods was not utilizing the best technology at the time .                                                          |
| **`pcc_eng_22_003.8824_x0046751_027:3-4-5`** | It 's not __`necessarily wrong`__ , but a coach trusts his own kid .                                                                                                                               |


14. _necessarily bad_

|                                                 | `token_str`                                                                                                                                                                                                                           |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_14_004.9912_x0064687_32:7-8-9`**     | Keep in mind that it 's not __`necessarily bad`__ to use these words here and there , but over-using them could devalue your content .                                                                                                |
| **`pcc_eng_11_052.9064_x0839737_07:38-39-40`**  | Wherever you place your priorities in terms of the actions of the executive branch , at this point in history , the nominating of Supreme Court justices has become extremely partisan , in a way that is n't __`necessarily bad`__ . |
| **`nyt_eng_20050708_0333_21:23-24-25`**         | the possibility of deviating this much from the return of the average stock , sometimes referred to as tracking error , is not __`necessarily bad`__ .                                                                                |
| **`pcc_eng_18_084.7136_x1355514_24:3-4-5`**     | This is not __`necessarily bad`__ as a few emergency plumbing contractors choose to target their expertise on a precise number to do with services and as a result do in the future with in conversation with .                       |
| **`pcc_eng_11_017.5606_x0267951_260:09-10-11`** | Just that reaction in and of itself is not __`necessarily bad`__ .                                                                                                                                                                    |


15. _necessarily true_

|                                                 | `token_str`                                                                                                                                                                                                                             |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_073.0937_x1166132_237:7-8-9`**    | I found out that this is n't __`necessarily true`__ .                                                                                                                                                                                   |
| **`pcc_eng_29_090.8341_x1450944_031:34-35-36`** | What if I told you that despite the popular belief that children are more likely to read on a device like an i Pad or a Kindle , research shows that this is n't __`necessarily true`__ .                                               |
| **`pcc_eng_04_108.09904_x1745197_08:2-3-4`**    | Although not __`necessarily true`__ , the build up to a fight helps bring in the crowd , when two men are trash talking one another , people want to see the outcome , who will get the last word per say .                             |
| **`pcc_eng_14_035.3114_x0554219_48:12-13-14`**  | Finally , I want to tell you a story that is not __`necessarily true`__ , but it 's a good story and that , after all , should be enough .                                                                                              |
| **`pcc_eng_11_008.5813_x0122676_04:21-22-23`**  | The perception that Self Managed Superannuation Funds ( SMSFs ) have too much focus on cash and terms deposit is n't __`necessarily true`__ when looking at the results of a Mercer report for the Financial Services Council ( FSC ) . |


16. _necessarily better_

|                                                 | `token_str`                                                                                                              |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_07_027.7515_x0432620_17:10-11-12`**  | As far as size is concerned , bigger is n't __`necessarily better`__ .                                                   |
| **`pcc_eng_24_028.2479_x0440581_056:09-10-11`** | But the officially endorsed alternatives to DARE are n't __`necessarily better`__ .                                      |
| **`pcc_eng_19_041.4701_x0653254_14:5-6-7`**     | Also , more is not __`necessarily better`__ .                                                                            |
| **`pcc_eng_23_044.7042_x0706177_33:08-09-10`**  | It strikes me as more satisfying but not __`necessarily better`__ in every way that these tracks embody their subjects . |
| **`pcc_eng_15_093.3141_x1492208_52:4-5-6`**     | The facilities are n't __`necessarily better`__ , he says .                                                              |


Saving Samples in /share/compling/projects/sanpi/results/top_AM/neg_bigram_examples/necessarily/...

Samples saved as...
1. `neg_bigram_examples/necessarily/necessarily_useful_99ex.csv`
1. `neg_bigram_examples/necessarily/necessarily_fun_99ex.csv`
1. `neg_bigram_examples/necessarily/necessarily_essential_99ex.csv`
1. `neg_bigram_examples/necessarily/necessarily_reliable_99ex.csv`
1. `neg_bigram_examples/necessarily/necessarily_proud_99ex.csv`
1. `neg_bigram_examples/necessarily/necessarily_surprising_99ex.csv`
1. `neg_bigram_examples/necessarily/necessarily_indicative_99ex.csv`
1. `neg_bigram_examples/necessarily/necessarily_easy_99ex.csv`
1. `neg_bigram_examples/necessarily/necessarily_representative_99ex.csv`
1. `neg_bigram_examples/necessarily/necessarily_new_99ex.csv`
1. `neg_bigram_examples/necessarily/necessarily_right_99ex.csv`
1. `neg_bigram_examples/necessarily/necessarily_illegal_99ex.csv`
1. `neg_bigram_examples/necessarily/necessarily_wrong_99ex.csv`
1. `neg_bigram_examples/necessarily/necessarily_bad_99ex.csv`
1. `neg_bigram_examples/necessarily/necessarily_true_99ex.csv`
1. `neg_bigram_examples/necessarily/necessarily_better_99ex.csv`

## 2. *that*

|                  |        `N` |      `f1` |   `adv_total` |
|:-----------------|-----------:|----------:|--------------:|
| **NEGATED_that** | 72,839,589 | 3,173,660 |       208,262 |
| **NEGMIR_that**  |  1,701,929 |   291,732 |         5,494 |


|                             |   `f` |   `adj_total` |   `dP1` |   `LRC` |      `G2` |   `exp_f` |   `unexp_f` |   `f2` |
|:----------------------------|------:|--------------:|--------:|--------:|----------:|----------:|------------:|-------:|
| **NEGany~that_far-fetched** |    59 |         5,185 |    0.96 |    5.83 |    369.74 |      2.57 |       56.43 |     59 |
| **NEGany~that_thrilled**    |    59 |        24,182 |    0.96 |    5.83 |    369.74 |      2.57 |       56.43 |     59 |
| **NEGany~that_uncommon**    |   802 |        11,312 |    0.95 |    9.43 |  4,998.32 |     35.03 |      766.97 |    804 |
| **NEGany~that_surprising**  | 1,133 |        70,540 |    0.95 |    9.20 |  6,986.81 |     49.80 |    1,083.20 |  1,143 |
| **NEGany~that_unusual**     |   977 |        71,234 |    0.95 |    8.92 |  6,003.05 |     43.05 |      933.95 |    988 |
| **NEGany~that_dissimilar**  |   304 |         4,605 |    0.95 |    7.86 |  1,871.65 |     13.38 |      290.62 |    307 |
| **NEGany~that_noticeable**  |   264 |        31,467 |    0.95 |    7.65 |  1,621.81 |     11.63 |      252.37 |    267 |
| **NEGany~that_complicated** | 1,207 |       159,822 |    0.94 |    8.68 |  7,337.84 |     53.59 |    1,153.41 |  1,230 |
| **NEGany~that_familiar**    | 1,126 |       156,296 |    0.93 |    8.35 |  6,781.11 |     50.37 |    1,075.63 |  1,156 |
| **NEGany~that_hard**        | 9,948 |       348,463 |    0.91 |    8.59 | 58,817.24 |    452.26 |    9,495.74 | 10,380 |
| **NEGmir~that_keen**        |    31 |         1,360 |    0.83 |    2.73 |    109.35 |      5.31 |       25.69 |     31 |
| **NEGmir~that_impressive**  |    23 |         5,007 |    0.83 |    2.15 |     81.13 |      3.94 |       19.06 |     23 |
| **NEGmir~that_fond**        |    23 |         1,115 |    0.79 |    1.84 |     73.19 |      4.11 |       18.89 |     24 |
| **NEGmir~that_comfortable** |    23 |         4,642 |    0.79 |    1.84 |     73.19 |      4.11 |       18.89 |     24 |
| **NEGmir~that_clear**       |    18 |         6,722 |    0.78 |    1.30 |     56.03 |      3.26 |       14.74 |     19 |
| **NEGmir~that_popular**     |    65 |         5,668 |    0.76 |    3.15 |    195.15 |     12.00 |       53.00 |     70 |
| **NEGmir~that_simple**      |   474 |        25,408 |    0.72 |    4.36 |  1,340.19 |     90.68 |      383.32 |    529 |
| **NEGmir~that_easy**        |   450 |        18,610 |    0.71 |    4.23 |  1,248.84 |     87.08 |      362.92 |    508 |
| **NEGmir~that_big**         |   113 |         8,177 |    0.69 |    3.17 |    300.54 |     22.46 |       90.54 |    131 |
| **NEGmir~that_great**       |   286 |         5,568 |    0.66 |    3.57 |    725.16 |     58.62 |      227.38 |    342 |


1. _that far-fetched_

|                                                 | `token_str`                                                                                                                                                                                                                             |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20000405_0292_6:36-38-39`**          | promoting Piero as a kind of celebrity , albeit one whose personal life remains sketchy -LRB- apart from several sly self-portraits and a historically resonant death date : Oct. 12 , 1492 -RRB- , may not be __`that far-fetched`__ . |
| **`nyt_eng_20051115_0368_3:08-10-11`**          | but today , Shugrue 's statement is n't all __`that far-fetched`__ : Abbot Kinney has , seemingly overnight , become the darling of Los Angeles ' art and architecture set .                                                            |
| **`pcc_eng_06_104.2003_x1669450_214:12-13-14`** | Of course , writing off a comeback from Edmure Tully is n't __`that far-fetched`__ , especially since we 've all probably been guilty of doing that several times already .                                                             |
| **`pcc_eng_06_103.0609_x1651098_20:14-15-16`**  | In his estimation , the actual nuts and bolts of the procedure are not __`that far-fetched`__ for any ED to perform , as long as the necessary back - up services to take patients to definitive therapy are close at hand .            |
| **`nyt_eng_20000321_0286_23:17-19-20`**         | based on the Golden Bear 's expanded playing schedule in 2000 , a Colonial cameo might not be __`that far-fetched`__ .                                                                                                                  |


2. _that thrilled_

|                                                | `token_str`                                                                                                                                                                              |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_100.0477_x1602316_03:14-15-16`** | Maybe you 've got a Word Press blog already , but you are n't __`that thrilled`__ with how it 's functioning .                                                                           |
| **`pcc_eng_15_100.4327_x1606964_08:3-4-5`**    | I was n't __`that thrilled`__ with the cybernetic additions to it anyway .                                                                                                               |
| **`pcc_eng_12_001.2080_x0003435_3:26-27-28`**  | Punk was asked about possibly facing The Dead Man at Wrestle Mania during a panel at last September 's Ohio Comic-Con , and Punk was n't __`that thrilled`__ about the potential match . |
| **`pcc_eng_00_065.6586_x1045361_19:15-16-17`** | And I even insist , though my wife [ who is Jewish ] is n't __`that thrilled`__ , on having for our daughter a little version of the Seder .                                             |
| **`pcc_eng_28_024.1654_x0374101_09:12-13-14`** | My expectations were low going in , but I still was n't __`that thrilled`__ with the book .                                                                                              |


3. _that uncommon_

|                                                | `token_str`                                                                                                                                                                                                                                                                                                                         |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_17_078.2891_x1249008_316:3-4-5`**   | That is not __`that uncommon`__ and I understand that happens every day .                                                                                                                                                                                                                                                           |
| **`pcc_eng_15_048.9603_x0775350_14:6-8-9`**    | Zero-day attacks against Reader are n't all __`that uncommon`__ , but the bulk of the problem comes from users running old versions , getting exploited through patched vulnerabilities .                                                                                                                                           |
| **`pcc_eng_18_008.9083_x0127940_11:19-20-21`** | Early morning FBI arrests for those who threaten national security based on wiretaps and email interceptions are now not __`that uncommon`__ , but you can walk the streets confident that you are not on camera the whole time , and the innocent seem to have relatively little to fear .                                         |
| **`pcc_eng_24_071.1969_x1135485_037:6-7-8`**   | Perhaps such a practice was not __`that uncommon`__ in Japan , especially during the Middle Ages .                                                                                                                                                                                                                                  |
| **`pcc_eng_17_040.5088_x0638512_33:08-09-10`** | Girls on a Little League team are n't __`that uncommon`__ ( and Chelsea 's teammates seem adorably unfazed about her gender in the ESPN segment ) , but resistance seems to build as the level of play increases and competition for spots means any possible distraction , even a genetic one , can push a player off the roster . |


4. _that surprising_

|                                                | `token_str`                                                                                                                                                                                                                                                                  |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_29_002.4752_x0023734_16:6-7-8`**    | That coffee is king is n't __`that surprising`__ , but the popularity of celery seems weird .                                                                                                                                                                                |
| **`pcc_eng_27_003.3409_x0037640_51:26-27-28`** | A quick Yahoo search reveals that this is something many individuals still believe today , which in the grand scheme of the Internet is honestly not __`that surprising`__ .                                                                                                 |
| **`pcc_eng_24_084.2654_x1346875_072:3-4-5`**   | This is not __`that surprising`__ as it is after all broken and broken economies are not quickly fixed .                                                                                                                                                                     |
| **`pcc_eng_04_070.4973_x1122489_19:10-11-12`** | In one way , the study 's results are n't __`that surprising`__ .                                                                                                                                                                                                            |
| **`pcc_eng_26_084.6989_x1353571_07:47-49-50`** | Thinking about the number of students involved - fifteen from each school - and the fact that each was having ten of their best shots on two display boards , plus boards full of their other pictures of the places we had been , it was n't really __`that surprising`__ . |


5. _that dissimilar_

|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                              |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_039.8031_x0627596_36:11-13-14`** | The science of our time tells us that you are not all __`that dissimilar`__ to a TV , and the station you pick up ( your reality ) has much to do with what you are tuned into .                                                                                                                                                                         |
| **`pcc_eng_02_010.0479_x0146168_75:4-5-6`**    | Their motives are n't __`that dissimilar`__ , either .                                                                                                                                                                                                                                                                                                   |
| **`pcc_eng_03_086.7031_x1387766_09:12-13-14`** | " The Universal Sony survey found that in reality we are not __`that dissimilar`__ to the Snow White fairytale ideal , with the research results mirroring the plot involving thecharacters of the evil queen Ravenna , ( Charlize Theron ) and Snow White ( Kristen Stewart ) , where a beautiful appearance is inferior to a kind heart , " she said . |
| **`pcc_eng_27_061.0198_x0970053_13:15-16-17`** | This counterpoint of motherly and gubernatorial duties reveals that the two jobs really are n't __`that dissimilar`__ .                                                                                                                                                                                                                                  |
| **`pcc_eng_20_009.7335_x0140842_09:11-12-13`** | For those that do n't know , hydrogen cars are n't __`that dissimilar`__ from electric cars in the way they move and operate .                                                                                                                                                                                                                           |


6. _that unusual_

|                                                | `token_str`                                                                                                                                                                                                         |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_20_001.4891_x0007730_50:4-5-6`**    | But that was n't __`that unusual`__ for minor-league arenas at the time .                                                                                                                                           |
| **`nyt_eng_20070201_0228_44:22-23-24`**        | the title of one 1861 Confederate song sheet , `` My God ! What Is This All For ? '' is not __`that unusual`__ .                                                                                                    |
| **`pcc_eng_09_037.4457_x0589982_05:7-8-9`**    | First Read notes that it 's not __`that unusual`__ for a presidential campaign to change managers when things are n't going well .                                                                                  |
| **`pcc_eng_18_045.5733_x0721472_07:5-6-7`**    | Logsdon said it is n't __`that unusual`__ for boys to do that .                                                                                                                                                     |
| **`pcc_eng_28_065.1249_x1037592_36:41-42-43`** | The $ 120 per vote that Khanna spent might seem like a lot , Gerston said , but given his low name recognition compared to seven- term incumbent Honda , " an almost 4 - to - 1 ratio is not __`that unusual`__ . " |


7. _that noticeable_

|                                                | `token_str`                                                                                                                                                                                                                                                        |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_05_034.9771_x0550267_54:4-5-6`**    | Puerto Ricans are not __`that noticeable`__ in the United States , because we have so much diversity as it is .                                                                                                                                                    |
| **`pcc_eng_16_085.0087_x1359795_52:10-12-13`** | Marijuana activists were hoping Colorado 's grand experiment would n't be __`that noticeable`__ after an initial rush of shopping .                                                                                                                                |
| **`pcc_eng_19_073.1119_x1164880_3:23-24-25`**  | I just got braces put on yesterday , they hurt of course but they 're ceramic so they look nice and are n't __`that noticeable`__ .                                                                                                                                |
| **`pcc_eng_17_100.1973_x1603440_3:46-47-48`**  | My hair dresser weaved the colour through my natural brown so it 's a little bit more subtle and blends a little better with the top layer of my hair , without looking like I 've dipped my hair in dye ( although it 's not __`that noticeable`__ in the photo ! |
| **`pcc_eng_22_066.4802_x1058237_02:7-8-9`**    | It will be slightly smaller but not __`that noticeable`__ .                                                                                                                                                                                                        |


8. _that complicated_

|                                                 | `token_str`                                                                            |
|:------------------------------------------------|:---------------------------------------------------------------------------------------|
| **`apw_eng_20020730_0397_13:6-7-8`**            | `` The operation itself is not __`that complicated`__ .                                |
| **`pcc_eng_02_091.2391_x1458980_50:5-6-7`**     | And the solutions are not __`that complicated`__ .                                     |
| **`pcc_eng_26_097.9696_x1567736_80:13-14-15`**  | I ca n't understand all the details but the basic idea is n't __`that complicated`__ . |
| **`pcc_eng_08_107.0102_x1716545_116:12-13-14`** | If you strive for that , all the other stuff is n't __`that complicated`__ .           |
| **`pcc_eng_01_040.7428_x0642175_24:6-7-8`**     | I guess life just is n't __`that complicated`__ when you 're marrying Hef .            |


9. _that familiar_

|                                                | `token_str`                                                                                                                                                                                                           |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_00_031.1827_x0487860_31:08-09-10`** | Now , most of the audience was not __`that familiar`__ with Jim and I 'm going go out on a limb and say many of them had no idea what kind of comedy he does or what his life is .                                    |
| **`pcc_eng_01_034.9562_x0548609_21:25-26-27`** | Though Suicide Squad certainly enjoys a wide following , the story builds upon quite a lot of Suicide Squad lore that I just was n't __`that familiar`__ with , and it made it hard to be all that engaged .          |
| **`pcc_eng_18_040.0411_x0631702_09:4-5-6`**    | If you are not __`that familiar`__ with Word Press the video tutorials will guide ...                                                                                                                                 |
| **`pcc_eng_02_086.1077_x1375939_07:17-18-19`** | Barnes also shed light on Solange 's role as matchmaker , revealing , " I was n't __`that familiar`__ with her Destiny 's                                                                                             |
| **`pcc_eng_23_035.3524_x0554898_07:11-12-13`** | This chart is not easily digestible by someone who is not __`that familiar`__ with Moto GP racing , and the original post did not contain much data as to what this infographic was all about or why it was created . |


10. _that hard_

|                                                 | `token_str`                                                                                                                                             |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_05_085.8509_x1373075_36:17-18-19`**  | Look Tom , I know there 's alot of turnover at Notre Dame but it 's not __`THAT hard`__ to learn the names and numbers of the freshmen and sophomores . |
| **`pcc_eng_28_077.7004_x1240696_046:09-10-11`** | So I will tell you that risotto is not __`that hard`__ .                                                                                                |
| **`nyt_eng_19990729_0090_12:16-17-18`**         | a lot of other people ended up cleaning up a lot of things that were n't __`that hard`__ to get done . ''                                               |
| **`pcc_eng_14_091.6059_x1464560_31:08-10-11`**  | " Numberwise and clubwise , it should n't be __`that hard`__ , " he said .                                                                              |
| **`pcc_eng_26_096.3568_x1541897_46:4-6-7`**     | Boiling water should n't be __`that hard`__ .                                                                                                           |


11. _that keen_

|                                                | `token_str`                                                                                                                                                                             |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_28_046.5788_x0737592_08:4-6-7`**    | Normally I am not all __`that keen`__ on watching a 19 min video , yet this one kept my interest from the beginning until the end .                                                     |
| **`pcc_eng_07_051.1010_x0809919_22:3-4-5`**    | I was n't __`that keen`__ on the color when I saw it in the bottle , but once I applied it I loved it .                                                                                 |
| **`pcc_eng_13_008.5844_x0122431_23:34-35-36`** | In the twenty - first century , we can no more write a child off because " he 's not into numbers " any more than we would accept that " she 's not __`that keen`__ on the alphabet " . |
| **`apw_eng_19980225_1426_48:14-16-17`**        | her folks are not home , and let 's just say she 's not all __`that keen`__ to hit the books .                                                                                          |
| **`pcc_eng_12_032.5640_x0510985_11:28-29-30`** | If you have a glut of tomatoes you can turn them into soup or Napoli sauce as these freeze well , even if the whole tomatoes are n't __`that keen`__ on getting frosty .                |


12. _that impressive_

|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                                                  |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_27_054.1707_x0859538_49:27-28-29`**  | Apple says that it has 7,000 movies to rent with over 3,400 available in HD , which compared to those available through the Netflix service is not __`that impressive`__ .                                                                                                                                                                                                   |
| **`pcc_eng_10_079.2508_x1264773_6:23-25-26`**   | Just do n't expect to blow anyone 's mind because without the added element of having to balance , your tricks wo n't be __`that impressive`__ .                                                                                                                                                                                                                             |
| **`pcc_eng_16_077.6387_x1240217_133:19-20-21`** | I 'm guessing that where he is , currently , being on the cutting edge of OER is n't __`that impressive`__ but maybe in another year or two of the OER movement growing ...?                                                                                                                                                                                                 |
| **`pcc_eng_11_018.6535_x0285562_86:5-6-7`**     | My first try was not __`that impressive`__ .                                                                                                                                                                                                                                                                                                                                 |
| **`pcc_eng_27_023.9948_x0371803_07:50-52-53`**  | Diaz , who homered twice in the MLB All - Star Futures Game while he was with the Dodgers and was traded days later , hit .285/392/.449 with 11 home runs and 15 doubles between Double -A Tulsa and Bowie , though his .239/.329/.403 line with the Baysox was n't all __`that impressive`__ as the Orioles got to work tinkering with his swing shortly after he arrived . |


13. _that fond_

|                                                | `token_str`                                                                                                                                                                                                      |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19970903_0955_23:41-42-43`**        | a woman who declined to be named said : `` Why does she have to hide behind the gates of Balmoral ? My God , if I died I hope my mother-in-law would say something , even if she was n't __`that fond`__ of me . |
| **`pcc_eng_05_008.4749_x0121348_04:08-10-11`** | To tell the truth , I have never been __`that fond`__ of calabacitas , a Mexican squash dish made with zucchini and / or yellow summer squash .                                                                  |
| **`pcc_eng_06_107.5121_x1722932_35:11-12-13`** | I 'm led to believe there are people who are n't __`that fond`__ of seals , saying they 're smelly , noisome vermin that litter the beaches and eat far too many fish .                                          |
| **`pcc_eng_21_027.7355_x0432217_072:3-4-5`**   | I was n't __`that fond`__ of the route overall .                                                                                                                                                                 |
| **`pcc_eng_24_021.6624_x0333739_15:12-13-14`** | It 's no surprise that Margaery ( Natalie Dormer ) was n't __`that fond`__ of Joffrey himself , but rather wanted the power her marriage to him would bring .                                                    |


14. _that comfortable_

|                                                | `token_str`                                                                                                                                                                                                                                                         |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_16_050.3279_x0798544_17:25-27-28`** | To accomplish this , the headphones sit very close to the head and we actually found that they felt quite tight so they were n't really __`that comfortable`__ after a while .                                                                                      |
| **`pcc_eng_07_054.7104_x0868207_18:19-20-21`** | A Jeep Wrangler is about as aerodynamic as a brick , has terrible fuel economy , and is n't __`that comfortable`__ compared to other cars in its price range .                                                                                                      |
| **`pcc_eng_27_055.0829_x0874062_41:10-11-12`** | Similarly , if you 're good with people but not __`that comfortable`__ with numbers , you should make the effort to learn some accounting fundamentals so you can follow these conversations in a business meeting and not have to rely solely on your accountant . |
| **`pcc_eng_05_035.7409_x0562739_05:19-20-21`** | They are green , suede , I do n't wear them all that often , because they are not __`that comfortable`__ , and they are falling apart .                                                                                                                             |
| **`pcc_eng_06_021.4078_x0330167_15:09-10-11`** | In fact , the only folks that were n't __`that comfortable`__ around Jesus - the only people who would n't invite Jesus to party - were the religious .                                                                                                             |


15. _that clear_

|                                                 | `token_str`                                                                                                                                                                                                                              |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_05_033.0708_x0519508_126:20-22-23`** | This is totally a lot much better than to wear a really fancy wedding dress but the value is not really __`that clear`__ .                                                                                                               |
| **`pcc_eng_10_029.0775_x0453834_52:43-44-45`**  | But if I have to choose from the first seven kings , my favorite goes to The Mystic , where he seems to have more portion to talk with Nyx in Kingsglaive : Final Fantasy XV movie , although his shape is n't __`that clear`__ either . |
| **`pcc_eng_16_022.0277_x0340541_18:28-29-30`**  | D Visualization is considered as the best tool for design professionals and facility users , as they are able to explore and understand special structures which are not __`that clear`__ on 2D drawings .                               |
| **`pcc_eng_04_001.7323_x0011889_41:5-7-8`**     | The broader issues are not always __`that clear`__ , however .                                                                                                                                                                           |
| **`pcc_eng_25_002.8456_x0030025_10:17-18-19`**  | They say you can see it from Tampa when it 's clear , but it was n't __`that clear`__ .                                                                                                                                                  |


16. _that popular_

|                                                | `token_str`                                                                                                                                                                                                                                                  |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_26_034.4971_x0541487_12:6-7-8`**    | " While the PC is not __`that popular`__ in front of general consumers , the company could be struggling in terms of recovering the profitability it had before . "                                                                                          |
| **`pcc_eng_26_085.0632_x1359461_04:25-26-27`** | With the exception of the " Medium " and " High " buy - in PL 5 - Card Draw events ( which are never __`that popular`__ with players ) , the events in the SCOOP series so far have had prize pools that have easily topped the guarantees from Pokerstars . |
| **`pcc_eng_12_030.7767_x0482120_062:5-6-7`**   | Fire tax hikes were not __`that popular`__ throughout the state .                                                                                                                                                                                            |
| **`pcc_eng_18_038.0052_x0598726_05:18-19-20`** | The economy is not growing that strongly and , partly as a consequence , President Obama is not __`that popular`__ .                                                                                                                                         |
| **`pcc_eng_26_034.6850_x0544490_08:23-24-25`** | " He 's amazing and should be regarded as one of the top three athletes Germany has ever had but golf is not __`that popular`__ and people over there do n't see it that way . "                                                                             |


17. _that simple_

|                                                | `token_str`                                                                                                                                                                                                 |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19960329_0696_3:09-12-13`**         | as is the custom lately , things would not be quite __`that simple`__ .                                                                                                                                     |
| **`pcc_eng_12_069.9272_x1113535_49:3-4-5`**    | Now was n't __`that simple`__ ?                                                                                                                                                                             |
| **`pcc_eng_15_013.0538_x0194490_08:09-11-12`** | She wants to belong , but life is n't always __`that simple`__ .                                                                                                                                            |
| **`nyt_eng_19980728_0070_16:5-7-8`**           | but the issue is n't quite __`that simple`__ .                                                                                                                                                              |
| **`pcc_eng_20_083.5545_x1333932_09:33-34-35`** | If you 're going to a decade themed party , getting a wig that matches your outfit is essential because re-creating most hairstyles that people used to wear in previous decades is n't __`that simple`__ ! |


18. _that easy_

|                                             | `token_str`                                                                                                                                |
|:--------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19990830_0188_15:5-6-7`**        | unfortunately , it 's not __`that easy`__ .                                                                                                |
| **`pcc_eng_14_096.3377_x1541308_53:4-5-6`** | It 's just not __`that easy`__ to generate income from viral videos , even using some app like this .                                      |
| **`nyt_eng_20070509_0290_3:21-22-23`**      | when an estimated one in five people has mental health issues , Dillinger continued , identifying them is `` just not __`that easy`__ . '' |
| **`pcc_eng_05_097.6227_x1563216_45:3-4-5`** | It is not __`that easy`__ , certainly not for so many African - Americans .                                                                |
| **`pcc_eng_04_084.3729_x1346954_42:5-6-7`** | Having an impact is not __`that easy`__ .                                                                                                  |


19. _that big_

|                                                | `token_str`                                                                                                                                                                                                                                                                                         |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_09_046.8438_x0741738_15:3-4-5`**    | He 's not __`that big`__ , but he hits hard .                                                                                                                                                                                                                                                       |
| **`pcc_eng_18_086.9031_x1391103_03:47-48-49`** | While there , they meet the handsome Spanish artist Juan Antonio ( Javier Bardem ) and his suicidal artist ex-wife , Maria Elena ( Penelope Cruz ) and engage in various love-triangles revolving around the Spanish stud , including a three - way that actually is n't __`that big`__ of a deal . |
| **`pcc_eng_18_089.7076_x1436496_05:14-15-16`** | You can find lots of fun t-shirts on Costume Squad if you 're not __`that big`__ on dressing up in a costume .                                                                                                                                                                                      |
| **`pcc_eng_08_107.6246_x1726385_13:10-11-12`** | It all worked out perfectly because the market was n't __`that big`__ and we had just enough time to look around before heading to a quaint little restaurant called The Pantry for lunch before heading home .                                                                                     |
| **`pcc_eng_28_042.0801_x0664534_65:17-19-20`** | With the low cost of compatible 16GB USB flash drives these days , it really should not be __`that big`__ of a deal .                                                                                                                                                                               |


20. _that great_

|                                                 | `token_str`                                                                                                                                                                                                                                                        |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_069.9289_x1113561_40:10-11-12`**  | I 've seen reviews on how this fin is n't __`that great`__ but it is n't true !                                                                                                                                                                                    |
| **`pcc_eng_04_101.6472_x1626020_17:14-15-16`**  | The main course of swordfish was n't too bad , but it was n't __`that great`__ either .                                                                                                                                                                            |
| **`pcc_eng_05_088.0013_x1407675_143:18-19-20`** | A lot of black people know we have a lot of shit in our families that are n't __`that great`__ .                                                                                                                                                                   |
| **`pcc_eng_16_055.8010_x0886926_156:51-52-53`** | For what it 's worth these are lots of lightweight models of hiking boots out there and there are a couple of them here on this review , but the Salomon Quest 4D 2 GTX simply is n't one of them and weight is the one thing that it 's not __`that great`__ at . |
| **`pcc_eng_25_085.8963_x1373939_33:43-45-46`**  | " I honestly do n't feel like it 's been a problem for me , but in fairness , I did n't take a test 18 years ago to compare it , " said Astles , who added that his hearing was n't all __`that great`__ to begin with .                                           |


Saving Samples in /share/compling/projects/sanpi/results/top_AM/neg_bigram_examples/that/...

Samples saved as...
1. `neg_bigram_examples/that/that_far-fetched_99ex.csv`
1. `neg_bigram_examples/that/that_thrilled_99ex.csv`
1. `neg_bigram_examples/that/that_uncommon_99ex.csv`
1. `neg_bigram_examples/that/that_surprising_99ex.csv`
1. `neg_bigram_examples/that/that_dissimilar_99ex.csv`
1. `neg_bigram_examples/that/that_unusual_99ex.csv`
1. `neg_bigram_examples/that/that_noticeable_99ex.csv`
1. `neg_bigram_examples/that/that_complicated_99ex.csv`
1. `neg_bigram_examples/that/that_familiar_99ex.csv`
1. `neg_bigram_examples/that/that_hard_99ex.csv`
1. `neg_bigram_examples/that/that_keen_99ex.csv`
1. `neg_bigram_examples/that/that_impressive_99ex.csv`
1. `neg_bigram_examples/that/that_fond_99ex.csv`
1. `neg_bigram_examples/that/that_comfortable_99ex.csv`
1. `neg_bigram_examples/that/that_clear_99ex.csv`
1. `neg_bigram_examples/that/that_popular_99ex.csv`
1. `neg_bigram_examples/that/that_simple_99ex.csv`
1. `neg_bigram_examples/that/that_easy_99ex.csv`
1. `neg_bigram_examples/that/that_big_99ex.csv`
1. `neg_bigram_examples/that/that_great_99ex.csv`

## 3. *exactly*

|                     |        `N` |      `f1` |   `adv_total` |
|:--------------------|-----------:|----------:|--------------:|
| **NEGATED_exactly** | 72,839,589 | 3,173,660 |        58,643 |
| **NEGMIR_exactly**  |  1,701,929 |   291,732 |         1,041 |


|                               |   `f` |   `adj_total` |   `dP1` |   `LRC` |      `G2` |   `exp_f` |   `unexp_f` |   `f2` |
|:------------------------------|------:|--------------:|--------:|--------:|----------:|----------:|------------:|-------:|
| **NEGany~exactly_conducive**  |   208 |         9,110 |    0.96 |    7.82 |  1,303.50 |      9.06 |      198.94 |    208 |
| **NEGany~exactly_shocking**   |   151 |        35,115 |    0.96 |    7.33 |    946.29 |      6.58 |      144.42 |    151 |
| **NEGany~exactly_pleasant**   |   142 |        52,223 |    0.96 |    7.24 |    889.88 |      6.19 |      135.81 |    142 |
| **NEGany~exactly_famous**     |   130 |       223,813 |    0.96 |    7.10 |    814.68 |      5.66 |      124.34 |    130 |
| **NEGany~exactly_difficult**  |   126 |       732,106 |    0.96 |    7.05 |    789.62 |      5.49 |      120.51 |    126 |
| **NEGany~exactly_easy**       | 1,066 |       579,827 |    0.95 |    9.32 |  6,596.91 |     46.75 |    1,019.25 |  1,073 |
| **NEGany~exactly_cheap**      |   691 |        60,531 |    0.95 |    8.96 |  4,281.59 |     30.28 |      660.72 |    695 |
| **NEGany~exactly_surprising** |   440 |        70,540 |    0.95 |    8.71 |  2,743.34 |     19.21 |      420.79 |    441 |
| **NEGany~exactly_new**        | 1,371 |       253,862 |    0.94 |    9.10 |  8,410.32 |     60.48 |    1,310.52 |  1,388 |
| **NEGany~exactly_clear**      | 1,746 |       349,214 |    0.94 |    8.78 | 10,578.33 |     77.73 |    1,668.27 |  1,784 |
| **NEGmir~exactly_sure**       |   148 |         6,761 |    0.83 |    5.31 |    522.11 |     25.37 |      122.63 |    148 |
| **NEGmir~exactly_easy**       |    20 |        18,610 |    0.83 |    1.86 |     70.55 |      3.43 |       16.57 |     20 |
| **NEGmir~exactly_clear**      |    52 |         6,722 |    0.81 |    3.38 |    173.89 |      9.08 |       42.92 |     53 |
| **NEGmir~exactly_new**        |    29 |        12,836 |    0.80 |    2.31 |     93.90 |      5.14 |       23.86 |     30 |


1. _exactly conducive_

|                                                | `token_str`                                                                                                                                                                                                 |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_19_045.4253_x0717120_21:1-2-3`**    | Not __`exactly conducive`__ to being a rock star .                                                                                                                                                          |
| **`nyt_eng_19991125_0190_34:21-22-23`**        | i 'm sure he realizes alternating three point guards and shuttling other players in and out of the lineup is n't __`exactly conducive`__ to team continuity and chemistry .                                 |
| **`pcc_eng_07_024.8862_x0386333_03:7-8-9`**    | However , Garcia 's style is n't __`exactly conducive`__ to having his hand raised either and has resulted in 3 - 6 - 1 record in his last ten tilts including losses in a quartet of consecutive clashes . |
| **`pcc_eng_04_054.9401_x0871212_25:13-14-15`** | I would caution you though that the setting of this book is not __`exactly conducive`__ to a morally clean setting , but Ms. Chaikin handled it very nicely .                                               |
| **`pcc_eng_21_074.5469_x1188529_13:21-22-23`** | Monday through Wednesday I 'm flying out to South Dakota to visit an operations center , and business trips are n't __`exactly conducive`__ to watching your calorie intake .                               |


2. _exactly shocking_

|                                                | `token_str`                                                                                                                                                                              |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_17_059.7527_x0949024_11:3-4-5`**    | Again , not __`exactly shocking`__ .                                                                                                                                                     |
| **`pcc_eng_14_003.1417_x0034674_45:11-12-13`** | The price is n't exactly cheap , but it 's not __`exactly shocking`__ , either .                                                                                                         |
| **`nyt_eng_20000418_0011_23:4-5-6`**           | but it was n't __`exactly shocking`__ , not for a man who has worked with the league owners since 1993 and was a known quantity within baseball 's political circle .                    |
| **`pcc_eng_14_086.3929_x1380285_09:6-7-8`**    | And yes , it 's not __`exactly shocking`__ that Wall Streeters want to leave big firms to form their own hedge funds .                                                                   |
| **`pcc_eng_18_001.0110_x0000163_23:3-4-5`**    | It 's not __`exactly shocking`__ , since , yes , the bull 's -eye on Gaylen 's head has been growing larger by the week , but the assassination 's quickness hits harder than expected . |


3. _exactly pleasant_

|                                                 | `token_str`                                                                                                                                                                                                                                       |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_00_069.6988_x1110332_24:24-25-26`**  | Crystal Fairy is tough at times to watch -- the characters are n't typically likable and Silva 's muddy , handheld aesthetic is n't __`exactly pleasant`__ .                                                                                      |
| **`pcc_eng_20_035.8938_x0564043_066:42-43-44`** | Things were kind of rushed at dinner time last night , and I just served it on croissants with no description , just like I serve regular chicken salad , and apparently the taste came as quite a surprise that was n't __`exactly pleasant`__ . |
| **`pcc_eng_06_073.2295_x1168344_215:3-4-5`**    | It 's not __`exactly pleasant`__ to read .                                                                                                                                                                                                        |
| **`pcc_eng_25_092.3663_x1478503_046:1-2-3`**    | Not __`exactly pleasant`__ to think about .                                                                                                                                                                                                       |
| **`pcc_eng_00_004.7264_x0060175_54:18-19-20`**  | The movie was a bomb with film critics and fans , and apparently making the film was n't __`exactly pleasant`__ for Munn .                                                                                                                        |


4. _exactly famous_

|                                               | `token_str`                                                                                                                                                                                                                                                                             |
|:----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_10_049.9895_x0792221_235:5-6-7`**  | Well , I was n't __`exactly famous`__ -- nobody knew who I was -- yet it felt as if I was famous because there were swarms of people trying to take a photo with me .                                                                                                                   |
| **`pcc_eng_20_003.9258_x0046965_5:3-4-5`**    | Well , not __`exactly famous`__ , but my new day after a long Thanksgiving holiday weekend began with such a delightful pleasure of likes cast upon by Dr. Suglia , a profound scholar of literature and philosophy and a sui generis writer par excellence , for my posts on my blog ! |
| **`pcc_eng_12_083.7662_x1337433_3:12-13-14`** | If you are lucky enough to live in the Netherlands ( not __`exactly famous`__ for its mountains ) and attend the University of Twente , you could have the chance .                                                                                                                     |
| **`pcc_eng_02_092.1554_x1473748_01:2-3-4`**   | Gujarat Not __`Exactly Famous`__ For Its Soldiers - Aakar Patel                                                                                                                                                                                                                         |
| **`pcc_eng_03_033.2877_x0522967_38:3-4-5`**   | Baseball is n't __`exactly famous`__ for making sound financial decisions .                                                                                                                                                                                                             |


5. _exactly difficult_

|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                               |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_27_051.6030_x0817819_026:21-22-23`** | Not many schools better than Uof C , and small class size makes it so getting big firm job is n't __`exactly difficult`__ .                                                                                                                                                                                                                               |
| **`pcc_eng_27_069.0405_x1100124_070:4-5-6`**    | Ryan Lochte 's not __`exactly difficult`__ to look at , either .                                                                                                                                                                                                                                                                                          |
| **`pcc_eng_00_075.5146_x1204581_28:14-15-16`**  | Re-implementing something like any ( ) , map ( ) , etc. is not __`exactly difficult`__ .                                                                                                                                                                                                                                                                  |
| **`pcc_eng_21_016.8255_x0255479_30:3-4-5`**     | It 's not __`exactly difficult`__ , is it ? "                                                                                                                                                                                                                                                                                                             |
| **`pcc_eng_00_062.1846_x0989130_17:43-44-45`**  | A neat image that fans of the comics - particularly the stellar " Long Halloween " series - can latch on to , one that does , in its way , give a small glimpse into the plot , although it was not __`exactly difficult`__ to figure out that Harvey Dent 's rise to the position of District Attorney was somehow going to be a part of the new movie . |


6. _exactly surprising_

|                                                | `token_str`                                                                                                                                                                                            |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_18_085.4516_x1367422_45:16-17-18`** | Some people begin to see me as a free meal ticket , but that 's not __`exactly surprising`__ .                                                                                                         |
| **`pcc_eng_17_007.9368_x0112216_29:3-4-5`**    | That 's not __`exactly surprising`__ to me considering I 'm new to all this .                                                                                                                          |
| **`pcc_eng_02_087.7692_x1402857_06:35-36-37`** | A few of Marvel 's major stars are set to exit the MCU after Infinity War , and between the battle with Thanos and a few contracts expiring behind the scenes , that 's not __`exactly surprising`__ . |
| **`pcc_eng_13_038.1122_x0600071_3:14-15-16`**  | And while encouraging , the lack of violent crimes for a day is n't __`exactly surprising`__ .                                                                                                         |
| **`pcc_eng_27_106.8027_x1711506_04:18-19-20`** | Shifting from an era when one man , Than Shwe , dominated the country , it is not __`exactly surprising`__ that politics in the reform period would be highly personalized .                           |


7. _exactly cheap_

|                                                | `token_str`                                                                                                                                             |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_05_004.6041_x0058545_32:5-6-7`**    | This prescription medication is not __`exactly cheap`__ .                                                                                               |
| **`pcc_eng_09_009.0138_x0129825_21:6-7-8`**    | " Cheap " money is not __`exactly cheap`__ .                                                                                                            |
| **`pcc_eng_17_058.0068_x0920985_31:12-13-14`** | The stock trades at 10.5 times revenue , so it 's not __`exactly cheap`__ .                                                                             |
| **`nyt_eng_20050509_0225_5:11-12-13`**         | here 's a look at two top-of-the-line PCs that are n't __`exactly cheap`__ , but they 'll do just about anything you need them to do -- and then some . |
| **`pcc_eng_25_009.0698_x0130764_15:7-8-9`**    | At 2000 euros , it 's not __`exactly cheap`__ , but definitely cool .                                                                                   |


8. _exactly easy_

|                                                | `token_str`                                                                                                                                                                 |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_04_106.9133_x1711045_03:25-26-27`** | Watching the show each week can be a fraught emotional experience -- death is often swift and brutal in Westeros -- but it 's not __`exactly easy`__ on the actors either . |
| **`pcc_eng_24_103.9223_x1665222_35:11-12-13`** | Once you 've been arrested or convicted , it is n't __`exactly easy`__ to find legitimate work .                                                                            |
| **`pcc_eng_03_005.6313_x0074826_37:5-6-7`**    | Answering this question is not __`exactly easy`__ .                                                                                                                         |
| **`pcc_eng_05_089.5852_x1433092_04:4-5-6`**    | But it is n't __`exactly easy`__ to care for .                                                                                                                              |
| **`pcc_eng_10_086.6351_x1384000_23:08-10-11`** | The company told ZDNet that Snapchat is n't " __`exactly easy`__ to get hold of . "                                                                                         |


9. _exactly new_

|                                                 | `token_str`                                                                                                                                                                            |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20060711_0296_42:3-4-5`**            | well , not __`exactly new`__ .                                                                                                                                                         |
| **`pcc_eng_06_107.9617_x1730227_038:25-26-27`** | The bass sound was largely influence by the likes of Meshuggah and Gojira , which was n't a bad thing per se but was n't __`exactly new`__ .                                           |
| **`pcc_eng_02_001.0536_x0000878_011:1-2-3`**    | Not __`exactly new`__ , but certainly limited in its production and its consumption .                                                                                                  |
| **`pcc_eng_14_031.7981_x0497623_16:5-6-7`**     | Well , it 's not __`exactly new`__ , but it 's definitely different from what most cities and towns have had in the past .                                                             |
| **`pcc_eng_18_089.2268_x1428683_30:14-15-16`**  | While China 's acts of censorship and routine banishments of Western technologies are n't __`exactly new`__ , their take - no- chances stance reflects a very real , logical concern . |


10. _exactly clear_

|                                                 | `token_str`                                                                                                                                                                                                                        |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_09_038.3661_x0604742_103:27-28-29`** | " He 's the funniest guy I 've ever met , " says Lochte , clapping his hands and laughing -- at what , it 's not __`exactly clear`__ .                                                                                             |
| **`pcc_eng_25_086.3400_x1381149_08:3-4-5`**     | It is not __`exactly clear`__ if the home of the candidate and that of the spouse are one and the same or whether the two maintain separate residences , however , for reasons hereinafter stated the outcome should be the same . |
| **`pcc_eng_28_047.3686_x0750336_13:3-4-5`**     | It is not __`exactly clear`__ whether Kokesh will carry through with the event , since he says it needs to reach a critical mass of 10,000 RSVPs first .                                                                           |
| **`pcc_eng_04_054.4254_x0863006_04:16-17-18`**  | The soldier 's free hand appears to have his fingers crossed , but it 's not __`exactly clear`__ .                                                                                                                                 |
| **`pcc_eng_06_025.3737_x0394504_123:16-17-18`** | Now , I get the argument for deliberate or not deliberate , and it 's not __`exactly clear`__ either way .                                                                                                                         |


11. _exactly sure_

|                                                 | `token_str`                                                                                                                                                                                 |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_11_014.1742_x0212980_02:1-2-3`**     | Not __`exactly sure`__ how I 'll use it , because it 's applicable for only one client at a time , whereas a website is applicable for multiple parties without using the special goggles . |
| **`pcc_eng_07_012.2839_x0182947_009:10-11-12`** | But now that you 're retiring , you 're not __`exactly sure`__ what to do with this newfound freedom .                                                                                      |
| **`pcc_eng_05_034.7676_x0546846_046:10-11-12`** | He did n't like it , and he was n't __`exactly sure`__ why he understood .                                                                                                                  |
| **`pcc_eng_26_007.6578_x0107292_12:12-13-14`**  | After reading the quotes from the ufo-ologists , I 'm still not __`exactly sure`__ what they want the government to do besides " act . "                                                    |
| **`pcc_eng_02_084.2673_x1346280_06:3-4-5`**     | I 'm not __`exactly sure`__ how it 's working in this context but the Note Length MIDI Effect is present in the Drum Rack and is clearly responsible .                                      |


Saving Samples in /share/compling/projects/sanpi/results/top_AM/neg_bigram_examples/exactly/...

Samples saved as...
1. `neg_bigram_examples/exactly/exactly_conducive_99ex.csv`
1. `neg_bigram_examples/exactly/exactly_shocking_99ex.csv`
1. `neg_bigram_examples/exactly/exactly_pleasant_99ex.csv`
1. `neg_bigram_examples/exactly/exactly_famous_99ex.csv`
1. `neg_bigram_examples/exactly/exactly_difficult_99ex.csv`
1. `neg_bigram_examples/exactly/exactly_surprising_99ex.csv`
1. `neg_bigram_examples/exactly/exactly_cheap_99ex.csv`
1. `neg_bigram_examples/exactly/exactly_easy_99ex.csv`
1. `neg_bigram_examples/exactly/exactly_new_99ex.csv`
1. `neg_bigram_examples/exactly/exactly_clear_99ex.csv`
1. `neg_bigram_examples/exactly/exactly_sure_99ex.csv`

## 4. *any*

|                 |        `N` |      `f1` |   `adv_total` |
|:----------------|-----------:|----------:|--------------:|
| **NEGMIR_any**  |  1,701,929 |   291,732 |         1,197 |
| **NEGATED_any** | 72,839,589 | 3,173,660 |        34,382 |


|                          |   `f` |   `adj_total` |   `dP1` |   `LRC` |     `G2` |   `exp_f` |   `unexp_f` |   `f2` |
|:-------------------------|------:|--------------:|--------:|--------:|---------:|----------:|------------:|-------:|
| **NEGmir~any_younger**   |    20 |           939 |    0.83 |    1.86 |    70.55 |      3.43 |       16.57 |     20 |
| **NEGmir~any_clearer**   |    17 |           130 |    0.83 |    1.50 |    59.97 |      2.91 |       14.09 |     17 |
| **NEGany~any_happier**   |   828 |        16,606 |    0.82 |    6.34 | 4,420.49 |     41.96 |      786.04 |    963 |
| **NEGmir~any_different** |    48 |        36,166 |    0.81 |    3.24 |   159.93 |      8.40 |       39.60 |     49 |
| **NEGmir~any_bigger**    |    36 |         3,923 |    0.80 |    2.73 |   118.17 |      6.34 |       29.66 |     37 |
| **NEGany~any_younger**   |   255 |        26,216 |    0.79 |    5.57 | 1,323.36 |     13.38 |      241.62 |    307 |
| **NEGany~any_nicer**     |    96 |         9,955 |    0.76 |    4.75 |   486.82 |      5.18 |       90.82 |    119 |
| **NEGmir~any_easier**    |    61 |         2,386 |    0.75 |    3.04 |   181.65 |     11.31 |       49.69 |     66 |
| **NEGmir~any_better**    |   380 |        14,013 |    0.74 |    4.38 | 1,096.01 |     71.82 |      308.18 |    419 |
| **NEGmir~any_good**      |    32 |        31,585 |    0.74 |    2.12 |    93.53 |      6.00 |       26.00 |     35 |
| **NEGmir~any_higher**    |    21 |         2,893 |    0.74 |    1.42 |    61.24 |      3.94 |       17.06 |     23 |
| **NEGany~any_simpler**   |   226 |        23,480 |    0.71 |    5.00 | 1,087.71 |     13.07 |      212.93 |    300 |
| **NEGany~any_brighter**  |    63 |         9,280 |    0.67 |    3.91 |   292.00 |      3.83 |       59.17 |     88 |
| **NEGmir~any_worse**     |    87 |         8,790 |    0.66 |    2.73 |   217.46 |     18.00 |       69.00 |    105 |
| **NEGmir~any_closer**    |    57 |           993 |    0.65 |    2.33 |   141.82 |     11.83 |       45.17 |     69 |
| **NEGany~any_smarter**   |    89 |         8,501 |    0.63 |    4.00 |   394.95 |      5.75 |       83.25 |    132 |
| **NEGany~any_easier**    | 1,594 |       209,940 |    0.62 |    5.08 | 6,987.78 |    104.79 |    1,489.21 |  2,405 |
| **NEGany~any_cheaper**   |   129 |        46,055 |    0.58 |    4.01 |   542.97 |      8.98 |      120.02 |    206 |
| **NEGany~any_clearer**   |   355 |        11,680 |    0.54 |    4.26 | 1,421.60 |     26.49 |      328.51 |    608 |
| **NEGany~any_worse**     | 1,686 |       179,012 |    0.42 |    3.94 | 5,676.37 |    160.03 |    1,525.97 |  3,673 |


1. _any younger_

|                                         | `token_str`                                                                                                                                                                                                             |
|:----------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`apw_eng_19981207_1742_12:13-15-16`** | and each year that goes by is another reminder that he is n't getting __`any younger`__ .                                                                                                                               |
| **`nyt_eng_19970125_0235_28:5-7-8`**    | Reggie , who was n't getting __`any younger`__ , wondered if perhaps time was running out for him although he never forgot his covenant nor lost his faith nor complained about the cold or about being double-teamed . |
| **`nyt_eng_20070325_0199_1:1-6-7`**     | none of them are getting __`any younger`__ .                                                                                                                                                                            |
| **`apw_eng_20080415_1153_21:19-21-22`** | although he does n't play in the field , baseball 's top home run hitter since 2004 is n't getting __`any younger`__ .                                                                                                  |
| **`nyt_eng_20071213_0030_29:07-09-10`** | but at 27 , I 'm not getting __`any younger`__ .                                                                                                                                                                        |


2. _any clearer_

|                                                 | `token_str`                                                                                                                                                           |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_00_066.2433_x1054653_006:14-16-17`** | The difference between the way we win matches and United manage to wo n't be __`any clearer`__ than after we welcome Leeds to the lane .                              |
| **`pcc_eng_04_084.4172_x1347684_36:2-3-4`**     | " Not __`any clearer`__ today , " said Chiarelli .                                                                                                                    |
| **`pcc_eng_29_033.5068_x0524668_30:3-5-6`**     | It could not be __`any clearer`__ to me that most Americans have tired of the war on drugs .                                                                          |
| **`pcc_eng_09_009.8529_x0143534_06:12-14-15`**  | During his response to the question about DAOs , Silbert could not be __`any clearer`__ in his support for the potential of Ethereum and the underlying ether token . |
| **`pcc_eng_12_065.5855_x1043962_04:16-18-19`**  | Wrapped in a cheerful , dreamy electro-pop melody with driving beats , the message could not be __`any clearer`__ !                                                   |


3. _any happier_

|                                                | `token_str`                                                                                                                                                                                                                                              |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_13_007.7011_x0108204_30:3-5-6`**    | I could n't be __`any happier`__ to speak louder , to speak clearer , and to speak more directly to the mission , the duty , of what we 've always been : an institution that educates young men in the most excellent tradition of the liberal arts . " |
| **`pcc_eng_27_055.1819_x0875672_18:17-19-20`** | I have done months of pricing and research before I committed to buying this and could n't be __`any happier`__ with the quality , effects and durability of this outstanding mixer !                                                                    |
| **`pcc_eng_17_043.3460_x0683882_09:4-6-7`**    | Happy Valley could n't be __`any happier`__ right now for Smith , the former Gateway High School star and coach , who was serving as an assistant coach at Temple until he got a phone call from Franklin .                                              |
| **`pcc_eng_04_108.09276_x1744207_17:3-4-5`**   | Eleanor is not __`any happier`__ about the situation , though she understands her father 's reasoning : he 's dying and wishes to see her happily settled first .                                                                                        |
| **`nyt_eng_20050829_0235_32:10-12-13`**        | `` I love her to death ; I could n't be __`any happier`__ than I am right now , '' Koso said , adding of Bruning : `` He 's a home wrecker .                                                                                                             |


4. _any different_

|                                                 | `token_str`                                                                                                                                                                                                                                                |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_17_075.7397_x1207980_20:08-09-10`**  | According to Turnmyre , these rules are n't __`any different`__ than the rules enforced in the past , so the skaters and bikers will not notice any differences .                                                                                          |
| **`pcc_eng_20_034.8488_x0547224_017:07-12-13`** | Valentinas adds , " It 's not that Hollywood pedophiles are __`any different`__ than the ones in a small town .                                                                                                                                            |
| **`nyt_eng_20060424_0275_37:5-7-8`**            | public financing would probably not be __`any different`__ .                                                                                                                                                                                               |
| **`pcc_eng_15_019.3993_x0297208_3:6-7-8`**      | The reaction in baseball was n't __`any different`__ than it was in football -- divisive and controversial -- but Maxwell made it clear at the time that he was from a military family , so he had the utmost respect for the military and the U.S. flag . |
| **`pcc_eng_14_084.6755_x1352504_37:6-8-9`**     | From a moral standpoint its not really __`any different`__ than masturbation ( which , despite some confusion about the story of Onan in the Bible , is not a sin in Christianity ) , but it is n't all people seem to think it would be .                 |


5. _any bigger_

|                                                | `token_str`                                                                                                                                                                                                                                                                                   |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_01_091.4802_x1462961_06:47-49-50`** | Interestingly enough , Arshavin 's goal against Aktobe came seven years and two days after he scored four goals at Anfield in what was the highlight of his stint in England , and the difference between Arsenal 's famous red and Central Stadium in Almaty could n't be __`any bigger`__ . |
| **`pcc_eng_13_039.2654_x0618708_15:3-4-5`**    | They are n't __`any bigger`__ than the wild turkeys I feed , and those birds she ignores with notable disdain .                                                                                                                                                                               |
| **`pcc_eng_26_035.9966_x0565730_43:4-5-6`**    | The cup is hardly __`any bigger`__ than the mouth of the jar , so it does n't work that well .                                                                                                                                                                                                |
| **`pcc_eng_12_087.2536_x1393762_20:6-7-8`**    | And so the balance is not __`any bigger`__ when compared to the prior thirty day period .                                                                                                                                                                                                     |
| **`nyt_eng_20070113_0155_19:23-25-26`**        | `` In college you have at least 10 guys throughout the country that are 250 to 265 -LRB- pounds -RRB- -- they never get __`any bigger`__ , so they 're not 4-3 ends .                                                                                                                         |


6. _any nicer_

|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                                                                            |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`apw_eng_20090814_1161_19:4-7-8`**           | `` He could n't have been __`any nicer`__ to them , '' Woolley added .                                                                                                                                                                                                                                                                                                                                 |
| **`pcc_eng_25_082.9038_x1325749_44:24-27-28`** | Instead of working with a megalomaniac , though , I found him to be this gentle , soft-spoken man , and he could n't have been __`any nicer`__ .                                                                                                                                                                                                                                                       |
| **`pcc_eng_22_059.4682_x0945278_112:7-8-9`**   | Relationships with the United States were n't __`any nicer`__ either as Kim would still portray the US as the bad guy and George W. Bush referring to North Korea as part of the ' axis of evil ' .                                                                                                                                                                                                    |
| **`pcc_eng_24_028.5471_x0445389_33:62-63-64`** | After this there is some vain effort of the movie to create a Snow White type story except instead of killing the daughter the stepmother wants to kill the Prince , at which point the movie kills off all of the boyfriends and ends with a cliffhanger where the nice boy , or in this case " the boy who is n't __`any nicer`__ but looks the most like Adam Sandler " is about to kiss the girl . |
| **`pcc_eng_24_033.1236_x0519651_80:3-4-5`**    | They are n't __`any nicer`__ than the cabins we 've seen on other ships .                                                                                                                                                                                                                                                                                                                              |


7. _any easier_

|                                                | `token_str`                                                                                                                                                                                                      |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_08_072.8917_x1163871_2:4-6-7`**     | This recipe could not be __`any easier`__ ; there 's no baking involved and it takes hardly any time to create these cute Cheesecake - Filled Chocolate Easter Eggs with a ' yolk ' made of passionfruit sauce . |
| **`pcc_eng_02_036.3409_x0572009_2:3-5-6`**     | It could n't be __`any easier`__ than using Aloha 2 GO .                                                                                                                                                         |
| **`pcc_eng_05_001.0850_x0001380_5:11-13-14`**  | After pressing , it takes minutes to make and could not be __`any easier`__ !                                                                                                                                    |
| **`nyt_eng_19961110_0020_26:3-4-5`**           | things were n't __`any easier`__ Saturday .                                                                                                                                                                      |
| **`pcc_eng_11_004.8877_x0063101_12:16-18-19`** | " It 's always difficult the deeper you go in the draw -- it wo n't be __`any easier`__ on Friday . "                                                                                                            |


8. _any good_

|                                             | `token_str`                                                                                                                                                                           |
|:--------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_29_033.5917_x0526037_19:4-5-6`** | The Republicans are n't __`any good`__ as they do it , only to a lesser degree of waste , more to the wealthy , and less crumbs to the poor .                                         |
| **`pcc_eng_18_084.9516_x1359345_23:7-8-9`** | The top personal injury lawyer will not __`any good`__ though you would require that divorce lawyer .                                                                                 |
| **`nyt_eng_20000919_0214_24:7-8-9`**        | `` Most of them just were n't __`any good`__ . ''                                                                                                                                     |
| **`nyt_eng_20000508_0513_7:11-12-13`**      | `` We are n't going to say their rankings are n't __`any good`__ , but in many ways Kansas City would top the list , '' said Becky Blades , chairwoman of the entrepreneurs council . |
| **`apw_eng_20081220_0329_7:26-27-28`**      | but Morozov , a 21-year-old university student who 's been gunning up support for the Moscow protest on YouTube , said Russian cars just are n't __`any good`__ .                     |


9. _any higher_

|                                                | `token_str`                                                                                                                                                                                                                        |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_13_006.1175_x0082499_39:14-16-17`** | In Gall 's view , the stakes for his UFC 203 bout could not be __`any higher`__ .                                                                                                                                                  |
| **`pcc_eng_09_001.1376_x0002178_04:6-8-9`**    | The price you pay will not be __`any higher`__ than if you had not shopped here ...                                                                                                                                                |
| **`pcc_eng_19_048.2343_x0762344_13:13-15-16`** | The gesture stands out even more when public hostility toward police could n't be __`any higher`__ , reminding us that despite all the riot gear and tear gas , police departments can care about the people they serve .          |
| **`pcc_eng_17_102.5790_x1642047_08:4-5-6`**    | Food prices are n't __`any higher`__ than you 'd see in a big city like Vancouver , and in the past few years , most Northern cities have gained some large supermarkets with a very decent selection of produce and other foods . |
| **`pcc_eng_02_080.3912_x1283657_21:19-20-21`** | You want to overexpose the white background by 2/3 to 1 stop over the key light , but not __`any higher`__ .                                                                                                                       |


10. _any better_

|                                                | `token_str`                                                                                                                                                                           |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_21_099.3335_x1588713_06:4-5-6`**    | And things were n't __`any better`__ over in Australia either , as the pound lost out on nearly one and a half cents there .                                                          |
| **`pcc_eng_24_022.5150_x0347577_15:19-22-23`** | " So I was sat in a perfect position in third and coming down the hill it could n't have been __`any better`__ .                                                                      |
| **`pcc_eng_20_080.9865_x1292283_11:22-23-24`** | As bad as the White Sox contract with Adam Dunn has been , the contracts given to Castro and Jackson are n't __`any better`__ .                                                       |
| **`pcc_eng_01_076.1207_x1214840_08:6-7-8`**    | In truth , Dove is n't __`any better`__ than all the other highly promoted toxic beauty products .                                                                                    |
| **`pcc_eng_26_009.2111_x0132609_08:09-10-11`** | However , a source said the offer was n't __`any better`__ than one they had rejected before the start of last season , which also included a guaranteed payment of their contracts . |


11. _any simpler_

|                                                | `token_str`                                                                                                                                                                                           |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_02_083.9536_x1341196_29:31-33-34`** | Workflow simplified Built to support the DV family of codecs , as well as the Quick Time animation codec , while using the industry standard .mov wrapper , workflow could not be __`any simpler`__ . |
| **`pcc_eng_02_094.5442_x1512577_36:3-5-6`**    | It could n't be __`any simpler`__ .                                                                                                                                                                   |
| **`nyt_eng_19961128_0003_5:23-25-26`**         | uncertainty remains about how this game will affect Robinson 's future , but for the 14 USC seniors , the predicament could not be __`any simpler`__ .                                                |
| **`pcc_eng_05_016.3328_x0248347_31:4-5-6`**    | Mainly they are not __`any simpler`__ than screens and require routine ladder maintenance .                                                                                                           |
| **`pcc_eng_01_100.1340_x1602483_03:14-16-17`** | The Lush Ice Stig is the perfect all in one device that could n't be __`any simpler`__ to use .                                                                                                       |


12. _any brighter_

|                                                | `token_str`                                                                                                                                                                                                                                                      |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_20_085.4477_x1364434_12:42-43-44`** | Actually , I do n't know my hippocampus from my amygdala , but it 's nice to know that all those mornings I 've sat on my butt wrestling silently with my monkey mind might actually keep me lucid -- if not __`any brighter`__ -- long into my crusty old age . |
| **`pcc_eng_27_104.5169_x1674612_34:13-15-16`** | In the biggest draft in franchise -history , the spot light will not be __`any brighter`__ than it is on the Flames organization .                                                                                                                               |
| **`pcc_eng_26_040.8483_x0644387_012:6-7-8`**   | Guess spies like him are n't __`any brighter`__ now than they were back in the 1940s !                                                                                                                                                                           |
| **`pcc_eng_19_042.2865_x0666602_11:32-33-34`** | As it stands , Black Berry is already in the red , losing as much as US $ 84 million in the last quarter ; the prediction for the next is n't __`any brighter`__ at this point .                                                                                 |
| **`pcc_eng_29_093.1157_x1487984_25:1-5-6`**    | Nor is the picture __`any brighter`__ in Scottish retail .                                                                                                                                                                                                       |


13. _any worse_

|                                                | `token_str`                                                                                                                                                                                                                                        |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_02_002.2781_x0020694_074:5-6-7`**   | Besides , it 's not __`any worse`__ than anything Motormaster 's done to me , Drag Strip rationalized , letting Smokescreen deepen the kiss .                                                                                                      |
| **`pcc_eng_04_070.0570_x1115376_09:3-5-6`**    | He ca n't be __`any worse`__ than douglas .                                                                                                                                                                                                        |
| **`pcc_eng_01_031.4085_x0491448_10:26-28-29`** | If there 's any positive for Von D , it 's that the embarrassment from having everyone watch her gush over her old boyfriend ca n't be __`any worse`__ than the embarrassment of having been involved with noted philanderer James to begin with . |
| **`pcc_eng_08_073.9026_x1180369_40:4-5-6`**    | Brett Favre is n't __`any worse`__ than Tony Romo which means that Donald Driver should have a monster game and Big Blue will be 0 - 2 .                                                                                                           |
| **`pcc_eng_12_039.2453_x0618572_056:1-7-8`**   | Nothing he came up with was __`any worse`__ than what she had already endured at the hands of vampires .                                                                                                                                           |


14. _any closer_

|                                                | `token_str`                                                                                                                                                            |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_03_034.0300_x0535048_09:3-6-7`**    | We could n't have been __`any closer`__ .                                                                                                                              |
| **`pcc_eng_18_085.2344_x1363893_45:4-6-7`**    | We really could n't be __`any closer`__ , and thinking of a quick 4 hour drive to go see my nephew on the weekends has me doing the never ending happy dance .         |
| **`pcc_eng_10_088.8470_x1419911_074:4-5-6`**   | And I was n't __`any closer`__ to coming out . "                                                                                                                       |
| **`pcc_eng_26_004.8324_x0061787_02:22-23-24`** | NASA has released stunning new views of those mysterious bright spots on the dwarf planet Ceres - but we 're still not __`any closer`__ to working out what they are . |
| **`pcc_eng_11_097.4880_x1561973_10:09-11-12`** | By which I mean I 'm not pregnant nor really __`any closer`__ to being pregnant .                                                                                      |


15. _any smarter_

|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                  |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_15_094.8222_x1516469_0949:4-5-6`**   | " You 're not __`any smarter`__ though , " He -man moved in .                                                                                                                                                                                                                                                                                                                                                |
| **`pcc_eng_17_071.0559_x1131909_25:3-4-5`**     | I 'm not __`any smarter`__ than they are , but I 'm real confident that , if it 's there , we can find it . "                                                                                                                                                                                                                                                                                                |
| **`pcc_eng_08_046.5222_x0736880_117:20-21-22`** | By the way , Jason Werth 's contract is more proof that some major league general manager 's are n't __`any smarter`__ than your middle - of- the - road fantasy baseballer .                                                                                                                                                                                                                                |
| **`pcc_eng_14_005.3539_x0070580_03:5-6-7`**     | Since I 'm probably not __`any smarter`__ than most of the people reading this column , I need to make sure I supply my readers with information that in one way or another is either uplifting , insightful to some degree , encouraging , entertaining , helpful in some way or emotionally stirring , bringing either a little lump to your throat or a little laugh or snicker ( or at least a smile ) . |
| **`pcc_eng_21_076.6557_x1222627_27:5-6-7`**     | The people highlighted are n't __`any smarter`__ or more capable than the rest of us .                                                                                                                                                                                                                                                                                                                       |


16. _any cheaper_

|                                                 | `token_str`                                                                                                                                                         |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_21_093.8012_x1499423_09:3-7-8`**     | It 's not like driving is __`any cheaper`__ .                                                                                                                       |
| **`pcc_eng_06_077.2619_x1233397_10:25-27-28`**  | If you are going to be setting up in photo restoration or trying it out for a hobby or whatever , Adobe Photoshop could not be __`any cheaper`__ right now . .      |
| **`pcc_eng_09_037.1937_x0585879_09:17-18-19`**  | So it actually required way more planning than buying pre-made organic juice -- and it was n't __`any cheaper`__ .                                                  |
| **`pcc_eng_20_007.7186_x0108344_112:3-4-5`**    | Prices were n't __`any cheaper`__ ( but also not much more expensive -- not always true in Canada ) , but the shipping was way less .                               |
| **`pcc_eng_22_055.2254_x0876246_041:26-27-28`** | This is because it is usually packed with NYU students who , one can only assume , do n't know any better because it is n't __`any cheaper`__ than the good stuff . |


Saving Samples in /share/compling/projects/sanpi/results/top_AM/neg_bigram_examples/any/...

Samples saved as...
1. `neg_bigram_examples/any/any_younger_99ex.csv`
1. `neg_bigram_examples/any/any_clearer_99ex.csv`
1. `neg_bigram_examples/any/any_happier_99ex.csv`
1. `neg_bigram_examples/any/any_different_99ex.csv`
1. `neg_bigram_examples/any/any_bigger_99ex.csv`
1. `neg_bigram_examples/any/any_nicer_99ex.csv`
1. `neg_bigram_examples/any/any_easier_99ex.csv`
1. `neg_bigram_examples/any/any_good_99ex.csv`
1. `neg_bigram_examples/any/any_higher_99ex.csv`
1. `neg_bigram_examples/any/any_better_99ex.csv`
1. `neg_bigram_examples/any/any_simpler_99ex.csv`
1. `neg_bigram_examples/any/any_brighter_99ex.csv`
1. `neg_bigram_examples/any/any_worse_99ex.csv`
1. `neg_bigram_examples/any/any_closer_99ex.csv`
1. `neg_bigram_examples/any/any_smarter_99ex.csv`
1. `neg_bigram_examples/any/any_cheaper_99ex.csv`

## 5. *remotely*

|                      |        `N` |      `f1` |   `adv_total` |
|:---------------------|-----------:|----------:|--------------:|
| **NEGATED_remotely** | 72,839,589 | 3,173,660 |        16,426 |
| **NEGMIR_remotely**  |  1,701,929 |   291,732 |         2,341 |


|                                 |   `f` |   `adj_total` |   `dP1` |   `LRC` |     `G2` |   `exp_f` |   `unexp_f` |   `f2` |
|:--------------------------------|------:|--------------:|--------:|--------:|---------:|----------:|------------:|-------:|
| **NEGany~remotely_surprising**  |    75 |        70,540 |    0.85 |    5.07 |   413.61 |      3.66 |       71.34 |     84 |
| **NEGmir~remotely_believable**  |    21 |           600 |    0.83 |    1.96 |    74.08 |      3.60 |       17.40 |     21 |
| **NEGmir~remotely_surprising**  |    17 |         2,662 |    0.83 |    1.50 |    59.97 |      2.91 |       14.09 |     17 |
| **NEGmir~remotely_comparable**  |    44 |           283 |    0.76 |    2.72 |   134.02 |      8.06 |       35.94 |     47 |
| **NEGmir~remotely_true**        |    61 |         6,191 |    0.75 |    3.04 |   181.65 |     11.31 |       49.69 |     66 |
| **NEGany~remotely_ready**       |    58 |       141,590 |    0.74 |    4.16 |   287.63 |      3.22 |       54.78 |     74 |
| **NEGmir~remotely_new**         |    19 |        12,836 |    0.69 |    1.00 |    50.62 |      3.77 |       15.23 |     22 |
| **NEGmir~remotely_close**       |   218 |        13,874 |    0.65 |    3.28 |   532.96 |     45.77 |      172.23 |    267 |
| **NEGany~remotely_true**        |   250 |       231,639 |    0.63 |    4.61 | 1,111.13 |     16.12 |      233.88 |    370 |
| **NEGmir~remotely_funny**       |    41 |         5,365 |    0.63 |    1.85 |    97.91 |      8.74 |       32.26 |     51 |
| **NEGany~remotely_funny**       |   137 |       122,927 |    0.60 |    4.15 |   589.74 |      9.24 |      127.76 |    212 |
| **NEGmir~remotely_interesting** |    56 |        12,447 |    0.56 |    1.80 |   115.20 |     13.20 |       42.80 |     77 |
| **NEGmir~remotely_possible**    |    38 |         3,160 |    0.56 |    1.42 |    78.73 |      8.91 |       29.09 |     52 |
| **NEGany~remotely_qualified**   |    57 |        74,643 |    0.52 |    3.13 |   222.79 |      4.40 |       52.60 |    101 |
| **NEGmir~remotely_similar**     |    71 |         7,011 |    0.49 |    1.70 |   127.32 |     18.34 |       52.66 |    107 |
| **NEGany~remotely_close**       |   694 |       411,329 |    0.39 |    3.65 | 2,243.22 |     69.58 |      624.42 |  1,597 |
| **NEGany~remotely_comparable**  |   118 |        12,252 |    0.38 |    2.98 |   375.73 |     12.07 |      105.93 |    277 |
| **NEGany~remotely_accurate**    |    50 |       152,299 |    0.33 |    2.10 |   143.78 |      5.84 |       44.16 |    134 |
| **NEGany~remotely_interested**  |   330 |       264,528 |    0.27 |    2.74 |   817.06 |     46.27 |      283.73 |  1,062 |
| **NEGany~remotely_similar**     |   152 |       203,453 |    0.23 |    2.20 |   334.61 |     24.36 |      127.64 |    559 |


1. _remotely surprising_

|                                                 | `token_str`                                                                                                                                                                                                                                                                         |
|:------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_22_085.4518_x1364936_020:24-25-26`** | Naturally , the reaction of President Donald J Trump to the latest atrocity was appalling , but , to anyone paying attention , not __`remotely surprising`__ .                                                                                                                      |
| **`pcc_eng_19_049.0450_x0775511_054:11-12-13`** | Starbound tries its hand at so much that it 's not __`remotely surprising`__ to see it dabbling in Metroidvania -style dungeons , as well .                                                                                                                                         |
| **`pcc_eng_test_1.1025_x01663_17:5-7-8`**       | The murder victim was n't even __`remotely surprising`__ nor was the murderer .                                                                                                                                                                                                     |
| **`pcc_eng_04_104.5051_x1672075_28:27-29-30`**  | There is no reason to celebrate Griner 's actions , but given our society 's obsession with sports , dominance , and aggression , it should n't be __`remotely surprising`__ that women entering into a male-dominated space will begin to measure their success in similar forms . |
| **`pcc_eng_24_087.1076_x1392625_05:14-16-17`**  | Well this is the talk of the office this morning , which is n't even __`remotely surprising`__ .                                                                                                                                                                                    |


2. _remotely believable_

|                                                | `token_str`                                                                                                                                                    |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_14_005.6855_x0075896_21:08-09-10`** | Is very boring , nothing unexpected and nothing __`remotely believable`__ .                                                                                    |
| **`nyt_eng_20050708_0167_69:16-18-19`**        | do n't try to shill for the GXP and the Impala , it 's just not even __`remotely believable`__ . ''                                                            |
| **`pcc_eng_03_032.9083_x0516822_21:13-18-19`** | There 's not much of a payoff at the end of , nor is the trial itself __`remotely believable`__ .                                                              |
| **`nyt_eng_19970623_0561_27:01-10-11`**        | none of this , needless to say , is __`remotely believable`__ _ nor is the apocalyptic ending that reunites the demented Devi with her alleged birth parents . |
| **`pcc_eng_00_062.0217_x0986499_27:3-5-6`**    | He 's not even __`remotely believable`__ as a legitimate " pundit " , there 's something so undeniably CRAZY about his thinking .                              |


3. _remotely comparable_

|                                                | `token_str`                                                                                                                                                                                                                                                  |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_19_016.2618_x0246224_090:3-4-5`**   | There is nothing __`remotely comparable`__ to Soviet-style government censorship and yet we have deliberate suppression of dissent .                                                                                                                         |
| **`pcc_eng_19_076.4855_x1219477_06:22-25-26`** | " Even now when you think about it in comparison to other events we 've had , summer and winter , nothing is even __`remotely comparable`__ to that event , " says Goldman .                                                                                 |
| **`pcc_eng_07_060.8241_x0967067_3:11-12-13`**  | Yes , yes , I know , the cases are not __`remotely comparable`__ .                                                                                                                                                                                           |
| **`pcc_eng_27_030.9843_x0483881_020:1-7-8`**   | Nothing on the present horizon is __`remotely comparable`__ to that .                                                                                                                                                                                        |
| **`pcc_eng_10_075.9474_x1211333_71:4-5-6`**    | " That is not __`remotely comparable`__ to the 5 million Syrians who fled the country in the first five years following the civil war - and that does n't include over a million per year who fled their homes inside Syria ( the internally displaced ) . " |


4. _remotely true_

|                                                 | `token_str`                                                                                                                                                                                                                                                  |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_00_004.9132_x0063132_2:4-5-6`**      | " It is not __`remotely true`__ that we are interested in Lichtsteiner , " he said in a press conference on the eve of their trip to Ajaccio on Friday .                                                                                                     |
| **`pcc_eng_14_083.4163_x1332170_29:3-4-5`**     | That 's not __`remotely true`__ ( we 're not even a paper ) , but even if it were , how does that refute our criticisms ?                                                                                                                                    |
| **`pcc_eng_13_007.1987_x0099976_12:44-45-46`**  | To begin , one must understand that while we often talk about abortion as though the issue is defined by a line running down the center of the American electorate , liberals on one side and conservatives on the other , that 's not __`remotely true`__ . |
| **`pcc_eng_29_044.1278_x0696845_097:09-10-11`** | This is only a manner of speaking and not __`remotely true`__ .                                                                                                                                                                                              |
| **`pcc_eng_20_006.3181_x0085665_11:10-11-12`**  | This is a lovely sentiment , but it 's not __`remotely true`__ .                                                                                                                                                                                             |


5. _remotely ready_

|                                                 | `token_str`                                                                                                                                                                                  |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19980427_0151_37:08-09-10`**         | Mark had finally confessed : He was n't __`remotely ready`__ to be a father .                                                                                                                |
| **`pcc_eng_07_017.0566_x0259758_096:11-13-14`** | I had been married less than a year and was n't even __`remotely ready`__ for kids .                                                                                                         |
| **`pcc_eng_19_075.9751_x1211201_07:11-12-13`**  | ' but if she 's dating a guy who 's not __`remotely ready`__ for children cougars reveal what it 's really like to date have you been pronouncing versace wrong .                            |
| **`pcc_eng_06_102.9020_x1648488_117:3-4-5`**    | I 'm not __`remotely ready`__ to talk about how to talk with kids about sex .                                                                                                                |
| **`pcc_eng_29_081.7315_x1304011_24:10-11-12`**  | He has to make peace with something he was n't __`remotely ready`__ to tackle , forced out of his shell to engage in a conflict sweeping the rest of the world whether he wanted to or not . |


6. _remotely new_

|                                                | `token_str`                                                                                                                                                                                                                                                                               |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_09_005.0515_x0065815_13:3-4-5`**    | This is n't __`remotely new`__ .                                                                                                                                                                                                                                                          |
| **`pcc_eng_25_099.4991_x1593996_18:4-5-6`**    | But there 's nothing __`remotely new`__ in this book beyond Wilson 's anti-Darwin bias .                                                                                                                                                                                                  |
| **`pcc_eng_16_058.5534_x0931869_15:14-16-17`** | For something the brand totes as " radically new , " there 's nothing even __`remotely new`__ about it .                                                                                                                                                                                  |
| **`pcc_eng_02_008.7063_x0124311_18:32-33-34`** | Teri Meherbaniyan is one of Jackie 's few solo hits and even then the trade openly talked about the dog outdoing him . :) But to be fair , there is nothing __`remotely new`__ or different in the film except for the title song .                                                       |
| **`pcc_eng_24_096.9629_x1552196_16:6-8-9`**    | The main narrative arc is not even __`remotely new`__ : boy goes from rags to riches , meets girl , break - up happens , he repurposes his life and finally wins the girl and a fortune with his special snowflake abilities , while he destroys the most evil big company in the world . |


7. _remotely close_

|                                                | `token_str`                                                                                                                                                                                                |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_03_008.8463_x0126923_17:17-18-19`** | So when we consider the purchasing power of the dollar , then our stock portfolio was n't __`remotely close`__ to flat , but rather had dropped by a staggering 70 % in fourteen years , from 919 to 274 . |
| **`pcc_eng_03_006.5670_x0089989_343:2-3-4`**   | Definitely nothing __`remotely close`__ to a blue .                                                                                                                                                        |
| **`pcc_eng_16_057.8963_x0921105_09:1-2-3`**    | Not __`remotely close`__ to being a bully .                                                                                                                                                                |
| **`pcc_eng_03_092.2383_x1477352_42:3-4-5`**    | This is n't __`remotely close`__ to what Reid claimed .                                                                                                                                                    |
| **`pcc_eng_14_087.1146_x1391904_29:3-5-6`**    | It 's not even __`remotely close`__ in terms of how many cards , and how many major power shifts , are allocated to dice rolls vs skillful play .                                                          |


8. _remotely funny_

|                                                | `token_str`                                                                                                                                                                                                                                        |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_07_057.0140_x0905249_14:12-14-15`** | And then let 's show that they 're not jokes , not even __`remotely funny`__ , as the Dalek cleanly and efficiently slaughters everyone in sight .                                                                                                 |
| **`pcc_eng_07_021.7718_x0336046_36:14-15-16`** | It 's common , its happened to people you know , and its not __`remotely funny`__ .                                                                                                                                                                |
| **`pcc_eng_14_088.5429_x1415038_07:18-22-23`** | Some of this sounds funny in writing , but thanks to witless execution and sloppy writing , none of it is __`remotely funny`__ on-screen .                                                                                                         |
| **`pcc_eng_07_055.6848_x0883912_10:22-23-24`** | They 're good for so many things , from venting and sorting feelings and seeing the humor in situations that are not __`remotely funny`__ in the moment , to tracking where you 've been and of course for assessing what worked and what didn't . |
| **`pcc_eng_27_063.1531_x1004836_23:08-10-11`** | That 's disturbing , disgusting , and not even __`remotely funny`__ .                                                                                                                                                                              |


9. _remotely possible_

|                                                 | `token_str`                                                                                                                                                                                                                                               |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`apw_eng_19971211_1274_20:16-18-19`**         | some ten years ago , many of us would have probably thought that this would not be __`remotely possible`__ .                                                                                                                                              |
| **`pcc_eng_16_055.7032_x0885357_51:3-5-6`**     | It 's not even __`remotely possible`__ that all inequality everywhere is due to the extension of patents and copyrights since 1981 .                                                                                                                      |
| **`pcc_eng_00_038.3633_x0603535_019:18-28-29`** | It 's not that we find ourselves in three wars and ca n't really understand why , not to mention if any sort of victory is even __`remotely possible`__ .                                                                                                 |
| **`pcc_eng_28_013.0626_x0195370_075:3-4-5`**    | This is not __`remotely possible`__ all the time , but you want to do it as much as possible .                                                                                                                                                            |
| **`apw_eng_20090511_1244_28:43-44-45`**         | `` The odds that you have Nazi wartime documents that put him at Sobibor , that former guards remember him and that he writes that he lived in Sobibor -- that he did n't serve at the death camp ? It 's not __`remotely possible`__ , '' Drimmer said . |


10. _remotely interesting_

|                                                 | `token_str`                                                                                                                                                                                                                                                       |
|:------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_29_011.1614_x0164316_12:1-5-6`**     | None of them were __`remotely interesting`__ .                                                                                                                                                                                                                    |
| **`pcc_eng_18_088.1871_x1411945_23:07-13-14`**  | If you 're like me , none of the above is even __`remotely interesting`__ except for the fact that it would likely prolong the ordeal , push the date of the next tip-off even further in the future , and roll our eyeballs even further back in their sockets . |
| **`pcc_eng_03_086.6062_x1386237_076:23-28-29`** | She tells her story through the eyes of half a dozen characters , all of whom speak in the same voice , none of whom are even __`remotely interesting`__ or convincing as human beings .                                                                          |
| **`pcc_eng_20_006.6013_x0090234_138:23-25-26`** | It is to be said however , that the introductory essay is too long , drawn out and in my opinion , not even __`remotely interesting`__ .                                                                                                                          |
| **`pcc_eng_07_027.7698_x0432922_3:10-12-13`**   | Vocally it 's woeful and depressing and there is nothing even __`remotely interesting`__ or enjoyable about any song on this crap album .                                                                                                                         |


11. _remotely qualified_

|                                                 | `token_str`                                                                                                                                                                                                                                                                                           |
|:------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_23_032.1435_x0502836_4:11-12-13`**   | My German is well below GCSE standard so I 'm not __`remotely qualified`__ to read the Ph D thesis in question , but nevertheless I 'm taken with the way that Der Spiegel has made elements of the document available on the web , along with sources , so that you can see the offending passages . |
| **`pcc_eng_25_085.2826_x1364049_17:22-25-26`**  | " The Skeleton Twins " can best be described as : two dysfunctional siblings telling one another how to live when neither is even __`remotely qualified`__ to give such advice .                                                                                                                      |
| **`pcc_eng_15_092.7806_x1483434_28:4-6-7`**     | If you are not even __`remotely qualified`__ for a position , do n't apply .                                                                                                                                                                                                                          |
| **`pcc_eng_03_037.9436_x0598305_32:27-28-29`**  | In the back of your mind , you constantly worry that every single parenting choice you 've made has been wrong , and that you are n't __`remotely qualified`__ for this business of raising young people .                                                                                            |
| **`pcc_eng_12_086.2209_x1377012_072:07-10-11`** | I 'm not a doctor , nor am I __`remotely qualified`__ to draw a conclusion .                                                                                                                                                                                                                          |


12. _remotely similar_

|                                                | `token_str`                                                                                                                                                                                                                        |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19990924_0222_26:5-6-7`**           | Kentucky 's offense is not __`remotely similar`__ to Tennessee 's .                                                                                                                                                                |
| **`nyt_eng_20000218_0186_47:1-2-3`**           | nothing __`remotely similar`__ went on at Paul Smith 's show Monday night .                                                                                                                                                        |
| **`pcc_eng_28_072.1910_x1151592_48:27-28-29`** | " Little Red Riding Hood " was almost certainly original , because earlier versions have not been recorded or do not seem to exist , and nothing __`remotely similar`__ can be found in older literature . [ 8 ]                   |
| **`pcc_eng_06_078.4987_x1253180_06:3-4-5`**    | There 's nothing __`remotely similar`__ .                                                                                                                                                                                          |
| **`pcc_eng_08_072.3153_x1154676_187:4-5-6`**   | My pace is n't __`remotely similar`__ to what it usually is , but it 's hard to judge ; I think I 've passed the border and entered New Jersey for over an hour when I finally reach the well - marked border -- as it should be ! |


13. _remotely accurate_

|                                                 | `token_str`                                                                                                                                                                                                                           |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_08_104.8274_x1681193_10:40-42-43`**  | A total smear to be sure , with its share of distortion , but still a concise biography of a man who has used a cozy relationship with the media to win popular support around a biography that 's not even __`remotely accurate`__ . |
| **`pcc_eng_11_081.2101_x1298323_42:1-4-5`**     | Nor is it __`remotely accurate`__ to state that " low - wage workers , schools , parents and advocates for the sick and disabled " do not enjoy a " level playing field " .                                                           |
| **`pcc_eng_21_104.5778_x1673867_36:3-4-5`**     | Again , not __`remotely accurate`__ .                                                                                                                                                                                                 |
| **`pcc_eng_16_058.3752_x0928899_07:3-5-6`**     | It 's not even __`remotely accurate`__ regarding the Numbers .                                                                                                                                                                        |
| **`pcc_eng_10_019.7965_x0303782_016:22-24-25`** | Thus Easterbrook 's claim that the IPCC TAR projected a 1 deg C global surface warming from 2000 to 2010 was not even __`remotely accurate`__ .                                                                                       |


14. _remotely interested_

|                                                | `token_str`                                                                                                                                                                                                                                                        |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_03_034.5954_x0544236_24:41-42-43`** | In an era when foreign policy debate has in most Western countries disappeared altogether , replaced by the reality TV of political entertainment , all of these things are much harder to explain and justify to a public that is n't __`remotely interested`__ . |
| **`nyt_eng_20000524_0257_10:1-4-5`**           | nor were they __`remotely interested`__ in `` democracy , '' in the way that their contemporaries , the veterans of Tiananmen Square , might define it .                                                                                                           |
| **`pcc_eng_00_035.9661_x0564825_07:07-13-14`** | And to his great distress , neither of Henry 's sons are __`remotely interested`__ in carrying on the family business .                                                                                                                                            |
| **`nyt_eng_20060127_0146_8:5-6-7`**            | `` But I was n't __`remotely interested`__ in what they 'd done . ''                                                                                                                                                                                               |
| **`pcc_eng_03_088.7082_x1420362_22:24-25-26`** | It was a manly night out , which means we get stupid by drinking too much , and smile at women that are n't __`remotely interested`__ in us .                                                                                                                      |


Saving Samples in /share/compling/projects/sanpi/results/top_AM/neg_bigram_examples/remotely/...

Samples saved as...
1. `neg_bigram_examples/remotely/remotely_surprising_99ex.csv`
1. `neg_bigram_examples/remotely/remotely_believable_99ex.csv`
1. `neg_bigram_examples/remotely/remotely_comparable_99ex.csv`
1. `neg_bigram_examples/remotely/remotely_true_99ex.csv`
1. `neg_bigram_examples/remotely/remotely_ready_99ex.csv`
1. `neg_bigram_examples/remotely/remotely_new_99ex.csv`
1. `neg_bigram_examples/remotely/remotely_close_99ex.csv`
1. `neg_bigram_examples/remotely/remotely_funny_99ex.csv`
1. `neg_bigram_examples/remotely/remotely_possible_99ex.csv`
1. `neg_bigram_examples/remotely/remotely_interesting_99ex.csv`
1. `neg_bigram_examples/remotely/remotely_qualified_99ex.csv`
1. `neg_bigram_examples/remotely/remotely_similar_99ex.csv`
1. `neg_bigram_examples/remotely/remotely_accurate_99ex.csv`
1. `neg_bigram_examples/remotely/remotely_interested_99ex.csv`

## 6. *ever*

|                     |        `N` |       `f1` |   `adv_total` |
|:--------------------|-----------:|-----------:|--------------:|
| **NEGMIR_ever**     |  1,701,929 |    291,732 |         5,060 |
| **NEGATED_ever**    | 72,839,589 |  3,173,660 |       114,075 |
| **COMPLEMENT_ever** | 72,839,589 | 69,662,736 |       114,075 |


|                           |   `f` |   `adj_total` |   `dP1` |   `LRC` |     `G2` |   `exp_f` |   `unexp_f` |   `f2` |
|:--------------------------|------:|--------------:|--------:|--------:|---------:|----------:|------------:|-------:|
| **NEGmir~ever_easy**      |   368 |        18,610 |    0.83 |    6.44 | 1,285.01 |     63.25 |      304.75 |    369 |
| **NEGmir~ever_simple**    |   206 |        25,408 |    0.83 |    5.82 |   726.76 |     35.31 |      170.69 |    206 |
| **NEGmir~ever_enough**    |   147 |         2,596 |    0.83 |    5.30 |   518.58 |     25.20 |      121.80 |    147 |
| **NEGmir~ever_certain**   |   143 |         1,800 |    0.83 |    5.26 |   504.47 |     24.51 |      118.49 |    143 |
| **NEGmir~ever_boring**    |    57 |         1,961 |    0.83 |    3.80 |   201.07 |      9.77 |       47.23 |     57 |
| **NEGmir~ever_black**     |    56 |         1,412 |    0.83 |    3.77 |   197.54 |      9.60 |       46.40 |     56 |
| **NEGmir~ever_sick**      |    39 |         1,895 |    0.83 |    3.15 |   137.57 |      6.69 |       32.31 |     39 |
| **NEGmir~ever_good**      |   299 |        31,585 |    0.82 |    5.68 | 1,013.87 |     51.94 |      247.06 |    303 |
| **NEGmir~ever_perfect**   |   206 |         3,134 |    0.82 |    5.57 |   714.47 |     35.48 |      170.52 |    207 |
| **NEGany~ever_simple**    |   211 |       396,749 |    0.80 |    5.56 | 1,109.28 |     10.89 |      200.11 |    250 |
| **NEGmir~ever_able**      |   136 |         3,704 |    0.78 |    4.17 |   426.52 |     24.51 |      111.49 |    143 |
| **NEGany~ever_boring**    |    72 |        45,891 |    0.76 |    4.46 |   362.74 |      3.92 |       68.08 |     90 |
| **NEGany~ever_easy**      |   429 |       579,827 |    0.73 |    5.41 | 2,105.13 |     24.18 |      404.82 |    555 |
| **NEGany~ever_certain**   |   147 |        74,952 |    0.72 |    4.80 |   713.34 |      8.41 |      138.59 |    193 |
| **NEGany~ever_sure**      |    87 |       262,825 |    0.49 |    3.34 |   328.20 |      7.06 |       79.94 |    162 |
| **NEGany~ever_good**      |   331 |     1,681,795 |    0.46 |    3.81 | 1,188.69 |     28.76 |      302.24 |    660 |
| **NEGany~ever_enough**    |   173 |       152,020 |    0.45 |    3.54 |   618.62 |     15.12 |      157.88 |    347 |
| **NEGany~ever_perfect**   |   216 |       104,659 |    0.40 |    3.34 |   706.71 |     21.31 |      194.69 |    489 |
| **NEGany~ever_satisfied** |    64 |        62,862 |    0.38 |    2.57 |   203.01 |      6.58 |       57.42 |    151 |
| **COM~ever_closer**       | 6,305 |        61,475 |    0.04 |    3.51 |   538.66 |  6,031.92 |      273.08 |  6,307 |


1. _ever simple_

|                                                | `token_str`                                                                                                                                                              |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_val_3.04586_x42025_16:3-7-8`**      | Historically , nothing about me is __`ever simple`__ or normal ( medically and otherwise ) .                                                                             |
| **`pcc_eng_23_039.3333_x0619416_06:29-31-32`** | First , some people have asked me for the cite on the Harper Collins publicity document I copied to my post of February 9th , With ARod , Nothing is __`Ever Simple`__ . |
| **`pcc_eng_11_083.1163_x1329077_21:23-25-26`** | In other words , even if we genuinely love someone , the realities of life have a way of reminding us that nothing is __`ever simple`__ ...                              |
| **`pcc_eng_06_102.2958_x1638582_01:4-6-7`**    | But of course nothing is __`ever simple`__ when bringing together national statistics .                                                                                  |
| **`pcc_eng_04_101.3371_x1621013_23:1-3-4`**    | Nothing is __`ever simple`__ and reality tends to be inconvenient .                                                                                                      |


2. _ever enough_

|                                                | `token_str`                                                                                                                                                                                                                                                      |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_09_006.0539_x0082006_15:1-7-8`**    | Nothing the writer can do is __`ever enough`__ .                                                                                                                                                                                                                 |
| **`pcc_eng_17_054.6155_x0866250_19:1-5-6`**    | Nothing I do is __`ever enough`__ to be excellent .                                                                                                                                                                                                              |
| **`pcc_eng_22_001.1797_x0002899_09:30-32-33`** | As I 've grown up it 's never left me , and has been poured into various projects , characters and creations but I 've never been satisfied , nothing was __`ever enough`__ and I always knew I could do more .                                                  |
| **`pcc_eng_18_084.5124_x1352360_034:2-4-5`**   | But nothing was __`ever enough`__ for him .                                                                                                                                                                                                                      |
| **`pcc_eng_11_085.3045_x1364599_50:48-50-51`** | Over the next few weeks , my little sister and I struggled to find closure by trying to remember him , getting a memorial tattoo , making a wreath for his grave on Memorial Day , using his t-shirts to make a quilt , but it 's not really __`ever enough`__ . |


3. _ever certain_

|                                                | `token_str`                                                                                                                                                                                     |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_078.9703_x1261259_11:07-09-10`** | But Ferguson knows from experience that nothing is __`ever certain`__ in football .                                                                                                             |
| **`pcc_eng_20_005.7871_x0077117_16:29-31-32`** | I have not wanted to write an update until things were absolutely certain , but I am afraid that our life thus far has proven to us that nothing is __`ever certain`__ until after it happens . |
| **`pcc_eng_20_009.0011_x0129103_26:23-25-26`** | Advancing any view or judgement is a no-no in the evidence - based research sphere , founded on the cardinal acceptance that nothing is __`ever certain`__ .                                    |
| **`pcc_eng_17_077.6713_x1238971_07:2-4-5`**    | While nothing is __`ever certain`__ in MMA , you can bet on fireworks when these two heavy handed fighters meet inside the O-Zone .                                                             |
| **`pcc_eng_27_058.7436_x0933359_11:1-3-4`**    | Nothing is __`ever certain`__ of course but , the more information the better if needing to profile an account reasonably quickly                                                               |


4. _ever boring_

|                                                 | `token_str`                                                                                                                                                                                                                               |
|:------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_03_006.3227_x0086059_152:15-17-18`** | " Sister " is special in that it is pensive and character - based without being __`ever boring`__ and it evokes deep emotions , yet is subjective and stays away from gooey sentimentality or blatant attempts at audience manipulation . |
| **`pcc_eng_27_064.0342_x1018997_06:6-8-9`**     | Above all , it was never , __`ever boring`__ .                                                                                                                                                                                            |
| **`pcc_eng_25_013.0080_x0194233_49:2-4-5`**     | Was n't it __`ever boring`__ to play the same five concertos for three years ?                                                                                                                                                            |
| **`pcc_eng_15_012.6522_x0188059_12:18-20-21`**  | It 's a show that goes for broke , does not apologize for its excesses and is never , __`ever boring`__ .                                                                                                                                 |
| **`pcc_eng_25_004.8609_x0062975_07:1-3-4`**     | Nothing is __`EVER boring`__ with me around ...                                                                                                                                                                                           |


5. _ever black_

|                                                | `token_str`                                                                                                                                           |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_16_052.8855_x0839845_08:03-15-16`** | However , nothing in politics , even when it comes to science , is __`ever black`__ and white .                                                       |
| **`pcc_eng_20_007.5800_x0106044_019:1-3-4`**   | Nothing is __`ever black`__ and white but this issue may hold the answer to the more fundamental question of - " What is school there for ? "         |
| **`pcc_eng_17_040.7624_x0642478_03:15-17-18`** | It 's a slightly surreal video that I hope encapsulates my philosophy of ' Nothing is __`ever black`__ and white ' .                                  |
| **`pcc_eng_22_058.3271_x0926728_09:18-20-21`** | But Trevor is about to discover that good and evil can look a lot alike , and nothing is __`ever black`__ and white -- not even the truth .           |
| **`pcc_eng_27_104.5460_x1675091_08:18-20-21`** | If I 've learnt anything in my twenty six years on the planet , it 's that nothing is __`ever black`__ and white ; there 's infinite shades of grey . |


6. _ever sick_

|                                                 | `token_str`                                                                                                                                         |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_00_035.7019_x0560528_13:7-8-9`**     | Maybe that 's why I 'm hardly __`ever sick`__ : I sleep as if I were sick ?                                                                         |
| **`pcc_eng_05_088.8188_x1420621_26:3-4-5`**     | J is hardly __`ever sick`__ & i'm quite sure it 's because he washes his hands so much !                                                            |
| **`pcc_eng_00_008.5074_x0121195_15:3-4-5`**     | I was n't __`ever sick`__ or dizzy or did I ever experience anything other than neck and head pain which I was able to control with pain medicine . |
| **`pcc_eng_07_016.3556_x0248479_10:3-4-5`**     | He 's hardly __`ever sick`__ .                                                                                                                      |
| **`pcc_eng_07_024.1556_x0374360_127:12-13-14`** | Compared to back then , I am healthy , happy and hardly __`ever sick`__ !                                                                           |


7. _ever easy_

|                                                | `token_str`                                                                                                                                   |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_22_055.1090_x0874381_12:12-14-15`** | This should n't be close , but as we know , nothing is __`ever easy`__ on the road .                                                          |
| **`pcc_eng_26_043.8679_x0693271_068:4-6-7`**   | Of course , nothing is __`ever easy`__ .                                                                                                      |
| **`pcc_eng_10_021.1137_x0324997_6:2-4-5`**     | But nothing is __`ever easy`__ in Walford ...                                                                                                 |
| **`pcc_eng_25_009.1004_x0131262_09:12-16-17`** | But recruiting has not been a strength of Kill 's ( not that it 's __`ever easy`__ with juggernauts Ohio State and Michigan in the league ) . |
| **`pcc_eng_14_096.3493_x1541510_025:1-3-4`**   | Nothing 's __`ever easy`__ ...                                                                                                                |


8. _ever perfect_

|                                                 | `token_str`                                                                                                                                                                                                                                                                              |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_28_011.9795_x0177767_095:30-31-32`** | You should always bulk that you are frequently and if that ending you are with , how old is stephen curry wife see nor character your sensitivity , do not __`ever perfect`__ yourself to be performed by merely being made a forcible .                                                 |
| **`pcc_eng_12_001.9586_x0015463_41:1-3-4`**     | Nothing is __`ever perfect`__ .                                                                                                                                                                                                                                                          |
| **`pcc_eng_19_074.8831_x1193534_066:2-6-7`**    | Well none of us is __`ever perfect`__ , and I know I still have negative thoughts and still have a long road to travel until I can say I can do real honour to the name ' Dreamwalker , and I know like you , we may forever pull ourselves up for making the same mistakes over again . |
| **`pcc_eng_29_006.4774_x0088640_028:2-4-5`**    | " Nothing is __`ever perfect`__ , " he always says .                                                                                                                                                                                                                                     |
| **`pcc_eng_03_095.2298_x1525666_39:2-4-5`**     | " Nothing is __`ever perfect`__ and I learned that last year when I felt perfect , " Williams said .                                                                                                                                                                                     |


9. _ever good_

|                                                 | `token_str`                                                                                                                                                                                            |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_28_013.4479_x0201674_101:3-5-6`**    | " Because nothing IS __`ever good`__ enough .                                                                                                                                                          |
| **`pcc_eng_27_065.5556_x1043570_150:12-14-15`** | The one thing that I 'm trying to embrace is that nothing is __`ever good`__ or bad .                                                                                                                  |
| **`pcc_eng_29_093.6824_x1497235_095:4-6-7`**    | Perfectionistic tendencies ; nothing is __`ever good`__ enough                                                                                                                                         |
| **`pcc_eng_09_007.9027_x0111868_107:32-34-35`** | Rousseau , for example , argued for the " ideal parent " , rational and complete - but his opponent Madame d'Epiney said that this " parent machine " ensures that nothing is __`ever good`__ enough . |
| **`pcc_eng_19_049.1285_x0776862_15:07-09-10`**  | Hope says she keeps trying but nothing 's __`ever good`__ enough .                                                                                                                                     |


10. _ever able_

|                                                 | `token_str`                                                                                                                                                                                                |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_10_047.5556_x0753258_094:10-11-12`** | I used to lament to him how I was n't __`ever able`__ to go back to Yankee Stadium after Thurman 's death .                                                                                                |
| **`pcc_eng_20_035.4087_x0556188_02:29-31-32`**  | A new CONAN movie has been in the works for some time with a number of people involved including the Wachowskis , John Milius and Robert Rodriguez but nothing was __`ever able`__ to get off the ground . |
| **`pcc_eng_14_085.9252_x1372749_17:23-25-26`**  | Artists are out there making and creating , many of them eager to share and talk about their work , but very few are __`ever able`__ to break into the high stakes art industry .                          |
| **`pcc_eng_13_088.0162_x1406374_036:08-09-10`** | So there was a neighbour who was hardly __`ever able`__ to accept a gift , for example some fruits from the garden .                                                                                       |
| **`pcc_eng_06_107.0901_x1716183_092:1-8-9`**    | Not even the greatest Aelfan wizards were __`ever able`__ to accomplish such a feat .                                                                                                                      |


11. _ever sure_

|                                                 | `token_str`                                                                                                                                                                                                                                           |
|:------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_19_079.6883_x1271238_31:08-09-10`**  | This is a blog post I was n't __`ever sure`__ I 'd write .                                                                                                                                                                                            |
| **`pcc_eng_29_084.0472_x1341322_29:14-15-16`**  | Normally I would refrain from telling her who was coming because I was not __`ever sure`__ who would really show up for her .                                                                                                                         |
| **`pcc_eng_18_030.8542_x0483214_049:30-31-32`** | I was so steeped in scarcity that I never EVER paid full price for anything , so I always ended up with 5 shirts on clearance that I was n't __`ever sure`__ I liked but thought I needed because my other shirts were getting too old or too small . |
| **`pcc_eng_25_081.5103_x1303404_15:16-18-19`**  | When the wind sweeps in , you know which way it is blowing , but no is __`ever sure`__ where it rose .                                                                                                                                                |
| **`pcc_eng_26_005.3020_x0069318_63:4-5-6`**     | " I 'm not __`ever sure`__ what they are thinking , but I tend to say ' Look , I might be making a mistake here , ' - I always say that - ' but I think this is the best team for today ' .                                                           |


12. _ever satisfied_

|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_084.0575_x1343432_43:2-4-5`**     | " Nobody is __`ever satisfied`__ that they get maximum exposure . "                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| **`pcc_eng_29_085.4475_x1364026_02:20-21-22`**  | I can only give you everything AMBERTONES I want you BOB DYLAN I shall be released ELVIS PRESLEY I ain't __`ever satisfied`__ JOHN MELLENCAMP ( happy bday John ) I could have stood you up KEITH RICHARDS I call your name BEATLES ( happy bday John Lennon ) I wanna be sedated JOHN HIATT I do n't want to grow up RAMONES ( happy bday Johnny and CJ Ramone ) I love you LONDON LEE I love you love me JOAN JETT and BLACKHEARTS I play drums THE DEL LORDS ( happy bday Del Lord ) I am your radio BOSS MARTIANS I ca n't stand up for falling down ELVIS COSTELLO & BRUCE SPRINGSTEEN ( happy bday Sam Moore ) I ca n't let go CONTINENTAL DRIFTERS I ca n't get you off of my mind WILLIE NILE I ca n't help it if i'm still in love with you HANK WILLIAMS I am alive ( live peak ) GARLAND JEFFREYS |
| **`pcc_eng_03_007.1777_x0099795_474:12-13-14`** | A lowkey but gutsy take on Steve Earle 's ' I Ain't __`Ever Satisfied`__ ' is effective if less forceful than Earle 's rocking original .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| **`pcc_eng_25_087.4014_x1398400_39:1-5-6`**     | None of us are __`ever satisfied`__ , one 's friends admit , before laughing maniacally and cartwheeling off into the Florentine night to harass strangers .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| **`nyt_eng_19970519_0243_27:11-12-13`**         | the search for those elusive `` family values '' is hardly __`ever satisfied`__ in my family , because no one ever seems to agree with my perfectly rational and commendable conception of what those values should be .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |


13. _ever closer_

|                                                | `token_str`                                                                                                           |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_09_042.0945_x0664783_19:11-13-14`** | Actually , in the long term , the rocket is not even that important .                                                 |
| **`pcc_eng_24_089.3777_x1429290_11:4-6-7`**    | And it would not be particularly difficult to give U.S. taxpayers the same option .                                   |
| **`pcc_eng_08_047.7928_x0757535_38:12-14-15`** | These guys trade big positions even though many of them are not really that good , they just are high on OPM .        |
| **`pcc_eng_01_095.7809_x1532508_06:16-17-18`** | When I was onboard a Carnival ship recently , I saw a busboy who appeared not terribly motivated , to say the least . |
| **`pcc_eng_21_072.7033_x1158795_64:08-09-10`** | First lap hurt , second lap was not that bad , but the third lap was hell .                                           |


Saving Samples in /share/compling/projects/sanpi/results/top_AM/neg_bigram_examples/ever/...

Samples saved as...
1. `neg_bigram_examples/ever/ever_simple_99ex.csv`
1. `neg_bigram_examples/ever/ever_enough_99ex.csv`
1. `neg_bigram_examples/ever/ever_certain_99ex.csv`
1. `neg_bigram_examples/ever/ever_boring_99ex.csv`
1. `neg_bigram_examples/ever/ever_black_99ex.csv`
1. `neg_bigram_examples/ever/ever_sick_99ex.csv`
1. `neg_bigram_examples/ever/ever_easy_99ex.csv`
1. `neg_bigram_examples/ever/ever_perfect_99ex.csv`
1. `neg_bigram_examples/ever/ever_good_99ex.csv`
1. `neg_bigram_examples/ever/ever_able_99ex.csv`
1. `neg_bigram_examples/ever/ever_sure_99ex.csv`
1. `neg_bigram_examples/ever/ever_satisfied_99ex.csv`
1. `neg_bigram_examples/ever/ever_closer_99ex.csv`

## 7. *yet*

|                 |        `N` |      `f1` |   `adv_total` |
|:----------------|-----------:|----------:|--------------:|
| **NEGATED_yet** | 72,839,589 | 3,173,660 |        95,763 |
| **NEGMIR_yet**  |  1,701,929 |   291,732 |           815 |


|                           |    `f` |   `adj_total` |   `dP1` |   `LRC` |      `G2` |   `exp_f` |   `unexp_f` |   `f2` |
|:--------------------------|-------:|--------------:|--------:|--------:|----------:|----------:|------------:|-------:|
| **NEGany~yet_eligible**   |    448 |        23,252 |    0.96 |    8.96 |  2,807.56 |     19.52 |      428.48 |    448 |
| **NEGany~yet_official**   |    352 |         6,853 |    0.96 |    8.61 |  2,205.93 |     15.34 |      336.66 |    352 |
| **NEGany~yet_convinced**  |    169 |        12,132 |    0.96 |    7.50 |  1,059.09 |      7.36 |      161.64 |    169 |
| **NEGany~yet_online**     |     98 |        15,650 |    0.96 |    6.66 |    614.14 |      4.27 |       93.73 |     98 |
| **NEGany~yet_mainstream** |     70 |        17,792 |    0.96 |    6.11 |    438.67 |      3.05 |       66.95 |     70 |
| **NEGany~yet_clear**      | 10,406 |       349,214 |    0.95 |   10.77 | 64,409.97 |    456.44 |    9,949.56 | 10,476 |
| **NEGany~yet_final**      |    640 |         5,860 |    0.95 |    8.97 |  3,972.92 |     28.02 |      611.98 |    643 |
| **NEGany~yet_over**       |    162 |         3,983 |    0.95 |    7.21 |  1,003.13 |      7.10 |      154.90 |    163 |
| **NEGany~yet_ready**      |  7,501 |       141,590 |    0.94 |    9.93 | 45,985.07 |    331.09 |    7,169.91 |  7,599 |
| **NEGany~yet_complete**   |  2,174 |        86,361 |    0.94 |    9.20 | 13,277.09 |     96.20 |    2,077.80 |  2,208 |
| **NEGmir~yet_available**  |     28 |        10,284 |    0.83 |    2.54 |     98.77 |      4.80 |       23.20 |     28 |
| **NEGmir~yet_certain**    |     21 |         1,800 |    0.83 |    1.96 |     74.08 |      3.60 |       17.40 |     21 |
| **NEGmir~yet_clear**      |     19 |         6,722 |    0.83 |    1.75 |     67.02 |      3.26 |       15.74 |     19 |
| **NEGmir~yet_sure**       |     19 |         6,761 |    0.83 |    1.75 |     67.02 |      3.26 |       15.74 |     19 |
| **NEGmir~yet_ready**      |     18 |         3,034 |    0.83 |    1.63 |     63.49 |      3.09 |       14.91 |     18 |


1. _yet eligible_

|                                                | `token_str`                                                                                                                                                                                                                                                      |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_15_015.1963_x0229141_21:17-18-19`** | An injured employee also may access their retirement benefits , even if the employee was otherwise not __`yet eligible`__ for retirement .                                                                                                                       |
| **`pcc_eng_23_034.5296_x0541444_35:21-22-23`** | That 's a big leap from next year , when Accenture expects private exchange enrollment , counting workers and retirees not __`yet eligible`__ for Medicare , to total 1 million .                                                                                |
| **`apw_eng_20031204_0435_17:16-17-18`**        | Johnson is eligible for salary arbitration for the first time this winter ; Rivera is n't __`yet eligible`__ .                                                                                                                                                   |
| **`pcc_eng_25_040.1656_x0633972_081:5-6-7`**   | Rivers and Roethlisberger are not __`yet eligible`__ , obviously , but I believe they 'll both get in .                                                                                                                                                          |
| **`pcc_eng_27_053.5705_x0849776_19:16-17-18`** | Individuals purchasing plans in the individual market tend to be self-employed , students , retirees not __`yet eligible`__ for Medicare , the unemployed , people between jobs , and those who are employed but do not take their employer -offered insurance . |


2. _yet official_

|                                                 | `token_str`                                                                                                                                                                                                                                   |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_23_080.1868_x1279530_12:6-7-8`**     | The deal with Axford is not __`yet official`__ , but will likely come to fruition after the pitcher undergoes a physical later this week .                                                                                                    |
| **`pcc_eng_18_085.7010_x1371502_15:11-12-13`**  | Pending publication in the Federal Register , the closure is not __`yet official`__ , but the emergency measures will be in effect when the bats return this winter to hibernate .                                                            |
| **`apw_eng_20090204_0383_4:3-4-5`**             | results are not __`yet official`__ because some lower-ranked skiers are still to race .                                                                                                                                                       |
| **`pcc_eng_06_079.0512_x1262178_100:10-11-12`** | It 's important to note that the move is n't __`yet official`__ , but the word around Texas Tech circles is Silva 's father is ill , and his family has serious financial needs .                                                             |
| **`pcc_eng_24_073.7862_x1177440_04:18-19-20`**  | Strange is a passion project for Marvel Studios president of production Kevin Feige and while it 's not __`yet official`__ we know Doctor Strange will be hitting theaters in July 2016 after Captain America 3 kicks off the summer in May . |


3. _yet convinced_

|                                         | `token_str`                                                                                                                                                                                                                                     |
|:----------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`apw_eng_20020206_0348_17:6-7-8`**    | but India says it is not __`yet convinced`__ about the sincerity of steps being taken by Musharraf .                                                                                                                                            |
| **`apw_eng_20021017_0641_7:17-18-19`**  | there was no opposition to Sinanan 's nomination , but some opposition lawmakers said they were not __`yet convinced`__ that Manning 's party , which won 20 seats in the this month 's election , was committed to unity in political policy . |
| **`nyt_eng_19960412_0476_22:5-6-7`**    | maybe the engineers are not __`yet convinced`__ that the machinery _ the pulleys are linked by a flexible steel drive belt that operates under enormous stress _ is as durable as the conventional automatic sold with other Civic models .     |
| **`nyt_eng_19970422_0730_20:5-6-7`**    | many doctors also are not __`yet convinced`__ cholesterol is a risk factor , in part because the research only recently yielded convincing evidence that LDL really is dangerous .                                                              |
| **`nyt_eng_20000716_0135_19:11-12-13`** | the boom has happened so speedily that some people are not __`yet convinced`__ that it is here to stay .                                                                                                                                        |


4. _yet online_

|                                                | `token_str`                                                                                                                                                                                                                                                                            |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_val_1.0279_x00462_12:15-16-17`**    | What 's more , our research shows that of the small businesses that 're not __`yet online`__ , 70 % would consider establishing and developing a website if it was easy to do and free of charge . "                                                                                   |
| **`pcc_eng_01_035.3009_x0554168_05:15-16-17`** | But , curiously , there are still a number of SMBs , which are not __`yet online`__ , or neglecting their online presence , either due to ignorance , offline considerations or a fear of alerting their competition to product features which give them the edge in the marketplace . |
| **`pcc_eng_24_107.00738_x1716226_06:6-7-8`**   | The app , which is not __`yet online`__ , does n't let the sad sack who 's missing the show hear the live versions of the songs , which would present significant copyright hurdles , as cool as that would be .                                                                       |
| **`pcc_eng_17_100.4950_x1608272_75:21-22-23`** | In India , for example , the population is growing and becoming more literate , but a substantial portion is not __`yet online`__ .                                                                                                                                                    |
| **`pcc_eng_29_090.5965_x1447125_07:5-6-7`**    | ( The episode is not __`yet online`__ , but will likely be available after the show repeats on Wednesday . )                                                                                                                                                                           |


5. _yet mainstream_

|                                                | `token_str`                                                                                                                                                                                                          |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_27_027.7118_x0431512_11:7-8-9`**    | Smart packaging , however , is not __`yet mainstream`__ .                                                                                                                                                            |
| **`pcc_eng_00_032.5322_x0509605_25:32-33-34`** | Back in 2010 , when environmental photographer and activist Luka Tomac began the journey of documenting stories from the front lines of climate change , the realities of the crisis were not __`yet mainstream`__ . |
| **`pcc_eng_22_059.1536_x0940227_17:14-15-16`** | While they 've been around for a few years , these networks are n't __`yet mainstream`__ , but they 're fast becoming the weapon of choice for agtech startups .                                                     |
| **`pcc_eng_08_107.2832_x1720979_02:13-14-15`** | Smartwatches are more prevalent than every before , but fashion smartwatches are not __`yet mainstream`__ .                                                                                                          |
| **`pcc_eng_03_089.9885_x1441165_01:5-6-7`**    | Insects as food are not __`yet mainstream`__ in American culture .                                                                                                                                                   |


6. _yet final_

|                                                | `token_str`                                                                                                                                              |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_06_075.0274_x1197410_08:16-17-18`** | In some cases , rebates would be banned altogether under the proposal , which is not __`yet final`__ .                                                   |
| **`pcc_eng_07_051.7803_x0820998_30:10-11-12`** | The avatar skeleton changes introduced by Project Bento are not __`yet final`__ because we want your feedback , either in the Creation Forumor in JIRA . |
| **`apw_eng_20090112_0900_2:12-13-14`**         | the person spoke on condition of anonymity because the deal was not __`yet final`__ .                                                                    |
| **`pcc_eng_18_008.6133_x0123205_11:5-6-7`**    | The death toll was not __`yet final`__ .                                                                                                                 |
| **`pcc_eng_23_048.6152_x0769304_21:08-09-10`** | The look of the aliens , while not __`yet final`__ , were largely faithful to their comic book counterparts .                                            |


7. _yet over_

|                                          | `token_str`                                                                                                                                                                        |
|:-----------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19960716_0328_21:7-8-9`**     | though the mens ' collections are not __`yet over`__ , with American designers unveiling creations beginning next Monday , look for the slim trend to continue .                   |
| **`nyt_eng_20070307_0087_252:29-30-31`** | so we play out the game , Margret and I - putting down words , guided by good intentions , grateful that the story that matters most is not __`yet over`__ .                       |
| **`nyt_eng_19971027_0120_10:18-19-20`**  | `` There is another move to bonds to from equities as the currency crisis in Asia is not __`yet over`__ , '' said Katherine Beattie , a technical analyst with MMS International . |
| **`nyt_eng_20051115_0316_24:21-22-23`**  | but its executive director warned that the pension system 's troubles , and the risk to the agency , were not __`yet over`__ .                                                     |
| **`apw_eng_19971124_1197_14:11-12-13`**  | JAKARTA -LRB- JP -RRB- : The bumper box-office year is not __`yet over`__ , and there are still plenty of blockbusters headed our way .                                            |


8. _yet clear_

|                                                | `token_str`                                                                                                                                                                                                     |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20001103_0036_7:3-4-5`**            | it is not __`yet clear`__ what brought about this more responsible position .                                                                                                                                   |
| **`pcc_eng_08_040.8031_x0644248_48:11-12-13`** | Authorization is also a desired feature , but it is not __`yet clear`__ that enough APIs need Authorization to justify handling this at the hosting architecture level .                                        |
| **`pcc_eng_14_089.4942_x1430453_15:3-4-5`**    | What is n't __`yet clear`__ is whether they 're planning to re-enlist the most popular games in a list that includes Battlefield 2 , Command & Conquer 3 , Star Wars : Battlefront 2 and Neverwinter Nights 2 . |
| **`pcc_eng_21_064.7865_x1030755_31:3-4-5`**    | It is not __`yet clear`__ to what extent the volume of inquiries and complaints will increase , but increase it most assuredly will .                                                                           |
| **`pcc_eng_13_038.3655_x0604208_290:3-4-5`**   | It is not __`yet clear`__ who carried it out , who collaborated with them , and who paved the way for them .                                                                                                    |


9. _yet ready_

|                                                 | `token_str`                                                                                                                                                                                                                    |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_02_087.1470_x1392830_012:24-25-26`** | This story -- the one coming vivid , the writing on a roll , where it dragged when I knew the story was not __`yet ready`__ -- is the one I knew was there all along , so I could n't be happier .                             |
| **`pcc_eng_07_052.6056_x0834196_24:14-15-16`**  | Slocum said the Glass recorded video of the incident , but she is not __`yet ready`__ to make that video public .                                                                                                              |
| **`nyt_eng_19970702_0877_22:15-16-17`**         | Hall testified this year at his Palm Beach County trial that although he was not __`yet ready`__ to die , he wanted McIver to give him a fatal dose of prescription drugs when he faced only a prolonged period of suffering . |
| **`pcc_eng_03_082.4186_x1318445_019:27-28-29`** | This is what England , France , and Czechoslovakia should have done to Germany when it demanded the Sudetenland in 1938 , noting that Germany was not __`yet ready`__ for war .                                                |
| **`pcc_eng_02_031.8212_x0498811_19:32-33-34`**  | As you might know there is such a lot of work to take several photos of any model , then editing the photos and setting to the Web , I 'm not __`yet ready`__ .                                                                |


10. _yet complete_

|                                                | `token_str`                                                                                                                                                                                   |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_00_002.9690_x0031770_04:18-19-20`** | Yellen said in delivering the Fed 's semi-annual economic report to Congress that the economic recovery is not __`yet complete`__ and the Fed intends to keep providing significant support . |
| **`pcc_eng_20_082.0391_x1309343_03:27-29-30`** | All of those late nights in the library suddenly feel worth it , but you know with the student loans , the work of school is not quite __`yet complete`__ .                                   |
| **`pcc_eng_26_009.7511_x0141454_38:13-14-15`** | Some of these volumes will be used to digitize titles that are not __`yet complete`__ .                                                                                                       |
| **`pcc_eng_26_007.3961_x0103101_42:21-22-23`** | Much of the parish lies outside the greater New Orleans levee system , and construction projects to bolster protection are not __`yet complete`__ .                                           |
| **`pcc_eng_02_049.3957_x0782884_2:10-11-12`**  | They should also clearly state where a database is not __`yet complete`__ .                                                                                                                   |


11. _yet available_

|                                                | `token_str`                                                                                                                                                         |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_06_028.2943_x0441561_78:20-21-22`** | A " sea- based prompt-strike missile is in the early stages of development " so these details " are not __`yet available`__ , " she said in the written responses . |
| **`pcc_eng_28_033.5381_x0526069_103:6-7-8`**   | If any pandemic-specific attributes are not __`yet available`__ for your category , consider adding them to your business name or " about the business " section .  |
| **`pcc_eng_24_071.8281_x1145632_03:24-25-26`** | The preview dialog in the " N+ P folder " tab displays all local data at a glance , including data which is not __`yet available`__ in Vault .                      |
| **`pcc_eng_02_094.9727_x1519408_03:14-15-16`** | See the village and its people through our digital archives , including material not __`yet available`__ through this website .                                     |
| **`pcc_eng_06_030.2105_x0472684_10:3-4-5`**    | Coauthoring is not __`yet available`__ for customers in the Semi-annual channel .                                                                                   |


12. _yet certain_

|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_02_099.8209_x1597894_55:3-4-5`**     | Hynek is n't __`yet certain`__ that he will have access to the instrument .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| **`pcc_eng_18_003.0518_x0033373_035:71-72-73`** | Or regular and highly reproducible figs d and hall a the circulation can result unless the underlying cause secondary brain injury is the only injury the child has ever received human derived growth factor induces nephrogenesis and wilmsa tumor wt gene causing waardenburg syndrome type ofd am j physiol a neissf morphogenesis and function of the non dominant arm are preferred , it was hard to answer patients who are not __`yet certain`__ a rules of thumba general indications for transfer to icu new presentation of guillian - barre syndrome . |
| **`pcc_eng_05_038.6640_x0609729_09:3-4-5`**     | It is not __`yet certain`__ that LUKoil will be the Russian partner in the joint development with Kazakhstan , although it has been assured that its investment will be repaid , " Argus " said .                                                                                                                                                                                                                                                                                                                                                                  |
| **`apw_eng_20090108_0579_2:6-7-8`**             | in fact , it is n't __`yet certain`__ that he 'll give up his hand-held device once he takes office .                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| **`pcc_eng_08_071.4996_x1141577_150:3-4-5`**    | We 're not __`yet certain`__ who is behind this terrorist plot , but we 've narrowed it down to bumbling head coach Wade Phillips , slick handed place holder Tony Romo , or all world receiver Terrell Owens . "                                                                                                                                                                                                                                                                                                                                                  |


13. _yet sure_

|                                                | `token_str`                                                                                                                                                        |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_09_034.2127_x0537713_74:11-12-13`** | Chances are his existence is busy enough and he is n't __`yet sure`__ he wants to take on your drama too .                                                         |
| **`pcc_eng_25_034.7007_x0545579_41:4-5-6`**    | But we are not __`yet sure`__ how it will affect both Binay 's and Roxas ' chances in the poll results .                                                           |
| **`pcc_eng_03_017.9754_x0274522_086:3-4-5`**   | I 'm not __`yet sure`__ how replicable it is , because every situation is different .                                                                              |
| **`pcc_eng_15_041.4761_x0654445_10:14-15-16`** | For instance , when I knew that I was leaving Mississippi -- though not __`yet sure`__ to where -- there was suddenly a list of things I needed to do beforehand . |
| **`nyt_eng_19990109_0269_26:22-23-24`**        | as for the Kings , team executives held their first meeting a few days ago to plan a goodbye but are n't __`yet sure`__ how they 'll do it .                       |


Saving Samples in /share/compling/projects/sanpi/results/top_AM/neg_bigram_examples/yet/...

Samples saved as...
1. `neg_bigram_examples/yet/yet_eligible_99ex.csv`
1. `neg_bigram_examples/yet/yet_official_99ex.csv`
1. `neg_bigram_examples/yet/yet_convinced_99ex.csv`
1. `neg_bigram_examples/yet/yet_online_99ex.csv`
1. `neg_bigram_examples/yet/yet_mainstream_99ex.csv`
1. `neg_bigram_examples/yet/yet_final_99ex.csv`
1. `neg_bigram_examples/yet/yet_over_99ex.csv`
1. `neg_bigram_examples/yet/yet_clear_99ex.csv`
1. `neg_bigram_examples/yet/yet_ready_99ex.csv`
1. `neg_bigram_examples/yet/yet_complete_99ex.csv`
1. `neg_bigram_examples/yet/yet_available_99ex.csv`
1. `neg_bigram_examples/yet/yet_certain_99ex.csv`
1. `neg_bigram_examples/yet/yet_sure_99ex.csv`

## 8. *immediately*

|                         |        `N` |      `f1` |   `adv_total` |
|:------------------------|-----------:|----------:|--------------:|
| **NEGATED_immediately** | 72,839,589 | 3,173,660 |        96,973 |
| **NEGMIR_immediately**  |  1,701,929 |   291,732 |         1,195 |


|                                    |    `f` |   `adj_total` |   `dP1` |   `LRC` |       `G2` |   `exp_f` |   `unexp_f` |   `f2` |
|:-----------------------------------|-------:|--------------:|--------:|--------:|-----------:|----------:|------------:|-------:|
| **NEGany~immediately_possible**    |  1,000 |       245,272 |    0.91 |    7.62 |   5,845.77 |     45.92 |      954.08 |  1,054 |
| **NEGany~immediately_clear**       | 24,416 |       349,214 |    0.89 |    8.16 | 141,186.69 |  1,134.49 |   23,281.51 | 26,038 |
| **NEGany~immediately_reachable**   |    109 |         2,672 |    0.86 |    5.56 |     610.53 |      5.23 |      103.77 |    120 |
| **NEGany~immediately_sure**        |    138 |       262,825 |    0.82 |    5.40 |     738.65 |      6.97 |      131.03 |    160 |
| **NEGany~immediately_certain**     |     70 |        74,952 |    0.71 |    4.19 |     336.68 |      4.05 |       65.95 |     93 |
| **NEGany~immediately_available**   | 21,078 |       666,909 |    0.67 |    5.70 |  98,046.67 |  1,278.84 |   19,799.16 | 29,351 |
| **NEGany~immediately_able**        |    626 |       223,196 |    0.59 |    4.70 |   2,655.18 |     43.09 |      582.91 |    989 |
| **NEGany~immediately_forthcoming** |    133 |         7,473 |    0.55 |    3.89 |     540.70 |      9.72 |      123.28 |    223 |
| **NEGany~immediately_intuitive**   |     54 |        20,664 |    0.55 |    3.21 |     218.74 |      3.96 |       50.04 |     91 |
| **NEGmir~immediately_clear**       |     31 |         6,722 |    0.55 |    1.13 |      62.95 |      7.37 |       23.63 |     43 |
| **NEGany~immediately_obvious**     |  2,238 |       165,439 |    0.52 |    4.55 |   8,712.07 |    173.80 |    2,064.20 |  3,989 |
| **NEGmir~immediately_available**   |    162 |        10,284 |    0.42 |    1.85 |     241.53 |     47.14 |      114.86 |    275 |


1. _immediately possible_

|                                                | `token_str`                                                                                                                                                                     |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`apw_eng_20091106_0036_18:3-4-5`**           | it was not __`immediately possible`__ to verify either claim due to the dangerous nature of the region .                                                                        |
| **`apw_eng_20031022_0508_17:3-4-5`**           | it was not __`immediately possible`__ to obtain comment from the Ministry of Health .                                                                                           |
| **`apw_eng_20020714_0260_3:4-5-6`**            | but it was not __`immediately possible`__ to confirm that , since it was not known how many survivors had met with Mandela , and none of them appeared at his news conference . |
| **`pcc_eng_15_098.7948_x1580575_20:10-11-12`** | Due to the remoteness of the region it was not __`immediately possible`__ to verify the information .                                                                           |
| **`apw_eng_20020826_0267_8:3-4-5`**            | it was not __`immediately possible`__ to get comment from government or army officials .                                                                                        |


2. _immediately clear_

|                                                | `token_str`                                                                                                               |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------|
| **`apw_eng_19970409_1194_7:3-4-5`**            | it was not __`immediately clear`__ whether the hearing was held .                                                         |
| **`pcc_eng_21_063.1630_x1004576_12:3-4-5`**    | It was not __`immediately clear`__ whether the property has gone into contract or has simply been taken off the market .  |
| **`nyt_eng_20001120_0306_28:3-4-5`**           | it was not __`immediately clear`__ what punishment would be meted out by the party to Kato and his parliamentary allies . |
| **`apw_eng_19971103_0259_5:3-4-5`**            | it was not __`immediately clear`__ how many of the seven American inspectors still in Baghdad were involved .             |
| **`pcc_eng_02_099.1098_x1586315_09:10-11-12`** | The impact of the decision on Exxon Mobil was not __`immediately clear`__ .                                               |


3. _immediately reachable_

|                                                | `token_str`                                                                                                                                            |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_27_032.8962_x0515012_12:6-7-8`**    | China's Ministry of Health was not __`immediately reachable`__ for comment after the Xinhua report .                                                   |
| **`pcc_eng_22_059.9128_x0952367_17:4-5-6`**    | Mr. Roberts was not __`immediately reachable`__ for comment , though he posted to Twitter : " Yes , I 'll be leaving@Reuters , though not right away . |
| **`pcc_eng_02_096.6577_x1546677_08:5-6-7`**    | Hybrid Air Vehicles was not __`immediately reachable`__ by telephone .                                                                                 |
| **`pcc_eng_13_003.1816_x0035024_08:4-5-6`**    | PNB officials were not __`immediately reachable`__ for comment , while Pw C declined to comment on the matter .                                        |
| **`pcc_eng_00_072.7105_x1159292_09:16-17-18`** | The other three councilors , Charlie Miranda , Guido Maniscalco and Yolie Capin , were not __`immediately reachable`__ .                               |


4. _immediately sure_

|                                                | `token_str`                                                                                                                                                                                                                  |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_14_087.1863_x1393018_07:4-5-6`**    | If you are n't __`immediately sure`__ what kind of fence will provide your house the greatest visual appeal while still accommodating your budgetary needs , then here 's a complete analysis on choosing a fencing design . |
| **`pcc_eng_08_102.9408_x1650453_07:4-5-6`**    | If you are not __`immediately sure`__ what type of fence will provide your property the best visual appeal while adapting your budgetary needs , then here 's a complete analysis on choosing a fencing layout .             |
| **`pcc_eng_18_083.2969_x1332643_01:18-19-20`** | The officer , who was an acquaintance of the well -known traditional healer , said policemen were not __`immediately sure`__ what the smell was as Mbatha walked up to the desk at about 6 pm on Friday .                    |
| **`pcc_eng_13_091.1921_x1457746_08:5-6-7`**    | Swanson said she was not __`immediately sure`__ what prompted the new restriction on after-session access .                                                                                                                  |
| **`apw_eng_20080205_0116_13:10-11-12`**        | Charming Shoppes spokeswoman Gayle Coolick said Monday she was not __`immediately sure`__ how many employees were working during the robbery or how much cash was in the store at the time .                                 |


5. _immediately certain_

|                                                | `token_str`                                                                                                                                                                                                       |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_13_038.9102_x0612956_50:3-4-5`**    | It was not __`immediately certain`__ how the Bastrop blaze began but it appeared that two fires merged to form the " monster " fire , Amen said .                                                                 |
| **`pcc_eng_08_072.3121_x1154626_30:3-4-5`**    | It was not __`immediately certain`__ if he would attend the next talks though .                                                                                                                                   |
| **`pcc_eng_06_079.1498_x1263719_15:19-20-21`** | She has since been identified by the medical examiner 's office , Brunner said , but it was not __`immediately certain`__ whether Raffo was among the 17 victims officially counted among the dead as of Friday . |
| **`apw_eng_20091023_0366_3:12-13-14`**         | Zabit Khan says the exact cause of the blast Friday was not __`immediately certain`__ but that it could have been a remote-controlled bomb .                                                                      |
| **`pcc_eng_07_056.7684_x0901241_06:6-7-8`**    | Graham notes it is " not __`immediately certain`__ who else received the preliminary packet . "                                                                                                                   |


6. _immediately available_

|                                                | `token_str`                                                                                                                                                                          |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_08_077.4909_x1238437_16:6-7-8`**    | Attorney General de Jong was not __`immediately available`__ for comment .                                                                                                           |
| **`apw_eng_19980220_0155_11:20-21-22`**        | other details of Hyundai 's deal with Adaptec , a computer connections maker based in Milpitas , Calif. were not __`immediately available`__ .                                       |
| **`pcc_eng_20_045.2456_x0714754_09:08-09-10`** | Spokespersons for Goldman Sachs and Citigroup were not __`immediately available`__ for comment .                                                                                     |
| **`apw_eng_19980401_0980_2:1-3-4`**            | nobody was __`immediately available`__ at Amsterdam police to confirm the report .                                                                                                   |
| **`apw_eng_20080713_0319_3:3-4-5`**            | details were not __`immediately available`__ about what caused the crash Sunday , which appeared to take place in a downhill curve and badly tore Evans ' jersey in several places . |


7. _immediately able_

|                                                | `token_str`                                                                                                                                                                      |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_22_056.3504_x0894473_06:09-10-11`** | Ontario 's Ministry of the Attorney General was not __`immediately able`__ to confirm the plea .                                                                                 |
| **`pcc_eng_07_019.0089_x0291229_07:5-6-7`**    | A Nordstrom spokeswoman was n't __`immediately able`__ to clarify whether plans for the store have been downsized .                                                              |
| **`pcc_eng_15_013.0457_x0194349_22:3-4-5`**    | CNN was n't __`immediately able`__ to reach Mitsubishi for comment outside of regular business hours .                                                                           |
| **`pcc_eng_28_016.0476_x0243596_06:08-09-10`** | An official of the State Department was not __`immediately able`__ to confirm Pompeo 's arrival in Pyongyang , from where outside communication can be limited .                 |
| **`apw_eng_20080819_0955_13:3-4-5`**           | Keller was n't __`immediately able`__ to say how much luggage an animal-sniffing dog could cover , but said a drug dog can get through about 2,000 bags in an eight-hour shift . |


8. _immediately forthcoming_

|                                                | `token_str`                                                                                                                                                                                                                                         |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_05_034.7526_x0546596_10:7-8-9`**    | When the petting she wants is not __`immediately forthcoming`__ , she complains .                                                                                                                                                                   |
| **`pcc_eng_05_012.2159_x0181855_087:6-7-8`**   | This time , that was n't __`immediately forthcoming`__ .                                                                                                                                                                                            |
| **`pcc_eng_13_003.6560_x0042741_14:13-14-15`** | However , an expected formal announcement of the council 's communique was not __`immediately forthcoming`__ , prompting speculation that the US - led Coalition Provisional Authority , the occupying power , had either delayed or squelched it . |
| **`pcc_eng_26_043.1286_x0681385_42:7-8-9`**    | When this narrative of victimization was not __`immediately forthcoming`__ , we kicked into high gear to manufacture one .                                                                                                                          |
| **`pcc_eng_15_094.7416_x1515176_36:19-20-21`** | Are able to persist through difficult tasks and difficult times , even when these rewards and benefits are not __`immediately forthcoming`__ .                                                                                                      |


9. _immediately intuitive_

|                                                | `token_str`                                                                                                                                                |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_20_089.4677_x1429224_02:13-14-15`** | The idea of going into debt to pay off other debts is not __`immediately intuitive`__ , but it can actually be a useful tool to free you from bad debts .  |
| **`pcc_eng_20_006.2838_x0085088_035:4-5-6`**   | Natural selection is not __`immediately intuitive`__ .                                                                                                     |
| **`pcc_eng_18_048.2076_x0764100_22:08-09-10`** | This format , Jaroslovsky says , is not __`immediately intuitive`__ for those unfamiliar with handheld devices .                                           |
| **`pcc_eng_25_085.6377_x1369811_12:4-5-6`**    | Some tools are n't __`immediately intuitive`__ , like the resyncing process , and the fact that only Tune Wiki-approved editors can edit existing lyrics . |
| **`pcc_eng_02_009.8674_x0143229_70:3-4-5`**    | It was not __`immediately intuitive`__ this represented healthcare                                                                                         |


10. _immediately obvious_

|                                                | `token_str`                                                                                                                        |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_21_091.0490_x1455323_25:5-6-7`**    | [ If it 's not __`immediately obvious`__ , explain more about what the product or service is and what makes it stand out here ] .  |
| **`pcc_eng_12_037.5816_x0591677_26:3-5-6`**    | It may not be __`immediately obvious`__ , but there 's a lot of thought that goes into the interior design of restaurants .        |
| **`pcc_eng_22_052.9930_x0839960_20:17-18-19`** | With the naked eye , the difference between the especially small or large full Moon is not __`immediately obvious`__ .             |
| **`pcc_eng_val_1.5882_x09587_56:13-14-15`**    | You may be wondering where the CD player is as it 's not __`immediately obvious`__ on looking at the interior .                    |
| **`pcc_eng_16_029.3592_x0459018_46:12-13-14`** | Equally , though , defying expectations and going somewhere that is n't __`immediately obvious`__ can be a really effective move . |


Saving Samples in /share/compling/projects/sanpi/results/top_AM/neg_bigram_examples/immediately/...

Samples saved as...
1. `neg_bigram_examples/immediately/immediately_possible_99ex.csv`
1. `neg_bigram_examples/immediately/immediately_clear_99ex.csv`
1. `neg_bigram_examples/immediately/immediately_reachable_99ex.csv`
1. `neg_bigram_examples/immediately/immediately_sure_99ex.csv`
1. `neg_bigram_examples/immediately/immediately_certain_99ex.csv`
1. `neg_bigram_examples/immediately/immediately_available_99ex.csv`
1. `neg_bigram_examples/immediately/immediately_able_99ex.csv`
1. `neg_bigram_examples/immediately/immediately_forthcoming_99ex.csv`
1. `neg_bigram_examples/immediately/immediately_intuitive_99ex.csv`
1. `neg_bigram_examples/immediately/immediately_obvious_99ex.csv`

## 9. *particularly*

|                          |        `N` |      `f1` |   `adv_total` |
|:-------------------------|-----------:|----------:|--------------:|
| **NEGATED_particularly** | 72,839,589 | 3,173,660 |       513,668 |
| **NEGMIR_particularly**  |  1,701,929 |   291,732 |        13,003 |


|                                       |   `f` |   `adj_total` |   `dP1` |   `LRC` |     `G2` |   `exp_f` |   `unexp_f` |   `f2` |
|:--------------------------------------|------:|--------------:|--------:|--------:|---------:|----------:|------------:|-------:|
| **NEGany~particularly_likable**       |   119 |         8,160 |    0.85 |    5.53 |   657.49 |      5.79 |      113.21 |    133 |
| **NEGany~particularly_likeable**      |   106 |         5,902 |    0.84 |    5.34 |   579.07 |      5.23 |      100.77 |    120 |
| **NEGmir~particularly_novel**         |    54 |           320 |    0.83 |    3.71 |   190.49 |      9.26 |       44.74 |     54 |
| **NEGmir~particularly_comfortable**   |    44 |         4,642 |    0.83 |    3.36 |   155.21 |      7.54 |       36.46 |     44 |
| **NEGmir~particularly_revolutionary** |    39 |           485 |    0.83 |    3.15 |   137.57 |      6.69 |       32.31 |     39 |
| **NEGmir~particularly_radical**       |    31 |         1,072 |    0.83 |    2.73 |   109.35 |      5.31 |       25.69 |     31 |
| **NEGmir~particularly_fast**          |    26 |         1,259 |    0.83 |    2.39 |    91.71 |      4.46 |       21.54 |     26 |
| **NEGmir~particularly_surprising**    |   166 |         2,662 |    0.82 |    5.06 |   564.67 |     28.80 |      137.20 |    168 |
| **NEGany~particularly_forthcoming**   |    72 |         7,473 |    0.82 |    4.85 |   387.25 |      3.62 |       68.38 |     83 |
| **NEGany~particularly_religious**     |   485 |        28,028 |    0.81 |    6.12 | 2,585.71 |     24.62 |      460.38 |    565 |
| **NEGmir~particularly_new**           |   404 |        12,836 |    0.81 |    5.92 | 1,365.17 |     70.28 |      333.72 |    410 |
| **NEGmir~particularly_wrong**         |   212 |        20,880 |    0.81 |    5.05 |   702.22 |     37.20 |      174.80 |    217 |
| **NEGany~particularly_original**      |   360 |        37,594 |    0.80 |    5.86 | 1,894.60 |     18.56 |      341.44 |    426 |
| **NEGmir~particularly_remarkable**    |   108 |         3,238 |    0.80 |    4.24 |   354.53 |     19.03 |       88.97 |    111 |
| **NEGany~particularly_new**           |   747 |       253,862 |    0.79 |    6.04 | 3,874.46 |     39.21 |      707.79 |    900 |
| **NEGmir~particularly_close**         |   136 |        13,874 |    0.77 |    4.01 |   415.70 |     24.85 |      111.15 |    145 |
| **NEGany~particularly_athletic**      |   108 |        17,142 |    0.75 |    4.77 |   541.01 |      5.93 |      102.07 |    136 |
| **NEGany~particularly_revolutionary** |    77 |        10,338 |    0.75 |    4.49 |   385.60 |      4.23 |       72.77 |     97 |
| **NEGany~particularly_flashy**        |    57 |         4,494 |    0.73 |    4.08 |   278.96 |      3.22 |       53.78 |     74 |
| **NEGany~particularly_surprising**    | 1,069 |        70,540 |    0.57 |    4.71 | 4,433.52 |     75.94 |      993.06 |  1,743 |


1. _particularly likable_

|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                              |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_23_044.3575_x0700605_09:12-13-14`**  | But it 's also because the story 's other characters are n't __`particularly likable`__ .                                                                                                                                                                                                                                                |
| **`pcc_eng_28_045.4466_x0719287_15:5-6-7`**     | And its narrator is not __`particularly likable`__ .                                                                                                                                                                                                                                                                                     |
| **`pcc_eng_26_031.9732_x0500519_052:36-48-49`** | The almost-unbearable bittersweetness of Li's carefully closing Chou 's suitcase and handing it to her , and then standing by the window and watching her as she leaves , is complicated by the fact that neither of these characters , nor any other in Terrorizer , is __`particularly likable`__ .                                    |
| **`pcc_eng_22_006.5138_x0088996_18:10-11-12`**  | In short , Tom Rhys Harries 's character is n't __`particularly likable`__ , to a point where a predictable switchover into a redemptive , understanding attitude would n't really work .                                                                                                                                                |
| **`pcc_eng_22_006.7930_x0093446_17:51-52-53`**  | Made for only $ 30 million , far less than what it cost for Zemeckis to make " The Polar Express " or " A Christmas Carol , " this is more of a character driven drama from the 1970 's as it gives us a main character who is not __`particularly likable`__ , and yet we are compelled to follow him all the way to the movie 's end . |


2. _particularly likeable_

|                                                 | `token_str`                                                                                                                                                                                                |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_21_025.4152_x0394502_31:6-7-8`**     | Most of the characters are n't __`particularly likeable`__ , but they are n't ones you despise either .                                                                                                    |
| **`pcc_eng_12_084.0926_x1342573_38:6-7-8`**     | His portrayal of God is n't __`particularly likeable`__ ( in fact , none of the characters are likeable or relatable ) , but it is interesting .                                                           |
| **`pcc_eng_13_084.4099_x1348390_26:1-6-7`**     | None of the characters were __`particularly likeable`__ , and that is n't just me judging them based on their past offences - there just is n't much warmth to anyone in the book .                        |
| **`pcc_eng_17_042.4874_x0670147_32:7-8-9`**     | Teri Garr's Frannie is attractive but not __`particularly likeable`__ - the poorly -written domestic squabbles keep us from caring about the lovers , and their dreams are far too shallow to involve us . |
| **`pcc_eng_18_046.2336_x0732013_038:17-18-19`** | It creates the problem that our hero is basically just a slab of meat , and not __`particularly likeable`__ .                                                                                              |


3. _particularly novel_

|                                                 | `token_str`                                                                                                                                                                                                                         |
|:------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_10_049.5724_x0785567_29:6-7-8`**     | In truth , there 's nothing __`particularly novel`__ or disquieting about the scheme Murray 's drawn up , except insofar as the procedural extremism conservatives have deployed in the Obama era is alarming in general .          |
| **`pcc_eng_24_020.6021_x0316588_24:18-19-20`**  | The fact that the Greeks then used the first two letters to create the word alphabet is n't __`particularly novel`__ .                                                                                                              |
| **`pcc_eng_18_084.9988_x1360098_13:30-31-32`**  | Langewiesche 's presentation of nuclear weapons as the great equalizer ( with increasing potential of becoming ' the great leveller ' ) in the international system was interesting but not __`particularly novel`__ .              |
| **`pcc_eng_21_097.8756_x1565111_16:7-8-9`**     | ALEC ROSS : There 's actually nothing __`particularly novel`__ about any of this , except that we all get to see it .                                                                                                               |
| **`pcc_eng_20_088.9868_x1421466_013:27-28-29`** | For example , though it is great to see the mainstream media come to condemn the intellectual fallacies inherent with protectionist trade policies , there is nothing __`particularly novel`__ about a president imposing tariffs . |


4. _particularly comfortable_

|                                                | `token_str`                                                                                                                                                            |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_29_005.7456_x0076868_17:3-4-5`**    | Cycling is not __`particularly comfortable`__ , but you will get used to the strange position , along with all its aches and pains .                                   |
| **`pcc_eng_13_089.8960_x1436844_27:22-24-25`** | For Muslim women who don their natural hair , this could cause unnecessary pressure to conform to hair practices they may not be __`particularly comfortable`__ with . |
| **`pcc_eng_22_055.0046_x0872632_15:4-5-6`**    | " I 'm not __`particularly comfortable`__ being under the water , " Briggs told JTA , admitting he did n't learn to swim until the " ripe age " of 12 .                |
| **`pcc_eng_21_068.0499_x1083588_29:20-21-22`** | For his part , the cab driver seemed out of place , too , neither interested in the fight nor __`particularly comfortable`__ with it .                                 |
| **`pcc_eng_23_003.3323_x0037537_3:3-4-5`**     | I am not __`particularly comfortable`__ in bars or clubs .                                                                                                             |


5. _particularly revolutionary_

|                                                | `token_str`                                                                                                                                                                                                                                                                                    |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_00_004.6567_x0059087_04:02-10-11`** | While none of the following tactics / ideas are __`particularly revolutionary`__ or profound , when wisely integrated into standard PR practices such as press release distribution and reporter calls - they combine to elevate a relatively static campaign into a dynamic , effective one : |
| **`pcc_eng_11_058.6598_x0932931_6:3-4-5`**     | This was n't __`particularly revolutionary`__ when it comes to earbud design in general , but it is a step away from the hard earpieces Apple gives you when you buy its products .                                                                                                            |
| **`pcc_eng_15_092.7814_x1483448_52:3-4-5`**    | There is nothing __`particularly revolutionary`__ about keeping connected to the office while on the road with the aid of a smartphone , i Pad , or other type of tablet computer .                                                                                                            |
| **`pcc_eng_24_071.8081_x1145292_10:1-5-6`**    | None of this is __`particularly revolutionary`__ : Google Maps has had very good transit directions forever -- and is even introducing real -time transit updates .                                                                                                                            |
| **`pcc_eng_16_080.3918_x1284895_011:1-6-7`**   | None of my ideas are __`particularly revolutionary`__ , or even unique - something that any developer who gives a damn could implement .                                                                                                                                                       |


6. _particularly radical_

|                                                 | `token_str`                                                                                                                                                                                                                             |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_02_032.5647_x0510895_07:1-3-4`**     | None are __`particularly radical`__ ; indeed , each idea simply develops programs and policies decided on by earlier FCC administrations , some of them Republican .                                                                    |
| **`pcc_eng_05_085.4864_x1367191_29:4-5-6`**     | The story does nothing __`particularly radical`__ with the incarnations of the various characters but winds them together with a modest complexity .                                                                                    |
| **`pcc_eng_05_088.6709_x1418290_31:6-7-8`**     | Though his policy positions are n't __`particularly radical`__ by European standards , Sanders ' candidacy inspired hope in people who'd otherwise given up on seeing their opinions and interests represented in mainstream politics . |
| **`pcc_eng_20_082.7683_x1321025_27:4-5-6`**     | Though there 's nothing __`particularly radical`__ about religious life , it 's meant to be prophetic .                                                                                                                                 |
| **`pcc_eng_26_044.2462_x0699430_250:08-09-10`** | I 'm not cool and I am not __`particularly radical`__ in my everyday life .                                                                                                                                                             |


7. _particularly fast_

|                                             | `token_str`                                                                                                                                                                           |
|:--------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_11_010.8164_x0158838_09:3-4-5`** | It 's not __`particularly fast`__ or agile though , and it 's not nearly as light as many of its contemporaries , but the updates to this shoe have made the ride even more sublime . |
| **`pcc_eng_00_068.9032_x1097447_22:1-2-3`** | Not __`particularly fast`__ , powerful , or quick .                                                                                                                                   |
| **`pcc_eng_08_057.3081_x0911903_14:1-2-3`** | Not __`particularly fast`__ , but fast enough .                                                                                                                                       |
| **`pcc_eng_23_014.9085_x0224111_12:3-4-5`** | He is not __`particularly fast`__ or powerful or strong - and spends large parts of matches grazing .                                                                                 |
| **`pcc_eng_09_089.0499_x1424734_06:1-3-4`** | Neither are __`particularly fast`__ or efficient , and it 's not a particularly good idea to interrupt them ( with C-g or anything else ) once you 've started one of them .          |


8. _particularly forthcoming_

|                                                 | `token_str`                                                                                                                                                                                                                                      |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_02_002.5441_x0025004_15:5-6-7`**     | Predictably , they were n't __`particularly forthcoming`__ .                                                                                                                                                                                     |
| **`apw_eng_19971112_1107_22:24-26-27`**         | Nazir said Mesdaq will serve as an alternative source of capital for growth companies , especially since venture capital companies and banks have not been __`particularly forthcoming`__ in the past in assisting such start-up operations .    |
| **`pcc_eng_23_039.3177_x0619144_077:29-31-32`** | Doombot shows up to give Victor a new body , but Victor does n't like the buff , weaponized Transformers body that Doombot makes , though Victor is n't being __`particularly forthcoming`__ about his hang-ups .                                |
| **`pcc_eng_13_005.9905_x0080443_37:08-10-11`**  | Editors , producers , and journalists have not been __`particularly forthcoming`__ in acknowledging to their own viewers , the media-consuming public , the degree to which the object of their reporting dictates its content and composition . |
| **`pcc_eng_11_098.2159_x1573678_117:14-16-17`** | It 's just a little bit slower , and the Trump camp has not been __`particularly forthcoming`__ in terms of providing witnesses and at least some information .                                                                                  |


9. _particularly surprising_

|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                                                               |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_15_002.1532_x0018421_11:1-6-7`**    | Neither of those departures are __`particularly surprising`__ , though , especially given the level of competition in midfield at PSG .                                                                                                                                                                                                                                                   |
| **`nyt_eng_20000104_0364_3:4-5-6`**            | so there was nothing __`particularly surprising`__ , or for that matter offensive , about Mayor Rudolph Giuliani 's high profile during the televised New Year 's Eve festivities in Times Square , or about the White House 's decision to have Hillary Rodham Clinton join President Clinton in delivering a televised greeting to the nation from the Oval Office on New Year 's Day . |
| **`pcc_eng_11_019.7981_x0304036_18:4-5-6`**    | So it 's not __`particularly surprising`__ that reducing our screen time increases our attention span .                                                                                                                                                                                                                                                                                   |
| **`pcc_eng_08_040.8491_x0644957_18:15-16-17`** | " The findings in the New York State Attorney General 's Office report are not __`particularly surprising`__ .                                                                                                                                                                                                                                                                            |
| **`pcc_eng_13_006.3289_x0086002_06:3-4-5`**    | That is n't __`particularly surprising`__ .                                                                                                                                                                                                                                                                                                                                               |


10. _particularly religious_

|                                                 | `token_str`                                                                                                                              |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_28_079.0946_x1263230_058:08-10-11`** | His father was surprised ; Patrick had never been __`particularly religious`__ and he had his whole life ahead of him .                  |
| **`pcc_eng_15_098.0483_x1568580_47:3-4-5`**     | I 'm not __`particularly religious`__ , I just notice how this guy seems to be really popular with Latin-American Catholics ?            |
| **`pcc_eng_03_035.6528_x0561291_08:08-09-10`**  | My daughter tells me Bear's Den is n't __`particularly religious`__ , maybe not at all , though they do use religious imagery at times . |
| **`apw_eng_20020831_0461_13:13-14-15`**         | Breeden appreciated the gesture , even though she said her family is not __`particularly religious`__ .                                  |
| **`pcc_eng_02_042.2977_x0668099_32:09-10-11`**  | He was a meek man , he has n't __`particularly religious`__ .                                                                            |


11. _particularly new_

|                                                | `token_str`                                                                                                                                            |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_23_088.0686_x1407113_38:1-6-7`**    | None of these features are __`particularly new`__ or surprising , especially to veterans of the racing genre , but it 's good to have them available . |
| **`pcc_eng_08_002.1047_x0017906_11:08-09-10`** | The concept of the smart TV is n't __`particularly new`__ .                                                                                            |
| **`pcc_eng_05_042.7741_x0676220_108:3-8-9`**   | Mundie : None of these things are __`particularly new`__ or unanticipated .                                                                            |
| **`pcc_eng_15_095.6456_x1529817_16:20-21-22`** | Senate Intelligence Vice Chair Saxby Chambliss ( R- GA ) backed up Feinstein , saying , " This is nothing __`particularly new`__ .                     |
| **`pcc_eng_27_028.1944_x0439194_13:2-3-4`**    | So nothing __`particularly new`__ here , storywise .                                                                                                   |


12. _particularly wrong_

|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_13_083.0944_x1326972_097:3-4-5`**   | There 's nothing __`particularly wrong`__ with Jenny 's personality , but George silently but perceptibly judges her for everything she says and does : her description of his work as " elevator music , " her casual , offhanded utterance of the word " fuck , " her obsession with timing their lovemaking to a cherished recording of Ravel 's Bolero , her constant smoking of joints to get in the mood , her habit of casually dropping revelations about her sexual awakening into conversation . |
| **`nyt_eng_20070303_0165_44:4-5-6`**           | `` There is nothing __`particularly wrong`__ with loss mitigation , '' Rosner said .                                                                                                                                                                                                                                                                                                                                                                                                                       |
| **`pcc_eng_21_090.2707_x1442773_41:08-09-10`** | THE OTHER HALF : Houston Dynamo - Nothing __`particularly wrong`__ with this side , just nothing exciting either .                                                                                                                                                                                                                                                                                                                                                                                         |
| **`pcc_eng_23_006.8833_x0095075_13:3-4-5`**    | There 's nothing __`particularly wrong`__ and there are some good funny moments , mostly based around the duo's altered perceptions - for some reason , Nick appears to the living as an elderly Chinese man ( James Hong ) , whereas Roy is a very sexy blonde Russian woman ( Marisa Miller ) .                                                                                                                                                                                                          |
| **`pcc_eng_24_022.7004_x0350623_10:3-4-5`**    | There is nothing __`particularly wrong`__ with this as a hobby , which is to say , a means of amusement , but is it not a method of producing a meaningful quantity of food !                                                                                                                                                                                                                                                                                                                              |


13. _particularly remarkable_

|                                                | `token_str`                                                                                                                                                                        |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_20_039.8203_x0627381_11:18-19-20`** | Plot-wise , very little happens in the film 's first half , and what happens post-intermission is n't __`particularly remarkable`__ .                                              |
| **`pcc_eng_00_039.3781_x0619973_113:6-7-8`**   | That was unusual , but not __`particularly remarkable`__ , nor was their anatomical composition - heavier than any deer or horse he 'd seen , but still roughly similar in shape . |
| **`pcc_eng_04_051.4067_x0814326_32:5-6-7`**    | The stories themselves are not __`particularly remarkable`__ , but the writing is straightforward and crisp .                                                                      |
| **`pcc_eng_28_077.9220_x1244355_08:6-8-9`**    | In itself , this would not be __`particularly remarkable`__ since the discovery of expoplanets has become quite a regular occurrence in the world of astronomy .                   |
| **`pcc_eng_22_073.6964_x1174918_04:3-4-5`**    | There 's nothing __`particularly remarkable`__ about it , no wacky laws of physics , no horrendous fauna , no incredibly advanced civilization .                                   |


14. _particularly original_

|                                                | `token_str`                                                                                                                                                                                                                                                             |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_23_039.9215_x0628904_41:4-5-6`**    | The music is n't __`particularly original`__ , but works well with the film 's tone , and is a nearly constant aural element -- blended quietly under the more expositional scenes , and used as a nice exclamation point during the most bombastic montage sequences . |
| **`pcc_eng_03_012.7429_x0189913_12:14-15-16`** | The whole " small town cut off by invisible dome " plot is n't __`particularly original`__ .                                                                                                                                                                            |
| **`pcc_eng_02_032.0976_x0503226_029:3-4-5`**   | There 's nothing __`particularly original`__ or insightful to set this teen - dystopia thriller apart from the crowd , but strong characters will build some anticipation for the next instalment in the franchise .                                                    |
| **`pcc_eng_08_079.4581_x1270500_16:7-8-9`**    | Huntington 's new schtick was the not __`particularly original`__ idea that the Post- Cold War world was polarizing according to ' the interactions around seven or eight major civilizations ' .                                                                       |
| **`pcc_eng_05_087.6358_x1401837_14:09-10-11`** | As for the political satire , it 's not __`particularly original`__ -- posh cabinet ministers , austerity , ' we 're all in this together ' -- but it 's cheeky and pitched at just the right level .                                                                   |


15. _particularly close_

|                                                 | `token_str`                                                                                                                                                  |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_09_038.3888_x0605091_59:3-4-5`**     | We 're not __`particularly close`__ but I know they do love me and that I 'd be inflicting an awful lot of pain if I told them .                             |
| **`pcc_eng_11_060.0048_x0954812_04:15-16-17`**  | ' Cisco remains the biggest single player in enterprise networking , and it 's not __`particularly close`__ . '                                              |
| **`pcc_eng_13_004.8641_x0062337_066:11-13-14`** | Sure , turnout was low in both contests , but neither was __`particularly close`__ .                                                                         |
| **`pcc_eng_00_065.0814_x1036080_05:17-18-19`**  | Day three , the final day , has just wrapped up and the final score was not __`particularly close`__ , although round 2 proved to be a bit more exciting .   |
| **`pcc_eng_23_088.1039_x1407719_16:07-09-10`**  | Gerbic , who said he had n't been __`particularly close`__ to either sibling , said he never noticed anything odd or antagonistic about their relationship . |


16. _particularly athletic_

|                                                | `token_str`                                                                                                                                                                                                                                                               |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_067.8447_x1080213_202:6-7-8`**   | Hes 6 - 7 , not __`particularly athletic`__ , and has just an average skill - set offensively as far as his position in the NBA is concerned .                                                                                                                            |
| **`pcc_eng_06_070.3981_x1122908_04:25-26-27`** | Hile , a contracts and grants administrator in the computer science and engineering department at the University of California in San Diego , was n't __`particularly athletic`__ as a child .                                                                            |
| **`pcc_eng_12_030.2209_x0473137_24:3-5-6`**    | She had never been __`particularly athletic`__ but loved nature and started hiking while working in the Grand Canyon .                                                                                                                                                    |
| **`pcc_eng_03_039.5625_x0624714_18:3-4-5`**    | She was not __`particularly athletic`__ and was a bit prone to be nervous or shy in new situations , especially in situations where there was a physical risk .                                                                                                           |
| **`pcc_eng_00_039.9610_x0629353_15:52-53-54`** | I am a black belt , and I spend most of my time when I am practicing taekwondo around adults , many of which are black belts , I often forget how hard it can be to learn good form for kicks , especially if you are a young child and not __`particularly athletic`__ . |


17. _particularly flashy_

|                                                 | `token_str`                                                                                                                                                                                                                                    |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_029.5903_x0462387_04:3-4-5`**     | There 's nothing __`particularly flashy`__ about Illume Bistro , other than a menu composed by celebrity chef Bradley Ogden and an interior lighted , in places , by rainbow colors that rotate from yellow to blue to lavender to raspberry . |
| **`pcc_eng_10_074.2718_x1184341_18:4-5-6`**     | The animation is n't __`particularly flashy`__ , and looks more like a children 's book come to life than a big adventure film , but it suits the characters and the story beautifully .                                                       |
| **`pcc_eng_20_033.6695_x0528216_105:22-23-24`** | Karabacek 's offensive game is more north - south than east-west , which works well for the scoring winger who is not __`particularly flashy`__ .                                                                                              |
| **`pcc_eng_20_033.9119_x0532125_04:09-11-12`**  | All of the members play their instruments skillfully without being __`particularly flashy`__ , while singer / guitarist Arthur Shepherd has a big singing voice that ranges from gravelly rasp to soaring howl .                               |
| **`pcc_eng_14_081.6195_x1303328_05:4-6-7`**     | The Wire has never been __`particularly flashy`__ or self-conscious , which owes something to its roots in no-nonsense , unembellished journalism .                                                                                            |


Saving Samples in /share/compling/projects/sanpi/results/top_AM/neg_bigram_examples/particularly/...

Samples saved as...
1. `neg_bigram_examples/particularly/particularly_likable_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_likeable_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_novel_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_comfortable_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_revolutionary_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_radical_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_fast_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_forthcoming_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_surprising_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_religious_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_new_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_wrong_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_remarkable_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_original_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_close_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_athletic_99ex.csv`
1. `neg_bigram_examples/particularly/particularly_flashy_99ex.csv`

## 10. *inherently*

|                        |        `N` |      `f1` |   `adv_total` |
|:-----------------------|-----------:|----------:|--------------:|
| **NEGMIR_inherently**  |  1,701,929 |   291,732 |         5,133 |
| **NEGATED_inherently** | 72,839,589 | 3,173,660 |        47,803 |


|                                   |   `f` |   `adj_total` |   `dP1` |   `LRC` |     `G2` |   `exp_f` |   `unexp_f` |   `f2` |
|:----------------------------------|------:|--------------:|--------:|--------:|---------:|----------:|------------:|-------:|
| **NEGmir~inherently_improper**    |    18 |           142 |    0.83 |    1.63 |    63.49 |      3.09 |       14.91 |     18 |
| **NEGmir~inherently_illegal**     |    26 |           937 |    0.79 |    2.10 |    83.54 |      4.63 |       21.37 |     27 |
| **NEGany~inherently_wrong**       | 1,639 |       149,064 |    0.66 |    5.36 | 7,535.97 |    100.87 |    1,538.13 |  2,315 |
| **NEGmir~inherently_wrong**       | 1,513 |        20,880 |    0.66 |    4.08 | 3,787.53 |    313.00 |    1,200.00 |  1,826 |
| **NEGmir~inherently_bad**         |   148 |        10,261 |    0.62 |    2.86 |   342.53 |     32.23 |      115.77 |    188 |
| **NEGany~inherently_illegal**     |    59 |        30,194 |    0.60 |    3.51 |   252.59 |      4.01 |       54.99 |     92 |
| **NEGmir~inherently_better**      |    44 |        14,013 |    0.57 |    1.65 |    93.95 |     10.11 |       33.89 |     59 |
| **NEGany~inherently_bad**         |   794 |       429,537 |    0.56 |    4.62 | 3,270.68 |     56.95 |      737.05 |  1,307 |
| **NEGmir~inherently_evil**        |    58 |         1,271 |    0.41 |    1.18 |    85.70 |     16.97 |       41.03 |     99 |
| **NEGany~inherently_negative**    |    75 |        53,385 |    0.35 |    2.50 |   222.63 |      8.41 |       66.59 |    193 |
| **NEGany~inherently_evil**        |   358 |        22,706 |    0.28 |    2.81 |   905.80 |     48.93 |      309.07 |  1,123 |
| **NEGany~inherently_good**        |   283 |     1,681,795 |    0.20 |    2.21 |   554.72 |     51.28 |      231.72 |  1,177 |
| **NEGany~inherently_problematic** |    58 |        33,408 |    0.17 |    1.17 |    98.70 |     12.07 |       45.93 |    277 |


1. _inherently improper_

|                                                | `token_str`                                                                                                                                                                                                            |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_00_034.9912_x0549040_3:23-24-25`**  | However , the court decided to take the approach of a substantial minority of states , and concluded that " it is not __`inherently improper`__ for a court to consider the possibility of inheritance in some cases . |
| **`nyt_eng_20070714_0032_29:7-8-9`**           | there is , of course , nothing __`inherently improper`__ about flipping properties for profit ; witness the spate of television shows like `` Flip That House . ''                                                     |
| **`pcc_eng_12_082.9270_x1323663_15:13-14-15`** | But even if we concede that counting the world 's Lutherans is not __`inherently improper`__ , doubts remain as to whether it is possible .                                                                            |
| **`pcc_eng_test_2.01516_x18509_38:3-4-5`**     | There is nothing __`inherently improper`__ about a subject in a criminal investigation seeking a pardon from a president given the president 's wide latitude in granting them .                                       |
| **`pcc_eng_07_020.9121_x0321975_55:16-17-18`** | Law firms advance expenses for clients as a matter of course , so there 's nothing __`inherently improper`__ about a lawyer covering a particular payment and then being reimbursed for it .                           |


2. _inherently illegal_

|                                                | `token_str`                                                                                                                                                                          |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19980311_0164_15:5-6-7`**           | becoming a monopoly is not __`inherently illegal`__ .                                                                                                                                |
| **`pcc_eng_15_097.1646_x1554298_14:5-6-7`**    | A prohibited device is not __`inherently illegal`__ in Canada but it does require an uncommon and very specific prohibited device license for its possession , use , and transport . |
| **`pcc_eng_03_005.3203_x0069828_46:10-11-12`** | Taking that difference one step further , there was nothing __`inherently illegal`__ in the quest for information on Manafort and how that might link Donald Trump to Russia .       |
| **`pcc_eng_12_065.4099_x1041138_10:09-10-11`** | Unlike American mafia organizations , the yakuza are not __`inherently illegal`__ despite their various illegal activities .                                                         |
| **`pcc_eng_14_038.6784_x0608659_17:19-20-21`** | The foundation , while accepting his resignation , quoted from the indictment which said that " Bitcoin are not __`inherently illegal`__ and have known legitimate uses . "          |


3. _inherently wrong_

|                                              | `token_str`                                                                                                                                                                                                                                  |
|:---------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_17_070.5626_x1123983_40:4-5-6`**  | " There 's nothing __`inherently wrong`__ with a core review -- but you have to do it right . "                                                                                                                                              |
| **`pcc_eng_25_081.9758_x1310817_030:5-6-7`** | Moreover , there 's nothing __`inherently wrong`__ with current and potential college students earning money through part-time jobs , be it jobs that involve moving furniture , frothing milk , or hammering forehands down the line .      |
| **`pcc_eng_18_001.9162_x0014936_013:3-4-5`** | There is nothing __`inherently wrong`__ with making a car from stainless steel -- the material was n't the problem .                                                                                                                         |
| **`pcc_eng_05_004.5268_x0057351_11:5-6-7`**  | While there is certainly nothing __`inherently wrong`__ with anti-racist activism of any stripe , there is very little that can be learned from how Class Three Racism affects the most powerful class of African Americans .                |
| **`pcc_eng_23_030.6981_x0479490_22:4-5-6`**  | There may be nothing __`inherently wrong`__ with the syrup bottles that flavor up several liters of seltzer , but Caps are cleaner , easier , and more in tune with what consumers associate with single -serve premium beverage platforms . |


4. _inherently bad_

|                                                 | `token_str`                                                                                                                                                                                                      |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_11_068.6992_x1095841_220:08-13-14`** | And in each case , it is n't that any choice is __`inherently bad`__ , it is that it depends on what we are prepared for , what skills we want to emphasize , what balance we hope to find .                     |
| **`pcc_eng_11_099.3683_x1592377_45:3-4-5`**     | There 's nothing __`inherently bad`__ about the game , but nothing really stands out as fresh either .                                                                                                           |
| **`pcc_eng_22_071.8210_x1144668_18:08-09-10`**  | It 's a shame because there 's nothing __`inherently bad`__ about 3D , and yet after eight years of what should have been the technology 's boom , few filmmakers and studios were able to do anything with it . |
| **`pcc_eng_17_076.9586_x1227494_01:4-5-6`**     | Insurance companies are not __`inherently bad`__ or inherently good .                                                                                                                                            |
| **`pcc_eng_24_029.3144_x0457872_14:3-8-9`**     | It 's not that high-mileage running is __`inherently bad`__ .                                                                                                                                                    |


5. _inherently better_

|                                              | `token_str`                                                                                                                                                                                         |
|:---------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_05_084.9393_x1358382_20:3-4-5`**  | Professionals are not __`inherently better`__ or smarter than amateurs , but they are less likely to be subject to perverse incentives that reward them for being grossly wrong over and over .     |
| **`pcc_eng_15_043.9799_x0694897_69:3-4-5`**  | One is n't __`inherently better`__ than any other , but breaking down stigmas and misconceptions around entrepreneurship allows people to more freely move to where their talents are best suited . |
| **`pcc_eng_29_001.1996_x0003206_30:3-4-5`**  | A is not __`inherently better`__ than Realtor B .                                                                                                                                                   |
| **`pcc_eng_val_1.9320_x15122_12:09-10-11`**  | It alluded to a more traditional -- if not __`inherently better`__ -- style of filmmaking , with models and replicas in lieu of ' dense ' CGI .                                                     |
| **`pcc_eng_09_038.1910_x0601901_146:4-5-6`** | One technique is not __`inherently better`__ than the other and each depends upon how you present and your dentist 's skills .                                                                      |


6. _inherently evil_

|                                                 | `token_str`                                                                                                                                   |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_09_002.8254_x0029540_07:14-15-16`**  | First , this portion of Scripture is clear that the physical world is not __`inherently evil`__ .                                             |
| **`pcc_eng_22_013.1966_x0196846_084:11-12-13`** | Because maybe , while nature in and of itself is n't __`inherently evil`__ , humanity is ?                                                    |
| **`pcc_eng_20_036.9435_x0581030_094:20-21-22`** | Violets need to realize that being prosperous can enable them to accomplish their missions here and that money is not __`inherently evil`__ . |
| **`pcc_eng_18_045.9761_x0727903_134:3-4-5`**    | There is nothing __`inherently evil`__ about snakes or any other animal for that matter .                                                     |
| **`pcc_eng_03_003.7486_x0044244_56:7-8-9`**     | He is a marvelous antagonist , not __`inherently evil`__ , but deluded in his attempts to justify his actions .                               |


7. _inherently negative_

|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_00_014.6210_x0220029_30:10-11-12`** | You see , each one of those failures is not __`inherently negative`__ .                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| **`pcc_eng_05_081.8953_x1309315_05:26-27-28`** | In my previous post I discussed how to emotions could be used in magic and made the point that an emotion such as anger is not __`inherently negative`__ .                                                                                                                                                                                                                                                                                                                                                   |
| **`pcc_eng_02_004.9962_x0064467_037:5-6-7`**   | It is neither insidious nor __`inherently negative`__ by artifice ; rather , it is the most natural of sensibilities , arising from a knowledge that reliance upon one another not only acknowledges and validates the vows of matrimony , but moreover , the eternal commitment each makes to the other forever forges the bonds of undiluted friendship , like kindred spirits floating in some ethereal universe unperturbed by distractions of consternation consecrated upon the altar of destruction . |
| **`pcc_eng_23_006.6814_x0091752_58:7-8-9`**    | Anger , for example , is not __`inherently negative`__ .                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| **`pcc_eng_10_028.9549_x0451869_10:20-21-22`** | This is because unemployment checks are a type of benefit intended for individuals who need it ; there 's nothing __`inherently negative`__ about seeking a financial cushion following job loss .                                                                                                                                                                                                                                                                                                           |


8. _inherently good_

|                                                | `token_str`                                                                                                                        |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_087.1201_x1392819_060:4-5-6`**   | Debt itself is not __`inherently good`__ or bad .                                                                                  |
| **`pcc_eng_18_007.8311_x0110590_11:20-24-25`** | You are always there to aid in times of need , but you are also smart enough to realize not all people are __`inherently good`__ . |
| **`pcc_eng_01_067.2942_x1072371_43:5-6-7`**    | Note that this is not __`inherently good`__ or bad .                                                                               |
| **`pcc_eng_09_004.6583_x0059430_05:3-4-5`**    | Abstractions are n't __`inherently good`__ .                                                                                       |
| **`pcc_eng_24_105.6788_x1693618_099:7-8-9`**   | The problem is , people are n't __`inherently good`__ .                                                                            |


9. _inherently problematic_

|                                                 | `token_str`                                                                                                            |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_11_013.3608_x0199808_21:3-8-9`**     | It 's not that the cross-cutting is __`inherently problematic`__ .                                                     |
| **`pcc_eng_03_004.1230_x0050261_23:14-15-16`**  | As noted at the beginning of this article , multi-level marketing organizations are n't __`inherently problematic`__ . |
| **`pcc_eng_04_006.5718_x0090170_21:5-6-7`**     | Patent pools themselves are n't __`inherently problematic`__ .                                                         |
| **`pcc_eng_28_027.5897_x0429485_074:08-09-10`** | Balanced literacy as an instructional model is not __`inherently problematic`__ .                                      |
| **`pcc_eng_25_041.4217_x0654254_40:04-09-10`**  | " It 's not that pre-made forms are __`inherently problematic`__ , " she says .                                        |


Saving Samples in /share/compling/projects/sanpi/results/top_AM/neg_bigram_examples/inherently/...

Samples saved as...
1. `neg_bigram_examples/inherently/inherently_improper_99ex.csv`
1. `neg_bigram_examples/inherently/inherently_illegal_99ex.csv`
1. `neg_bigram_examples/inherently/inherently_wrong_99ex.csv`
1. `neg_bigram_examples/inherently/inherently_bad_99ex.csv`
1. `neg_bigram_examples/inherently/inherently_better_99ex.csv`
1. `neg_bigram_examples/inherently/inherently_evil_99ex.csv`
1. `neg_bigram_examples/inherently/inherently_negative_99ex.csv`
1. `neg_bigram_examples/inherently/inherently_good_99ex.csv`
1. `neg_bigram_examples/inherently/inherently_problematic_99ex.csv`

## 11. *terribly*

|                      |        `N` |      `f1` |   `adv_total` |
|:---------------------|-----------:|----------:|--------------:|
| **NEGATED_terribly** | 72,839,589 | 3,173,660 |        58,964 |
| **NEGMIR_terribly**  |  1,701,929 |   291,732 |         4,610 |


|                                 |   `f` |   `adj_total` |   `dP1` |   `LRC` |     `G2` |   `exp_f` |   `unexp_f` |   `f2` |
|:--------------------------------|------:|--------------:|--------:|--------:|---------:|----------:|------------:|-------:|
| **NEGany~terribly_surprising**  |   949 |        70,540 |    0.94 |    8.66 | 5,794.10 |     42.00 |      907.00 |    964 |
| **NEGany~terribly_uncommon**    |   103 |        11,312 |    0.94 |    6.33 |   625.85 |      4.57 |       98.43 |    105 |
| **NEGany~terribly_likely**      |   108 |       890,421 |    0.93 |    6.26 |   649.50 |      4.84 |      103.16 |    111 |
| **NEGany~terribly_productive**  |    64 |       102,361 |    0.91 |    5.39 |   376.84 |      2.92 |       61.08 |     67 |
| **NEGany~terribly_original**    |   199 |        37,594 |    0.90 |    6.41 | 1,150.48 |      9.24 |      189.76 |    212 |
| **NEGany~terribly_unusual**     |   146 |        71,234 |    0.90 |    6.17 |   847.05 |      6.75 |      139.25 |    155 |
| **NEGany~terribly_reliable**    |    51 |        90,598 |    0.90 |    4.99 |   296.70 |      2.35 |       48.65 |     54 |
| **NEGany~terribly_impressive**  |   196 |       178,051 |    0.86 |    5.97 | 1,087.64 |      9.50 |      186.50 |    218 |
| **NEGany~terribly_sure**        |    69 |       262,825 |    0.86 |    5.08 |   386.31 |      3.31 |       65.69 |     76 |
| **NEGany~terribly_different**   |   366 |       825,838 |    0.85 |    6.33 | 2,022.47 |     17.82 |      348.18 |    409 |
| **NEGmir~terribly_surprising**  |    67 |         2,662 |    0.83 |    4.07 |   236.35 |     11.48 |       55.52 |     67 |
| **NEGmir~terribly_original**    |    45 |         1,555 |    0.83 |    3.40 |   158.74 |      7.71 |       37.29 |     45 |
| **NEGmir~terribly_unusual**     |    24 |         2,302 |    0.83 |    2.23 |    84.66 |      4.11 |       19.89 |     24 |
| **NEGmir~terribly_special**     |    24 |        15,541 |    0.83 |    2.23 |    84.66 |      4.11 |       19.89 |     24 |
| **NEGmir~terribly_popular**     |    19 |         5,668 |    0.83 |    1.75 |    67.02 |      3.26 |       15.74 |     19 |
| **NEGmir~terribly_clear**       |    15 |         6,722 |    0.83 |    1.20 |    52.91 |      2.57 |       12.43 |     15 |
| **NEGmir~terribly_remarkable**  |    14 |         3,238 |    0.83 |    1.03 |    49.38 |      2.40 |       11.60 |     14 |
| **NEGmir~terribly_new**         |    69 |        12,836 |    0.80 |    3.66 |   225.93 |     12.17 |       56.83 |     71 |
| **NEGmir~terribly_interested**  |    39 |         8,255 |    0.74 |    2.36 |   112.46 |      7.37 |       31.63 |     43 |
| **NEGmir~terribly_interesting** |    56 |        12,447 |    0.68 |    2.44 |   145.16 |     11.31 |       44.69 |     66 |


1. _terribly surprising_

|                                                | `token_str`                                                                                                                                                                                                                               |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_034.2598_x0538167_39:3-4-5`**    | Revolutions are not __`terribly surprising`__ in cases like these .                                                                                                                                                                       |
| **`pcc_eng_08_076.9677_x1229986_32:5-6-7`**    | Interesting news , but not __`terribly surprising`__ .                                                                                                                                                                                    |
| **`pcc_eng_11_016.0291_x0243237_28:3-4-5`**    | It 's not __`terribly surprising`__ that the deadbeats of Europe want access to the money of German taxpayers , but it is rather shocking that German politicians are willing to play this no-win game .                                  |
| **`pcc_eng_26_094.9685_x1519625_01:30-31-32`** | In retrospect , it should have seemed like the perfect fit and by proxy something that we should have expected so really , the announcement of Scribblenauts while perhaps not __`terribly surprising`__ , is still nonetheless welcome . |
| **`pcc_eng_15_009.4319_x0136213_06:6-7-8`**    | That 's why it 's not __`terribly surprising`__ that the New York Post is reporting that ESPN baseball analyst Steve Phillips has been making some really bad calls in his personal life .                                                |


2. _terribly uncommon_

|                                                | `token_str`                                                                                                                                                                                                                               |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_28_026.9821_x0419639_32:3-4-5`**    | Plums are n't __`terribly uncommon`__ , but not a lot of people are familiar with pluots .                                                                                                                                                |
| **`pcc_eng_22_084.5345_x1350322_60:40-41-42`** | Naturally given how powerless inmates are to do anything about it , cases of identify theft , the entrusted person running off with all the inmate 's money and not paying any of the bills and the like is n't __`terribly uncommon`__ . |
| **`pcc_eng_14_031.3481_x0490431_40:3-4-5`**    | It 's not __`terribly uncommon`__ for the entire party to be decimated in a seemingly - routine random encounter , especially if they 're just trying to breeze their way through an encounter .                                          |
| **`pcc_eng_11_064.8935_x1034009_14:5-6-7`**    | Although lunar-planetary partnerships are not __`terribly uncommon`__ , Boberg said it is the specific planet 's presence that makes this an occasion to note .                                                                           |
| **`pcc_eng_20_081.8641_x1306459_03:3-4-5`**    | This is n't __`terribly uncommon`__ , but this also was a school nightmare .                                                                                                                                                              |


3. _terribly likely_

|                                                | `token_str`                                                                                                                                                                                                                   |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_028.0402_x0437306_18:09-10-11`** | It 's slightly more possible -- but still not __`terribly likely`__ -- that Whedon will resurrect his dead script Goners , which was a Buffy - esque story of a young woman who gains superpowers and faces ultimate horror . |
| **`pcc_eng_02_092.0224_x1471611_38:6-7-8`**    | But then , victory is n't __`terribly likely`__ .                                                                                                                                                                             |
| **`pcc_eng_26_004.7209_x0059956_32:3-4-5`**    | This is not __`terribly likely`__ to help reduce long-term , chronic anxiety , but it is unlikely to hurt either .                                                                                                            |
| **`pcc_eng_03_046.4181_x0735823_07:6-7-8`**    | In fact , that 's not __`terribly likely`__ .                                                                                                                                                                                 |
| **`pcc_eng_09_007.8954_x0111744_06:3-4-5`**    | It 's not __`terribly likely`__ that he 's going to be a successful NHL player .                                                                                                                                              |


4. _terribly productive_

|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                            |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_06_026.8551_x0418312_41:20-22-23`**  | Purdue was far more desperate in the secondary a year ago , and Williams had gone his whole career without being __`terribly productive`__ at his position of choice .                                                                                                                                                                 |
| **`pcc_eng_19_079.8810_x1274310_023:10-11-12`** | So I 'm not doing much or at least nothing __`terribly productive`__ : trying to solve a lot of computer problems , dealing with a lot of broken appliances and thus shopping for replacements ( ick ) , a lot of having to deal with ornery contractors , a couple of doctor appointments , air bag replacements , things like that . |
| **`pcc_eng_00_005.4608_x0072059_16:22-23-24`**  | I posted to Facebook that my air conditioner always has reminded me of a mediocre co-worker : inefficient , cranky , not __`terribly productive`__ , and doing just enough to get by .                                                                                                                                                 |
| **`pcc_eng_08_070.8614_x1131127_15:08-09-10`**  | Even with the home runs he 's not __`terribly productive`__ , some team will find a place for him given his versatility and defense but they should n't expect much offensively .                                                                                                                                                      |
| **`pcc_eng_24_077.0033_x1229492_007:34-36-37`** | We also , a few years ago , had an opinion in terms of transatlantic relations , but our experience in terms of looking at things from a transatlantic point of view has not been __`terribly productive`__ .                                                                                                                          |


5. _terribly reliable_

|                                                | `token_str`                                                                                                                                                                                                             |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20050227_0090_4:1-7-8`**            | not that the suicide watch was __`terribly reliable`__ ; it depended in part on inmates paid 39 cents an hour to check on their suicidal peers .                                                                        |
| **`pcc_eng_18_001.6036_x0009816_099:7-8-9`**   | In my experience , psychics are not __`terribly reliable`__ , but they sometimes provide valid data points along with what they acquire through your answering their questions or what their own imagination produced . |
| **`pcc_eng_21_093.6106_x1496377_02:6-7-8`**    | But the six-legged soldiers were n't __`terribly reliable`__ by JOSEPH TREVITHICK Mao Tse -Tung famously wrote in On Guerrilla Warfare that guerrillas are proverbial fish who have ...                                 |
| **`pcc_eng_22_008.7482_x0124989_03:20-22-23`** | We all love Sigourney Weaver , even when she stars in crap like The Assignment , but she has n't been __`terribly reliable`__ when it comes to these Avatar sequels .                                                   |
| **`pcc_eng_11_016.2035_x0246016_25:14-18-19`** | Many of us depend on public transit in its many forms , and none of it is __`terribly reliable`__ .                                                                                                                     |


6. _terribly unusual_

|                                               | `token_str`                                                                                                                                                                                                                                                                                |
|:----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_08_051.5427_x0818147_38:5-6-7`**   | And 10 months is not __`terribly unusual`__ in mathematics ...                                                                                                                                                                                                                             |
| **`pcc_eng_08_086.0228_x1376438_03:4-6-7`**   | That development would not be __`terribly unusual`__ .                                                                                                                                                                                                                                     |
| **`pcc_eng_13_088.4614_x1413593_1:35-36-37`** | Display Wifi Password in Mac via Terminal Given the complexity of some wi-fi network passwords combined with the general infrequency of entering them and that they 're typically saved on use , it 's not __`terribly unusual`__ to forget what a specific routers wireless password is . |
| **`pcc_eng_01_044.4064_x0701249_14:3-4-5`**   | This is n't __`terribly unusual`__ in itself ; since 1952 , Congress has established a " national day of prayer " and most other states do the same thing .                                                                                                                                |
| **`pcc_eng_27_007.1798_x0099343_07:3-5-6`**   | It would not be __`terribly unusual`__ to see a 70 year old woman sans brassiere strut by in the latest fashions , sporting an air that says she owns the sidewalk and possibly much of this part of town .                                                                                |


7. _terribly original_

|                                                | `token_str`                                                                                                                                                                            |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20000109_0032_38:27-29-30`**        | to Stein , the fact that he reveals intimate , seemingly eccentric but actually quite normal aspects of his personality to the readers of Time does not seem __`terribly original`__ . |
| **`pcc_eng_01_097.7344_x1563861_033:2-3-4`**   | While not __`terribly original`__ , I found these characters compelling .                                                                                                              |
| **`pcc_eng_11_083.5065_x1335366_5:5-6-7`**     | Though the name is n't __`terribly original`__ , the fact that there are still a few ways to raft 2 for the price of 1 is .                                                            |
| **`pcc_eng_25_014.7857_x0222916_145:3-4-5`**   | THERE is nothing __`terribly original`__ about Carrillo 's ideas .                                                                                                                     |
| **`pcc_eng_27_006.6762_x0091245_11:13-14-15`** | The movie gets bogged down in its anti-technology message , which is n't __`terribly original`__ .                                                                                     |


8. _terribly sure_

|                                                 | `token_str`                                                                                 |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------|
| **`pcc_eng_28_048.7232_x0772129_19:6-7-8`**     | ( Note : I 'm not __`terribly sure`__ if it was the game that made me feel sick .           |
| **`pcc_eng_12_086.3176_x1378617_35:3-4-5`**     | I 'm not __`terribly sure`__ I engage in this examination .                                 |
| **`pcc_eng_05_015.0048_x0226944_160:11-13-14`** | The minister cannot really provide any information ; he is not really __`terribly sure`__ . |
| **`pcc_eng_04_083.0535_x1325528_2:3-4-5`**      | I 'm not __`terribly sure`__ how many people can actually help me with my question .        |
| **`pcc_eng_22_006.7896_x0093389_215:4-5-6`**    | " I am not __`terribly sure`__ , " I said .                                                 |


9. _terribly impressive_

|                                                 | `token_str`                                                                                                                                                                                              |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_01_099.1827_x1587312_54:18-19-20`**  | Yet even here Kramer is forced to admit in terms of tangible results that his record was not __`terribly impressive`__ .                                                                                 |
| **`pcc_eng_08_072.5256_x1158007_25:28-29-30`**  | It 's possible WWE enthusiasts might thrill to a game that depicts the heroes of the squared circle , though really , the graphics are cartoonish and not __`terribly impressive`__ .                    |
| **`pcc_eng_14_002.4239_x0023104_154:19-20-21`** | So , reluctantly , he went back out and he did his best - which back then was n't __`terribly impressive`__ - to fight back the dark .                                                                   |
| **`pcc_eng_26_036.5383_x0574399_16:08-09-10`**  | The sped- up chase through Paris is n't __`terribly impressive`__ as car chases go , but it 's still an E-type Jaguar chasing Senta Berger through the streets of Sixties Paris , so that 's something ! |
| **`pcc_eng_03_087.0287_x1393111_25:3-4-5`**     | There 's nothing __`terribly impressive`__ about that .                                                                                                                                                  |


10. _terribly different_

|                                                 | `token_str`                                                                                                                                                                                                                                                               |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_11_087.6016_x1401792_48:11-12-13`**  | What we learned was that shooting an indie pilot is not __`terribly different`__ than shooting the micro-budget features that find their way to film festivals . "                                                                                                        |
| **`nyt_eng_20000328_0115_60:5-6-7`**            | the hectic lifestyle is n't __`terribly different`__ from investment banking .                                                                                                                                                                                            |
| **`pcc_eng_05_032.2207_x0505754_117:26-28-29`** | Once the layout is established for this world and we 've kind of blocked in the shots , the work flow for the animators is n't necessarily __`terribly different`__ than what it usually is .                                                                             |
| **`apw_eng_20020731_0871_4:3-4-5`**             | reality was n't __`terribly different`__ , other than the fact that the huge steel boat I arrived in was filled with wet bars and television screens and that my first view of the land was of an enormous parking lot .                                                  |
| **`pcc_eng_03_038.1477_x0601633_10:22-26-27`**  | . . . Also , most of it did n't really look like ninja stuff so much as typical krotty ( not that they 're __`terribly different`__ combatively , but would n't ninja training involve more " remaining unseen " or " moving through difficult terrain " or something ? ) |


11. _terribly special_

|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                                                |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_25_083.0387_x1327858_16:14-15-16`** | Though they act as a high- profile example , the Bush boys are n't __`terribly special`__ in their mixture of sects of Christianity .                                                                                                                                                                                                                                                                                                      |
| **`pcc_eng_07_015.2012_x0229864_04:11-12-13`** | Just your basic folk - rock riff in A ; nothing __`terribly special`__ , right ?                                                                                                                                                                                                                                                                                                                                                           |
| **`pcc_eng_19_071.8832_x1144882_37:5-6-7`**    | The special effects are not __`terribly special`__ , though Les Bowie - Hammer 's top effects man for many years - performed miracles on the tiny budget , but Wordsworth 's shambling pathetic performance is memorable , and the film put the studio back on its feet and encouraged it to follow -on with many more man-into-monster movies , including the direct sequels Quatermass II ( 1957 ) and Quatermass And The Pit ( 1967 ) . |
| **`pcc_eng_11_064.8935_x1034009_12:15-16-17`** | " It 's a full moon and it 's real , but there 's nothing __`terribly special`__ or spectacular about it , " he said .                                                                                                                                                                                                                                                                                                                     |
| **`pcc_eng_22_004.2037_x0051958_047:7-8-9`**   | The chunky sauce was good but not __`terribly special`__ .                                                                                                                                                                                                                                                                                                                                                                                 |


12. _terribly popular_

|                                                 | `token_str`                                                                                                                                                                   |
|:------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_071.1107_x1134121_23:11-12-13`**  | However , it seems like the arcade game itself was n't __`terribly popular`__ and is going away , so maybe there 's plenty of blame to go around .                            |
| **`pcc_eng_09_082.8117_x1323529_068:24-25-26`** | This kind of market theory of artistic value puts our music on a slippery slope , as classical music as a whole is not __`terribly popular`__ with a large , general public . |
| **`nyt_eng_20070526_0069_23:4-5-6`**            | things Californian are not __`terribly popular`__ with Nevada residents , but Schneider says he has a way around that .                                                       |
| **`pcc_eng_17_070.9176_x1129636_14:13-14-15`**  | The non-cabbage bits added pleasant flavor contrast , although the beans were n't __`terribly popular`__ .                                                                    |
| **`pcc_eng_03_003.6324_x0042370_59:14-15-16`**  | I would n't go below two , but right now graveyard decks are n't __`terribly popular`__ .                                                                                     |


13. _terribly clear_

|                                                 | `token_str`                                                                                                                                                                                                               |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_14_038.3868_x0603979_3:4-5-6`**      | " It 's not __`terribly clear`__ to me that my actions were explicitly done for ' peace , ' " Manning wrote in a statement to the Guardian .                                                                              |
| **`pcc_eng_15_014.9744_x0225589_356:17-18-19`** | She also seems to have some kind of bandage across her forehead , though it 's not __`terribly clear`__ given she 's wearing an elaborate hat .                                                                           |
| **`pcc_eng_11_007.4004_x0103608_029:4-5-6`**    | This picture is n't __`terribly clear`__ , but this is a rat hole at the perimeter fence of my chicken run .                                                                                                              |
| **`pcc_eng_09_005.4795_x0072752_067:16-17-18`** | A two -gallon petrol can is fitted to the offside running board and , although not __`terribly clear`__ , there is also an AA badge positioned just ahead of the Motometer ( temperature gauge ) on top of the radiator . |
| **`pcc_eng_17_072.4313_x1154287_17:3-4-5`**     | It 's not __`terribly clear`__ how much of a boost most of the ensemble cast of " The Avengers " got from the film 's giant success .                                                                                     |


14. _terribly remarkable_

|                                                 | `token_str`                                                                                                                                                                                                |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_29_036.6025_x0574483_33:19-20-21`**  | What Price did manage to do was shake off 60 minutes where he was thoroughly outplayed by a not __`terribly remarkable`__ goalie .                                                                         |
| **`pcc_eng_21_019.6636_x0301381_06:6-7-8`**     | By itself , this is n't __`terribly remarkable`__ or exciting , because a single finger touching the surface in question does n't create a signature that 's unique enough for individual identification . |
| **`pcc_eng_27_068.9149_x1098061_42:1-5-6`**     | None of this is __`terribly remarkable`__ in the world of the book .                                                                                                                                       |
| **`pcc_eng_18_010.9384_x0160799_059:08-09-10`** | I have lots of little plans but nothing __`terribly remarkable`__ .                                                                                                                                        |
| **`pcc_eng_13_037.5812_x0591439_11:11-12-13`**  | It 's a good look for the figure , though not __`terribly remarkable`__ .                                                                                                                                  |


15. _terribly new_

|                                                | `token_str`                                                                                                                                                                                                                                                                                                |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20000302_0121_10:7-8-9`**           | it 's a variation on the not __`terribly new`__ concept of the software emulator , a computer program that makes one computer act as if it were another computer .                                                                                                                                         |
| **`pcc_eng_02_048.3682_x0766345_8:1-2-3`**     | Nothing __`terribly new`__ here , except that Musk 's vision is being more widely disseminated .                                                                                                                                                                                                           |
| **`pcc_eng_07_053.2546_x0844707_02:14-15-16`** | Aaaand it 's time for another timely review of a game that 's not __`terribly new`__ ( it 's been out about 6 months now ) .                                                                                                                                                                               |
| **`apw_eng_20090506_0541_38:4-5-6`**           | but it 's not __`terribly new`__ , just getting the word out and labeling it . ''                                                                                                                                                                                                                          |
| **`pcc_eng_29_082.7561_x1320397_15:27-28-29`** | Titles like " Crack Headed Woman , " " I Got A Woman , " and " Oo Wee Pretty Baby " define the territory - nothing __`terribly new`__ or innovative on display - but Hinds and company put their all into every tune , with fervently energetic performances that give every cut an irresistible urgency . |


16. _terribly interested_

|                                                | `token_str`                                                                                                                                                                                                                                                                                                                 |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_11_010.8196_x0158891_21:3-4-5`**    | She 's not __`terribly interested`__ in writing for fun at home , though .                                                                                                                                                                                                                                                  |
| **`pcc_eng_08_075.1040_x1199825_86:3-4-5`**    | Clute is n't __`terribly interested`__ in digital writing .                                                                                                                                                                                                                                                                 |
| **`pcc_eng_25_009.0421_x0130317_09:6-7-8`**    | But Aly & AJ are n't __`terribly interested`__ in upholding expectations :                                                                                                                                                                                                                                                  |
| **`pcc_eng_28_065.2257_x1039269_34:3-4-5`**    | I was n't __`terribly interested`__ by the film when it first came out ; having seen it , I made sure that I 'd catch Catching Fire on the big screen .                                                                                                                                                                     |
| **`pcc_eng_21_016.8492_x0255882_06:16-17-18`** | Sorry , friends back home , but if you 're very honest , you were n't __`terribly interested`__ in hearing any more about this experience ; what 's more , it turns out that after thinking of almost nothing but the same two papers for two months , one really does n't want to think about them for a Good Long While . |


17. _terribly interesting_

|                                                 | `token_str`                                                                                                                                                                                                                                                                    |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_25_042.4799_x0671422_016:07-13-14`** | But such a legal status would not , by itself , be __`terribly interesting`__ philosophically .                                                                                                                                                                                |
| **`pcc_eng_09_002.9666_x0031828_39:11-12-13`**  | Her playing up to that point had been efficient but not __`terribly interesting`__ .                                                                                                                                                                                           |
| **`pcc_eng_11_091.6585_x1467486_29:09-10-11`**  | This , in and of itself , is not __`terribly interesting`__ .                                                                                                                                                                                                                  |
| **`pcc_eng_28_074.0636_x1181707_032:13-15-16`** | What I came to realize about Orleans was that the decisions were n't always __`terribly interesting`__ to me after a while .                                                                                                                                                   |
| **`pcc_eng_05_083.6032_x1336890_060:34-35-36`** | Hader 's Aaron is a perfectly nice , likable guy ( his scene playing one - on- one basketball with Le Bron James is a stone- cold classic ) , but he 's not __`terribly interesting`__ , generous to a fault , and definitely not the kind of guy that Amy would be drawn to . |


Saving Samples in /share/compling/projects/sanpi/results/top_AM/neg_bigram_examples/terribly/...

Samples saved as...
1. `neg_bigram_examples/terribly/terribly_surprising_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_uncommon_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_likely_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_productive_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_reliable_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_unusual_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_original_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_sure_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_impressive_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_different_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_special_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_popular_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_clear_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_remarkable_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_new_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_interested_99ex.csv`
1. `neg_bigram_examples/terribly/terribly_interesting_99ex.csv`
